In [1]:
import numpy as np
import pandas as pd
import functools
import itertools
import warnings
from builtins import range
from collections import defaultdict

import matrixprofile as mp
import numpy as np
import pandas as pd
import stumpy
from matrixprofile.exceptions import NoSolutionPossible
from numpy.linalg import LinAlgError
from scipy.signal import cwt, find_peaks_cwt, ricker, welch
from scipy.stats import linregress
from statsmodels.tools.sm_exceptions import MissingDataError

from tsfresh.utilities.string_manipulation import convert_to_output_format

with warnings.catch_warnings():
    # Ignore warnings of the patsy package
    warnings.simplefilter("ignore", DeprecationWarning)

    from statsmodels.tsa.ar_model import AR

from statsmodels.tsa.stattools import acf, adfuller, pacf

### Loading gyroscope data 

In [2]:
v_g=[]
o_g=[]
p_g=[]
k_g=[]
data_1g=[]
`
with open("D:\\M-Tech-Data-Science\\PROJECT\\dataset\\Phone1_Sensor_org/User9\Training/Gyr.txt") as in_file_g:
    for line_g in in_file_g:
        if line_g.strip() == "Info: ========= Hands Left Corridor Walk Started! ======":
            for line_g in in_file_g:
                if line_g.strip() == "Info: ========= Hands Left Corridor Walk Stopped! ======":
                    break
                #print(line)
                data_1g=line_g
                data_1g = data_1g.strip()
                data_1g = data_1g.split(",")
                #print(data_1)
                v_g.append(data_1g)
                o_g=v_g
p_g=o_g  

In [3]:
p_g=pd.DataFrame(p_g)
p_g.shape

(8109, 5)

### Loading accelerometer data 

In [4]:
v_a=[]
o_a=[]
p_a=[]
k_a=[]
data_1a=[]

with open("D:\\M-Tech-Data-Science\\PROJECT\\dataset\\Phone1_Sensor_org/User9\Training/LinAcc.txt") as in_file_a:
    for line_a in in_file_a:
        if line_a.strip() == "Info: ========= Hands Left Corridor Walk Started! ======":
            for line_a in in_file_a:
                if line_a.strip() == "Info: ========= Hands Left Corridor Walk Stopped! ======":
                    break
                #print(line)
                data_1a=line_a
                data_1a = data_1a.strip()
                data_1a = data_1a.split(",")
                #print(data_1)
                v_a.append(data_1a)
                o_a=v_a
p_a=o_a  

In [5]:
p_a=pd.DataFrame(p_a)
p_a.shape

(8109, 5)

### Input data 

In [6]:
s=pd.concat([p_g, p_a], axis=1,ignore_index=True)
s.columns=['Gx','Gy','Gz','T1g','T2g','Ax','Ay','Az','Timestamp','T2a']
User_1=s.drop(columns=['T1g','T2g','T2a','Timestamp'])
#print(pd.DataFrame(User_1))
User_1=pd.DataFrame(User_1)
User_1

,Gx,Gy,Gz,Ax,Ay,Az
0,0.0489,0.1655,0.9703,-0.014,0.0626,0.7691
1,0.0487,0.1644,0.9705,0.0769,-0.0396,0.2126
2,0.0484,0.1643,0.9706,-0.0602,0.053,-0.4076
3,0.0483,0.1649,0.9706,-0.0434,-0.0259,-0.3333
4,0.0482,0.1651,0.9706,-0.0332,-0.0379,-0.0488
...,...,...,...,...,...,...
8104,-0.006,-0.1043,0.9699,0.1028,-0.2113,-2.8363
8105,-0.0059,-0.1077,0.9695,-0.0262,0.0201,-0.4222
8106,-0.0063,-0.1117,0.969,0.0208,0.1937,3.4226
8107,-0.0061,-0.1102,0.9692,-0.066,0.0829,1.7074


### Windowing 

In [7]:
Gx_list = []
Gy_list = []
Gz_list = []
Ax_list = []
Ay_list = []
Az_list = []
#train_labels = []

window_size =20
step_size = 10

# creating overlaping windows of size window-size 100
for i in range(0, User_1.shape[0]-window_size, step_size):
    Gxs = User_1['Gx'].values[i: i + window_size]
    Gys = User_1['Gy'].values[i: i + window_size]
    Gzs = User_1['Gz'].values[i: i + window_size]
    Axs = User_1['Ax'].values[i: i + window_size]
    Ays = User_1['Ay'].values[i: i + window_size]
    Azs = User_1['Az'].values[i: i + window_size]
    #label = stats.mode(df_train['activity'][i: i + 100])[0][0]

    Gx_list.append(Gxs)
    Gy_list.append(Gys)
    Gz_list.append(Gzs)
    Ax_list.append(Axs)
    Ay_list.append(Ays)
    Az_list.append(Azs)

### X-axis gyroscope 

In [8]:
Gx_list=pd.DataFrame(Gx_list)
Gx_list=Gx_list.astype(float)
#Gx_list
Gx_list_trans=Gx_list.T
#Gx_list_trans['ID'] = 1
Gx_list_trans

,0,1,2,3,4,5,6,7,8,9,...,799,800,801,802,803,804,805,806,807,808
0,0.0489,0.0476,0.0477,0.0480,0.0499,0.0501,0.0514,0.0493,0.0485,0.0504,...,-0.0093,-0.0078,-0.0078,-0.0090,-0.0089,-0.0102,-0.0106,-0.0101,-0.0086,-0.0088
1,0.0487,0.0475,0.0477,0.0478,0.0504,0.0502,0.0510,0.0488,0.0489,0.0506,...,-0.0096,-0.0076,-0.0075,-0.0089,-0.0091,-0.0103,-0.0106,-0.0100,-0.0086,-0.0084
2,0.0484,0.0477,0.0480,0.0480,0.0504,0.0502,0.0502,0.0484,0.0493,0.0507,...,-0.0097,-0.0075,-0.0074,-0.0088,-0.0093,-0.0104,-0.0108,-0.0099,-0.0087,-0.0079
3,0.0483,0.0477,0.0483,0.0482,0.0504,0.0503,0.0497,0.0483,0.0493,0.0508,...,-0.0091,-0.0075,-0.0075,-0.0087,-0.0094,-0.0104,-0.0107,-0.0098,-0.0089,-0.0074
4,0.0482,0.0477,0.0484,0.0484,0.0506,0.0505,0.0493,0.0482,0.0493,0.0510,...,-0.0084,-0.0075,-0.0077,-0.0085,-0.0096,-0.0106,-0.0103,-0.0096,-0.0091,-0.0070
5,0.0482,0.0481,0.0483,0.0487,0.0510,0.0506,0.0493,0.0481,0.0495,0.0509,...,-0.0077,-0.0078,-0.0080,-0.0085,-0.0097,-0.0108,-0.0100,-0.0093,-0.0094,-0.0067
6,0.0482,0.0484,0.0481,0.0489,0.0509,0.0507,0.0492,0.0480,0.0497,0.0504,...,-0.0072,-0.0082,-0.0082,-0.0085,-0.0098,-0.0123,-0.0100,-0.0092,-0.0095,-0.0066
7,0.0479,0.0482,0.0480,0.0491,0.0503,0.0510,0.0493,0.0481,0.0499,0.0506,...,-0.0072,-0.0083,-0.0085,-0.0085,-0.0100,-0.0123,-0.0102,-0.0091,-0.0095,-0.0066
8,0.0477,0.0481,0.0480,0.0494,0.0502,0.0512,0.0496,0.0479,0.0501,0.0508,...,-0.0074,-0.0083,-0.0088,-0.0087,-0.0101,-0.0119,-0.0104,-0.0089,-0.0091,-0.0068
9,0.0478,0.0480,0.0481,0.0495,0.0501,0.0514,0.0494,0.0480,0.0501,0.0508,...,-0.0077,-0.0082,-0.0090,-0.0088,-0.0102,-0.0110,-0.0103,-0.0088,-0.0089,-0.0069


#### feature extraction 

In [9]:
from tsfresh import extract_features
#c=c.fillna(0)
l=[]
m=[]
z=[]
u=[]
X=[]
for i in range ( Gx_list_trans.shape[1]):
    p=Gx_list_trans.iloc[:,i]
    p=pd.DataFrame(p)
    p['ID'] = '1'
    X = extract_features(p, column_id='ID')
    #print(X.shape)
    l.append(X)
    #m.append(l)
    z=l
u=z   

Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.67s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.70s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.78s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.71s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.76s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.67s/it]


In [10]:
fea_user_1gx=np.array(u)
header_gx=['gx_variance_larger_than_standard_deviation',	'gx_has_duplicate_max',	'gx_has_duplicate_min',	'gx_has_duplicate',	'gx_sum_values',	'gx_abs_energy',	'gx_mean_abs_change',	'gx_mean_change',	'gx_mean_second_derivative_central',	'gx_median',	'gx_mean',	'gx_length',	'gx_standard_deviation',	'gx_variation_coefficient',	'gx_variance',	'gx_skewness',	'gx_kurtosis',	'gx_root_mean_square',	'gx_absolute_sum_of_changes',	'gx_longest_strike_below_mean',	'gx_longest_strike_above_mean',	'gx_count_above_mean',	'gx_count_below_mean',	'gx_last_location_of_maximum',	'gx_first_location_of_maximum',	'gx_last_location_of_minimum',	'gx_first_location_of_minimum',	'gx_percentage_of_reoccurring_values_to_all_values',	'gx_percentage_of_reoccurring_datapoints_to_all_datapoints',	'gx_sum_of_reoccurring_values',	'gx_sum_of_reoccurring_data_points',	'gx_ratio_value_number_to_time_series_length',	'gx_sample_entropy',	'gx_maximum',	'gx_minimum',	'gx_benford_correlation',	'gx_time_reversal_asymmetry_statistic__lag_1',	'gx_time_reversal_asymmetry_statistic__lag_2',	'gx_time_reversal_asymmetry_statistic__lag_3',	'gx_c3__lag_1',	'gx_c3__lag_2',	'gx_c3__lag_3',	'gx_cid_ce__normalize_True',	'gx_cid_ce__normalize_False',	'gx_symmetry_looking__r_0.0',	'gx_symmetry_looking__r_0.05',	'gx_symmetry_looking__r_0.1',	'gx_symmetry_looking__r_0.15000000000000002',	'gx_symmetry_looking__r_0.2',	'gx_symmetry_looking__r_0.25',	'gx_symmetry_looking__r_0.30000000000000004',	'gx_symmetry_looking__r_0.35000000000000003',	'gx_symmetry_looking__r_0.4',	'gx_symmetry_looking__r_0.45',	'gx_symmetry_looking__r_0.5',	'gx_symmetry_looking__r_0.55',	'gx_symmetry_looking__r_0.6000000000000001',	'gx_symmetry_looking__r_0.65',	'gx_symmetry_looking__r_0.7000000000000001',	'gx_symmetry_looking__r_0.75',	'gx_symmetry_looking__r_0.8',	'gx_symmetry_looking__r_0.8500000000000001',	'gx_symmetry_looking__r_0.9',	'gx_symmetry_looking__r_0.9500000000000001',	'gx_large_standard_deviation__r_0.05',	'gx_large_standard_deviation__r_0.1',	'gx_large_standard_deviation__r_0.15000000000000002',	'gx_large_standard_deviation__r_0.2',	'gx_large_standard_deviation__r_0.25',	'gx_large_standard_deviation__r_0.30000000000000004',	'gx_large_standard_deviation__r_0.35000000000000003',	'gx_large_standard_deviation__r_0.4',	'gx_large_standard_deviation__r_0.45',	'gx_large_standard_deviation__r_0.5',	'gx_large_standard_deviation__r_0.55',	'gx_large_standard_deviation__r_0.6000000000000001',	'gx_large_standard_deviation__r_0.65',	'gx_large_standard_deviation__r_0.7000000000000001',	'gx_large_standard_deviation__r_0.75',	'gx_large_standard_deviation__r_0.8',	'gx_large_standard_deviation__r_0.8500000000000001',	'gx_large_standard_deviation__r_0.9',	'gx_large_standard_deviation__r_0.9500000000000001',	'gx_quantile__q_0.1',	'gx_quantile__q_0.2',	'gx_quantile__q_0.3',	'gx_quantile__q_0.4',	'gx_quantile__q_0.6',	'gx_quantile__q_0.7',	'gx_quantile__q_0.8',	'gx_quantile__q_0.9',	'gx_autocorrelation__lag_0',	'gx_autocorrelation__lag_1',	'gx_autocorrelation__lag_2',	'gx_autocorrelation__lag_3',	'gx_autocorrelation__lag_4',	'gx_autocorrelation__lag_5',	'gx_autocorrelation__lag_6',	'gx_autocorrelation__lag_7',	'gx_autocorrelation__lag_8',	'gx_autocorrelation__lag_9',	'gx_agg_autocorrelation__f_agg_"mean"__maxlag_40',	'gx_agg_autocorrelation__f_agg_"median"__maxlag_40',	'gx_agg_autocorrelation__f_agg_"var"__maxlag_40',	'gx_partial_autocorrelation__lag_0',	'gx_partial_autocorrelation__lag_1',	'gx_partial_autocorrelation__lag_2',	'gx_partial_autocorrelation__lag_3',	'gx_partial_autocorrelation__lag_4',	'gx_partial_autocorrelation__lag_5',	'gx_partial_autocorrelation__lag_6',	'gx_partial_autocorrelation__lag_7',	'gx_partial_autocorrelation__lag_8',	'gx_partial_autocorrelation__lag_9',	'gx_number_cwt_peaks__n_1',	'gx_number_cwt_peaks__n_5',	'gx_number_peaks__n_1',	'gx_number_peaks__n_3',	'gx_number_peaks__n_5',	'gx_number_peaks__n_10',	'gx_number_peaks__n_50',	'gx_binned_entropy__max_bins_10',	'gx_index_mass_quantile__q_0.1',	'gx_index_mass_quantile__q_0.2',	'gx_index_mass_quantile__q_0.3',	'gx_index_mass_quantile__q_0.4',	'gx_index_mass_quantile__q_0.6',	'gx_index_mass_quantile__q_0.7',	'gx_index_mass_quantile__q_0.8',	'gx_index_mass_quantile__q_0.9',	'gx_cwt_coefficients__coeff_0__w_2__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_0__w_5__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_0__w_10__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_0__w_20__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_w_2__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_w_5__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_w_10__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_w_20__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_2__w_2__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_2__w_5__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_2__w_10__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_2__w_20__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_3__w_2__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_3__w_5__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_3__w_10__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_3__w_20__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_4__w_2__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_4__w_5__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_4__w_10__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_4__w_20__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_5__w_2__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_5__w_5__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_5__w_10__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_5__w_20__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_6__w_2__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_6__w_5__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_6__w_10__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_6__w_20__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_7__w_2__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_7__w_5__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_7__w_10__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_7__w_20__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_8__w_2__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_8__w_5__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_8__w_10__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_8__w_20__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_9__w_2__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_9__w_5__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_9__w_10__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_9__w_20__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_10__w_2__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_10__w_5__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_10__w_10__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_10__w_20__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_1w_2__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_1w_5__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_1w_10__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_1w_20__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_12__w_2__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_12__w_5__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_12__w_10__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_12__w_20__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_13__w_2__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_13__w_5__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_13__w_10__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_13__w_20__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_14__w_2__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_14__w_5__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_14__w_10__widths_(2 5 10 20)',	'gx_cwt_coefficients__coeff_14__w_20__widths_(2 5 10 20)',	'gx_spkt_welch_density__coeff_2',	'gx_spkt_welch_density__coeff_5',	'gx_spkt_welch_density__coeff_8',	'gx_ar_coefficient__coeff_0__k_10',	'gx_ar_coefficient__coeff_k_10',	'gx_ar_coefficient__coeff_2__k_10',	'gx_ar_coefficient__coeff_3__k_10',	'gx_ar_coefficient__coeff_4__k_10',	'gx_ar_coefficient__coeff_5__k_10',	'gx_ar_coefficient__coeff_6__k_10',	'gx_ar_coefficient__coeff_7__k_10',	'gx_ar_coefficient__coeff_8__k_10',	'gx_ar_coefficient__coeff_9__k_10',	'gx_ar_coefficient__coeff_10__k_10',	'gx_change_quantiles__f_agg_"mean"__isabs_False__qh_0.2__ql_0.0',	'gx_change_quantiles__f_agg_"var"__isabs_False__qh_0.2__ql_0.0',	'gx_change_quantiles__f_agg_"mean"__isabs_True__qh_0.2__ql_0.0',	'gx_change_quantiles__f_agg_"var"__isabs_True__qh_0.2__ql_0.0',	'gx_change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.0',	'gx_change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.0',	'gx_change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.0',	'gx_change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.0',	'gx_change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.0',	'gx_change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.0',	'gx_change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0',	'gx_change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.0',	'gx_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.0',	'gx_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0',	'gx_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0',	'gx_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.0',	'gx_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.0',	'gx_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.0',	'gx_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0',	'gx_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.0',	'gx_change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.2',	'gx_change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.2',	'gx_change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.2',	'gx_change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.2',	'gx_change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.2',	'gx_change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.2',	'gx_change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.2',	'gx_change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.2',	'gx_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.2',	'gx_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.2',	'gx_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2',	'gx_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.2',	'gx_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.2',	'gx_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.2',	'gx_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.2',	'gx_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.2',	'gx_change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.4',	'gx_change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.4',	'gx_change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.4',	'gx_change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.4',	'gx_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.4',	'gx_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.4',	'gx_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.4',	'gx_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.4',	'gx_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.4',	'gx_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.4',	'gx_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.4',	'gx_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.4',	'gx_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.6',	'gx_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.6',	'gx_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.6',	'gx_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.6',	'gx_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.6',	'gx_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.6',	'gx_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.6',	'gx_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.6',	'gx_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.8',	'gx_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.8',	'gx_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.8',	'gx_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.8',	'gx_fft_coefficient__attr_"real"__coeff_0',	'gx_fft_coefficient__attr_"real"__coeff_1',	'gx_fft_coefficient__attr_"real"__coeff_2',	'gx_fft_coefficient__attr_"real"__coeff_3',	'gx_fft_coefficient__attr_"real"__coeff_4',	'gx_fft_coefficient__attr_"real"__coeff_5',	'gx_fft_coefficient__attr_"real"__coeff_6',	'gx_fft_coefficient__attr_"real"__coeff_7',	'gx_fft_coefficient__attr_"real"__coeff_8',	'gx_fft_coefficient__attr_"real"__coeff_9',	'gx_fft_coefficient__attr_"real"__coeff_10',	'gx_fft_coefficient__attr_"real"__coeff_11',	'gx_fft_coefficient__attr_"real"__coeff_12',	'gx_fft_coefficient__attr_"real"__coeff_13',	'gx_fft_coefficient__attr_"real"__coeff_14',	'gx_fft_coefficient__attr_"real"__coeff_15',	'gx_fft_coefficient__attr_"real"__coeff_16',	'gx_fft_coefficient__attr_"real"__coeff_17',	'gx_fft_coefficient__attr_"real"__coeff_18',	'gx_fft_coefficient__attr_"real"__coeff_19',	'gx_fft_coefficient__attr_"real"__coeff_20',	'gx_fft_coefficient__attr_"real"__coeff_21',	'gx_fft_coefficient__attr_"real"__coeff_22',	'gx_fft_coefficient__attr_"real"__coeff_23',	'gx_fft_coefficient__attr_"real"__coeff_24',	'gx_fft_coefficient__attr_"real"__coeff_25',	'gx_fft_coefficient__attr_"real"__coeff_26',	'gx_fft_coefficient__attr_"real"__coeff_27',	'gx_fft_coefficient__attr_"real"__coeff_28',	'gx_fft_coefficient__attr_"real"__coeff_29',	'gx_fft_coefficient__attr_"real"__coeff_30',	'gx_fft_coefficient__attr_"real"__coeff_31',	'gx_fft_coefficient__attr_"real"__coeff_32',	'gx_fft_coefficient__attr_"real"__coeff_33',	'gx_fft_coefficient__attr_"real"__coeff_34',	'gx_fft_coefficient__attr_"real"__coeff_35',	'gx_fft_coefficient__attr_"real"__coeff_36',	'gx_fft_coefficient__attr_"real"__coeff_37',	'gx_fft_coefficient__attr_"real"__coeff_38',	'gx_fft_coefficient__attr_"real"__coeff_39',	'gx_fft_coefficient__attr_"real"__coeff_40',	'gx_fft_coefficient__attr_"real"__coeff_41',	'gx_fft_coefficient__attr_"real"__coeff_42',	'gx_fft_coefficient__attr_"real"__coeff_43',	'gx_fft_coefficient__attr_"real"__coeff_44',	'gx_fft_coefficient__attr_"real"__coeff_45',	'gx_fft_coefficient__attr_"real"__coeff_46',	'gx_fft_coefficient__attr_"real"__coeff_47',	'gx_fft_coefficient__attr_"real"__coeff_48',	'gx_fft_coefficient__attr_"real"__coeff_49',	'gx_fft_coefficient__attr_"real"__coeff_50',	'gx_fft_coefficient__attr_"real"__coeff_51',	'gx_fft_coefficient__attr_"real"__coeff_52',	'gx_fft_coefficient__attr_"real"__coeff_53',	'gx_fft_coefficient__attr_"real"__coeff_54',	'gx_fft_coefficient__attr_"real"__coeff_55',	'gx_fft_coefficient__attr_"real"__coeff_56',	'gx_fft_coefficient__attr_"real"__coeff_57',	'gx_fft_coefficient__attr_"real"__coeff_58',	'gx_fft_coefficient__attr_"real"__coeff_59',	'gx_fft_coefficient__attr_"real"__coeff_60',	'gx_fft_coefficient__attr_"real"__coeff_61',	'gx_fft_coefficient__attr_"real"__coeff_62',	'gx_fft_coefficient__attr_"real"__coeff_63',	'gx_fft_coefficient__attr_"real"__coeff_64',	'gx_fft_coefficient__attr_"real"__coeff_65',	'gx_fft_coefficient__attr_"real"__coeff_66',	'gx_fft_coefficient__attr_"real"__coeff_67',	'gx_fft_coefficient__attr_"real"__coeff_68',	'gx_fft_coefficient__attr_"real"__coeff_69',	'gx_fft_coefficient__attr_"real"__coeff_70',	'gx_fft_coefficient__attr_"real"__coeff_71',	'gx_fft_coefficient__attr_"real"__coeff_72',	'gx_fft_coefficient__attr_"real"__coeff_73',	'gx_fft_coefficient__attr_"real"__coeff_74',	'gx_fft_coefficient__attr_"real"__coeff_75',	'gx_fft_coefficient__attr_"real"__coeff_76',	'gx_fft_coefficient__attr_"real"__coeff_77',	'gx_fft_coefficient__attr_"real"__coeff_78',	'gx_fft_coefficient__attr_"real"__coeff_79',	'gx_fft_coefficient__attr_"real"__coeff_80',	'gx_fft_coefficient__attr_"real"__coeff_81',	'gx_fft_coefficient__attr_"real"__coeff_82',	'gx_fft_coefficient__attr_"real"__coeff_83',	'gx_fft_coefficient__attr_"real"__coeff_84',	'gx_fft_coefficient__attr_"real"__coeff_85',	'gx_fft_coefficient__attr_"real"__coeff_86',	'gx_fft_coefficient__attr_"real"__coeff_87',	'gx_fft_coefficient__attr_"real"__coeff_88',	'gx_fft_coefficient__attr_"real"__coeff_89',	'gx_fft_coefficient__attr_"real"__coeff_90',	'gx_fft_coefficient__attr_"real"__coeff_91',	'gx_fft_coefficient__attr_"real"__coeff_92',	'gx_fft_coefficient__attr_"real"__coeff_93',	'gx_fft_coefficient__attr_"real"__coeff_94',	'gx_fft_coefficient__attr_"real"__coeff_95',	'gx_fft_coefficient__attr_"real"__coeff_96',	'gx_fft_coefficient__attr_"real"__coeff_97',	'gx_fft_coefficient__attr_"real"__coeff_98',	'gx_fft_coefficient__attr_"real"__coeff_99',	'gx_fft_coefficient__attr_"imag"__coeff_0',	'gx_fft_coefficient__attr_"imag"__coeff_1',	'gx_fft_coefficient__attr_"imag"__coeff_2',	'gx_fft_coefficient__attr_"imag"__coeff_3',	'gx_fft_coefficient__attr_"imag"__coeff_4',	'gx_fft_coefficient__attr_"imag"__coeff_5',	'gx_fft_coefficient__attr_"imag"__coeff_6',	'gx_fft_coefficient__attr_"imag"__coeff_7',	'gx_fft_coefficient__attr_"imag"__coeff_8',	'gx_fft_coefficient__attr_"imag"__coeff_9',	'gx_fft_coefficient__attr_"imag"__coeff_10',	'gx_fft_coefficient__attr_"imag"__coeff_11',	'gx_fft_coefficient__attr_"imag"__coeff_12',	'gx_fft_coefficient__attr_"imag"__coeff_13',	'gx_fft_coefficient__attr_"imag"__coeff_14',	'gx_fft_coefficient__attr_"imag"__coeff_15',	'gx_fft_coefficient__attr_"imag"__coeff_16',	'gx_fft_coefficient__attr_"imag"__coeff_17',	'gx_fft_coefficient__attr_"imag"__coeff_18',	'gx_fft_coefficient__attr_"imag"__coeff_19',	'gx_fft_coefficient__attr_"imag"__coeff_20',	'gx_fft_coefficient__attr_"imag"__coeff_21',	'gx_fft_coefficient__attr_"imag"__coeff_22',	'gx_fft_coefficient__attr_"imag"__coeff_23',	'gx_fft_coefficient__attr_"imag"__coeff_24',	'gx_fft_coefficient__attr_"imag"__coeff_25',	'gx_fft_coefficient__attr_"imag"__coeff_26',	'gx_fft_coefficient__attr_"imag"__coeff_27',	'gx_fft_coefficient__attr_"imag"__coeff_28',	'gx_fft_coefficient__attr_"imag"__coeff_29',	'gx_fft_coefficient__attr_"imag"__coeff_30',	'gx_fft_coefficient__attr_"imag"__coeff_31',	'gx_fft_coefficient__attr_"imag"__coeff_32',	'gx_fft_coefficient__attr_"imag"__coeff_33',	'gx_fft_coefficient__attr_"imag"__coeff_34',	'gx_fft_coefficient__attr_"imag"__coeff_35',	'gx_fft_coefficient__attr_"imag"__coeff_36',	'gx_fft_coefficient__attr_"imag"__coeff_37',	'gx_fft_coefficient__attr_"imag"__coeff_38',	'gx_fft_coefficient__attr_"imag"__coeff_39',	'gx_fft_coefficient__attr_"imag"__coeff_40',	'gx_fft_coefficient__attr_"imag"__coeff_41',	'gx_fft_coefficient__attr_"imag"__coeff_42',	'gx_fft_coefficient__attr_"imag"__coeff_43',	'gx_fft_coefficient__attr_"imag"__coeff_44',	'gx_fft_coefficient__attr_"imag"__coeff_45',	'gx_fft_coefficient__attr_"imag"__coeff_46',	'gx_fft_coefficient__attr_"imag"__coeff_47',	'gx_fft_coefficient__attr_"imag"__coeff_48',	'gx_fft_coefficient__attr_"imag"__coeff_49',	'gx_fft_coefficient__attr_"imag"__coeff_50',	'gx_fft_coefficient__attr_"imag"__coeff_51',	'gx_fft_coefficient__attr_"imag"__coeff_52',	'gx_fft_coefficient__attr_"imag"__coeff_53',	'gx_fft_coefficient__attr_"imag"__coeff_54',	'gx_fft_coefficient__attr_"imag"__coeff_55',	'gx_fft_coefficient__attr_"imag"__coeff_56',	'gx_fft_coefficient__attr_"imag"__coeff_57',	'gx_fft_coefficient__attr_"imag"__coeff_58',	'gx_fft_coefficient__attr_"imag"__coeff_59',	'gx_fft_coefficient__attr_"imag"__coeff_60',	'gx_fft_coefficient__attr_"imag"__coeff_61',	'gx_fft_coefficient__attr_"imag"__coeff_62',	'gx_fft_coefficient__attr_"imag"__coeff_63',	'gx_fft_coefficient__attr_"imag"__coeff_64',	'gx_fft_coefficient__attr_"imag"__coeff_65',	'gx_fft_coefficient__attr_"imag"__coeff_66',	'gx_fft_coefficient__attr_"imag"__coeff_67',	'gx_fft_coefficient__attr_"imag"__coeff_68',	'gx_fft_coefficient__attr_"imag"__coeff_69',	'gx_fft_coefficient__attr_"imag"__coeff_70',	'gx_fft_coefficient__attr_"imag"__coeff_71',	'gx_fft_coefficient__attr_"imag"__coeff_72',	'gx_fft_coefficient__attr_"imag"__coeff_73',	'gx_fft_coefficient__attr_"imag"__coeff_74',	'gx_fft_coefficient__attr_"imag"__coeff_75',	'gx_fft_coefficient__attr_"imag"__coeff_76',	'gx_fft_coefficient__attr_"imag"__coeff_77',	'gx_fft_coefficient__attr_"imag"__coeff_78',	'gx_fft_coefficient__attr_"imag"__coeff_79',	'gx_fft_coefficient__attr_"imag"__coeff_80',	'gx_fft_coefficient__attr_"imag"__coeff_81',	'gx_fft_coefficient__attr_"imag"__coeff_82',	'gx_fft_coefficient__attr_"imag"__coeff_83',	'gx_fft_coefficient__attr_"imag"__coeff_84',	'gx_fft_coefficient__attr_"imag"__coeff_85',	'gx_fft_coefficient__attr_"imag"__coeff_86',	'gx_fft_coefficient__attr_"imag"__coeff_87',	'gx_fft_coefficient__attr_"imag"__coeff_88',	'gx_fft_coefficient__attr_"imag"__coeff_89',	'gx_fft_coefficient__attr_"imag"__coeff_90',	'gx_fft_coefficient__attr_"imag"__coeff_91',	'gx_fft_coefficient__attr_"imag"__coeff_92',	'gx_fft_coefficient__attr_"imag"__coeff_93',	'gx_fft_coefficient__attr_"imag"__coeff_94',	'gx_fft_coefficient__attr_"imag"__coeff_95',	'gx_fft_coefficient__attr_"imag"__coeff_96',	'gx_fft_coefficient__attr_"imag"__coeff_97',	'gx_fft_coefficient__attr_"imag"__coeff_98',	'gx_fft_coefficient__attr_"imag"__coeff_99',	'gx_fft_coefficient__attr_"abs"__coeff_0',	'gx_fft_coefficient__attr_"abs"__coeff_1',	'gx_fft_coefficient__attr_"abs"__coeff_2',	'gx_fft_coefficient__attr_"abs"__coeff_3',	'gx_fft_coefficient__attr_"abs"__coeff_4',	'gx_fft_coefficient__attr_"abs"__coeff_5',	'gx_fft_coefficient__attr_"abs"__coeff_6',	'gx_fft_coefficient__attr_"abs"__coeff_7',	'gx_fft_coefficient__attr_"abs"__coeff_8',	'gx_fft_coefficient__attr_"abs"__coeff_9',	'gx_fft_coefficient__attr_"abs"__coeff_10',	'gx_fft_coefficient__attr_"abs"__coeff_11',	'gx_fft_coefficient__attr_"abs"__coeff_12',	'gx_fft_coefficient__attr_"abs"__coeff_13',	'gx_fft_coefficient__attr_"abs"__coeff_14',	'gx_fft_coefficient__attr_"abs"__coeff_15',	'gx_fft_coefficient__attr_"abs"__coeff_16',	'gx_fft_coefficient__attr_"abs"__coeff_17',	'gx_fft_coefficient__attr_"abs"__coeff_18',	'gx_fft_coefficient__attr_"abs"__coeff_19',	'gx_fft_coefficient__attr_"abs"__coeff_20',	'gx_fft_coefficient__attr_"abs"__coeff_21',	'gx_fft_coefficient__attr_"abs"__coeff_22',	'gx_fft_coefficient__attr_"abs"__coeff_23',	'gx_fft_coefficient__attr_"abs"__coeff_24',	'gx_fft_coefficient__attr_"abs"__coeff_25',	'gx_fft_coefficient__attr_"abs"__coeff_26',	'gx_fft_coefficient__attr_"abs"__coeff_27',	'gx_fft_coefficient__attr_"abs"__coeff_28',	'gx_fft_coefficient__attr_"abs"__coeff_29',	'gx_fft_coefficient__attr_"abs"__coeff_30',	'gx_fft_coefficient__attr_"abs"__coeff_31',	'gx_fft_coefficient__attr_"abs"__coeff_32',	'gx_fft_coefficient__attr_"abs"__coeff_33',	'gx_fft_coefficient__attr_"abs"__coeff_34',	'gx_fft_coefficient__attr_"abs"__coeff_35',	'gx_fft_coefficient__attr_"abs"__coeff_36',	'gx_fft_coefficient__attr_"abs"__coeff_37',	'gx_fft_coefficient__attr_"abs"__coeff_38',	'gx_fft_coefficient__attr_"abs"__coeff_39',	'gx_fft_coefficient__attr_"abs"__coeff_40',	'gx_fft_coefficient__attr_"abs"__coeff_41',	'gx_fft_coefficient__attr_"abs"__coeff_42',	'gx_fft_coefficient__attr_"abs"__coeff_43',	'gx_fft_coefficient__attr_"abs"__coeff_44',	'gx_fft_coefficient__attr_"abs"__coeff_45',	'gx_fft_coefficient__attr_"abs"__coeff_46',	'gx_fft_coefficient__attr_"abs"__coeff_47',	'gx_fft_coefficient__attr_"abs"__coeff_48',	'gx_fft_coefficient__attr_"abs"__coeff_49',	'gx_fft_coefficient__attr_"abs"__coeff_50',	'gx_fft_coefficient__attr_"abs"__coeff_51',	'gx_fft_coefficient__attr_"abs"__coeff_52',	'gx_fft_coefficient__attr_"abs"__coeff_53',	'gx_fft_coefficient__attr_"abs"__coeff_54',	'gx_fft_coefficient__attr_"abs"__coeff_55',	'gx_fft_coefficient__attr_"abs"__coeff_56',	'gx_fft_coefficient__attr_"abs"__coeff_57',	'gx_fft_coefficient__attr_"abs"__coeff_58',	'gx_fft_coefficient__attr_"abs"__coeff_59',	'gx_fft_coefficient__attr_"abs"__coeff_60',	'gx_fft_coefficient__attr_"abs"__coeff_61',	'gx_fft_coefficient__attr_"abs"__coeff_62',	'gx_fft_coefficient__attr_"abs"__coeff_63',	'gx_fft_coefficient__attr_"abs"__coeff_64',	'gx_fft_coefficient__attr_"abs"__coeff_65',	'gx_fft_coefficient__attr_"abs"__coeff_66',	'gx_fft_coefficient__attr_"abs"__coeff_67',	'gx_fft_coefficient__attr_"abs"__coeff_68',	'gx_fft_coefficient__attr_"abs"__coeff_69',	'gx_fft_coefficient__attr_"abs"__coeff_70',	'gx_fft_coefficient__attr_"abs"__coeff_71',	'gx_fft_coefficient__attr_"abs"__coeff_72',	'gx_fft_coefficient__attr_"abs"__coeff_73',	'gx_fft_coefficient__attr_"abs"__coeff_74',	'gx_fft_coefficient__attr_"abs"__coeff_75',	'gx_fft_coefficient__attr_"abs"__coeff_76',	'gx_fft_coefficient__attr_"abs"__coeff_77',	'gx_fft_coefficient__attr_"abs"__coeff_78',	'gx_fft_coefficient__attr_"abs"__coeff_79',	'gx_fft_coefficient__attr_"abs"__coeff_80',	'gx_fft_coefficient__attr_"abs"__coeff_81',	'gx_fft_coefficient__attr_"abs"__coeff_82',	'gx_fft_coefficient__attr_"abs"__coeff_83',	'gx_fft_coefficient__attr_"abs"__coeff_84',	'gx_fft_coefficient__attr_"abs"__coeff_85',	'gx_fft_coefficient__attr_"abs"__coeff_86',	'gx_fft_coefficient__attr_"abs"__coeff_87',	'gx_fft_coefficient__attr_"abs"__coeff_88',	'gx_fft_coefficient__attr_"abs"__coeff_89',	'gx_fft_coefficient__attr_"abs"__coeff_90',	'gx_fft_coefficient__attr_"abs"__coeff_91',	'gx_fft_coefficient__attr_"abs"__coeff_92',	'gx_fft_coefficient__attr_"abs"__coeff_93',	'gx_fft_coefficient__attr_"abs"__coeff_94',	'gx_fft_coefficient__attr_"abs"__coeff_95',	'gx_fft_coefficient__attr_"abs"__coeff_96',	'gx_fft_coefficient__attr_"abs"__coeff_97',	'gx_fft_coefficient__attr_"abs"__coeff_98',	'gx_fft_coefficient__attr_"abs"__coeff_99',	'gx_fft_coefficient__attr_"angle"__coeff_0',	'gx_fft_coefficient__attr_"angle"__coeff_1',	'gx_fft_coefficient__attr_"angle"__coeff_2',	'gx_fft_coefficient__attr_"angle"__coeff_3',	'gx_fft_coefficient__attr_"angle"__coeff_4',	'gx_fft_coefficient__attr_"angle"__coeff_5',	'gx_fft_coefficient__attr_"angle"__coeff_6',	'gx_fft_coefficient__attr_"angle"__coeff_7',	'gx_fft_coefficient__attr_"angle"__coeff_8',	'gx_fft_coefficient__attr_"angle"__coeff_9',	'gx_fft_coefficient__attr_"angle"__coeff_10',	'gx_fft_coefficient__attr_"angle"__coeff_11',	'gx_fft_coefficient__attr_"angle"__coeff_12',	'gx_fft_coefficient__attr_"angle"__coeff_13',	'gx_fft_coefficient__attr_"angle"__coeff_14',	'gx_fft_coefficient__attr_"angle"__coeff_15',	'gx_fft_coefficient__attr_"angle"__coeff_16',	'gx_fft_coefficient__attr_"angle"__coeff_17',	'gx_fft_coefficient__attr_"angle"__coeff_18',	'gx_fft_coefficient__attr_"angle"__coeff_19',	'gx_fft_coefficient__attr_"angle"__coeff_20',	'gx_fft_coefficient__attr_"angle"__coeff_21',	'gx_fft_coefficient__attr_"angle"__coeff_22',	'gx_fft_coefficient__attr_"angle"__coeff_23',	'gx_fft_coefficient__attr_"angle"__coeff_24',	'gx_fft_coefficient__attr_"angle"__coeff_25',	'gx_fft_coefficient__attr_"angle"__coeff_26',	'gx_fft_coefficient__attr_"angle"__coeff_27',	'gx_fft_coefficient__attr_"angle"__coeff_28',	'gx_fft_coefficient__attr_"angle"__coeff_29',	'gx_fft_coefficient__attr_"angle"__coeff_30',	'gx_fft_coefficient__attr_"angle"__coeff_31',	'gx_fft_coefficient__attr_"angle"__coeff_32',	'gx_fft_coefficient__attr_"angle"__coeff_33',	'gx_fft_coefficient__attr_"angle"__coeff_34',	'gx_fft_coefficient__attr_"angle"__coeff_35',	'gx_fft_coefficient__attr_"angle"__coeff_36',	'gx_fft_coefficient__attr_"angle"__coeff_37',	'gx_fft_coefficient__attr_"angle"__coeff_38',	'gx_fft_coefficient__attr_"angle"__coeff_39',	'gx_fft_coefficient__attr_"angle"__coeff_40',	'gx_fft_coefficient__attr_"angle"__coeff_41',	'gx_fft_coefficient__attr_"angle"__coeff_42',	'gx_fft_coefficient__attr_"angle"__coeff_43',	'gx_fft_coefficient__attr_"angle"__coeff_44',	'gx_fft_coefficient__attr_"angle"__coeff_45',	'gx_fft_coefficient__attr_"angle"__coeff_46',	'gx_fft_coefficient__attr_"angle"__coeff_47',	'gx_fft_coefficient__attr_"angle"__coeff_48',	'gx_fft_coefficient__attr_"angle"__coeff_49',	'gx_fft_coefficient__attr_"angle"__coeff_50',	'gx_fft_coefficient__attr_"angle"__coeff_51',	'gx_fft_coefficient__attr_"angle"__coeff_52',	'gx_fft_coefficient__attr_"angle"__coeff_53',	'gx_fft_coefficient__attr_"angle"__coeff_54',	'gx_fft_coefficient__attr_"angle"__coeff_55',	'gx_fft_coefficient__attr_"angle"__coeff_56',	'gx_fft_coefficient__attr_"angle"__coeff_57',	'gx_fft_coefficient__attr_"angle"__coeff_58',	'gx_fft_coefficient__attr_"angle"__coeff_59',	'gx_fft_coefficient__attr_"angle"__coeff_60',	'gx_fft_coefficient__attr_"angle"__coeff_61',	'gx_fft_coefficient__attr_"angle"__coeff_62',	'gx_fft_coefficient__attr_"angle"__coeff_63',	'gx_fft_coefficient__attr_"angle"__coeff_64',	'gx_fft_coefficient__attr_"angle"__coeff_65',	'gx_fft_coefficient__attr_"angle"__coeff_66',	'gx_fft_coefficient__attr_"angle"__coeff_67',	'gx_fft_coefficient__attr_"angle"__coeff_68',	'gx_fft_coefficient__attr_"angle"__coeff_69',	'gx_fft_coefficient__attr_"angle"__coeff_70',	'gx_fft_coefficient__attr_"angle"__coeff_71',	'gx_fft_coefficient__attr_"angle"__coeff_72',	'gx_fft_coefficient__attr_"angle"__coeff_73',	'gx_fft_coefficient__attr_"angle"__coeff_74',	'gx_fft_coefficient__attr_"angle"__coeff_75',	'gx_fft_coefficient__attr_"angle"__coeff_76',	'gx_fft_coefficient__attr_"angle"__coeff_77',	'gx_fft_coefficient__attr_"angle"__coeff_78',	'gx_fft_coefficient__attr_"angle"__coeff_79',	'gx_fft_coefficient__attr_"angle"__coeff_80',	'gx_fft_coefficient__attr_"angle"__coeff_81',	'gx_fft_coefficient__attr_"angle"__coeff_82',	'gx_fft_coefficient__attr_"angle"__coeff_83',	'gx_fft_coefficient__attr_"angle"__coeff_84',	'gx_fft_coefficient__attr_"angle"__coeff_85',	'gx_fft_coefficient__attr_"angle"__coeff_86',	'gx_fft_coefficient__attr_"angle"__coeff_87',	'gx_fft_coefficient__attr_"angle"__coeff_88',	'gx_fft_coefficient__attr_"angle"__coeff_89',	'gx_fft_coefficient__attr_"angle"__coeff_90',	'gx_fft_coefficient__attr_"angle"__coeff_91',	'gx_fft_coefficient__attr_"angle"__coeff_92',	'gx_fft_coefficient__attr_"angle"__coeff_93',	'gx_fft_coefficient__attr_"angle"__coeff_94',	'gx_fft_coefficient__attr_"angle"__coeff_95',	'gx_fft_coefficient__attr_"angle"__coeff_96',	'gx_fft_coefficient__attr_"angle"__coeff_97',	'gx_fft_coefficient__attr_"angle"__coeff_98',	'gx_fft_coefficient__attr_"angle"__coeff_99',	'gx_fft_aggregated__aggtype_"centroid"',	'gx_fft_aggregated__aggtype_"variance"',	'gx_fft_aggregated__aggtype_"skew"',	'gx_fft_aggregated__aggtype_"kurtosis"',	'gx_value_count__value_0',	'gx_value_count__value_1',	'gx_value_count__value_-1',	'gx_range_count__max_min_-1',	'gx_range_count__max_0__min_1000000000000.0',	'gx_range_count__max_1000000000000.0__min_0',	'gx_approximate_entropy__m_2__r_0.1',	'gx_approximate_entropy__m_2__r_0.3',	'gx_approximate_entropy__m_2__r_0.5',	'gx_approximate_entropy__m_2__r_0.7',	'gx_approximate_entropy__m_2__r_0.9',	'gx_friedrich_coefficients__coeff_0__m_3__r_30',	'gx_friedrich_coefficients__coeff_m_3__r_30',	'gx_friedrich_coefficients__coeff_2__m_3__r_30',	'gx_friedrich_coefficients__coeff_3__m_3__r_30',	'gx_max_langevin_fixed_point__m_3__r_30',	'gx_linear_trend__attr_"pvalue"',	'gx_linear_trend__attr_"rvalue"',	'gx_linear_trend__attr_"intercept"',	'gx_linear_trend__attr_"slope"',	'gx_linear_trend__attr_"stderr"',	'gx_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"max"',	'gx_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"min"',	'gx_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"mean"',	'gx_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"var"',	'gx_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"max"',	'gx_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"min"',	'gx_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"mean"',	'gx_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"var"',	'gx_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"max"',	'gx_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"min"',	'gx_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"mean"',	'gx_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"var"',	'gx_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"max"',	'gx_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"min"',	'gx_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"mean"',	'gx_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"var"',	'gx_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"max"',	'gx_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"min"',	'gx_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"mean"',	'gx_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"var"',	'gx_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"max"',	'gx_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"min"',	'gx_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"mean"',	'gx_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"var"',	'gx_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"max"',	'gx_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"min"',	'gx_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"mean"',	'gx_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"var"',	'gx_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"max"',	'gx_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"min"',	'gx_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"mean"',	'gx_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"var"',	'gx_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"max"',	'gx_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"min"',	'gx_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"mean"',	'gx_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"var"',	'gx_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"max"',	'gx_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"min"',	'gx_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"mean"',	'gx_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"var"',	'gx_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"max"',	'gx_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"min"',	'gx_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"mean"',	'gx_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"var"',	'gx_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"max"',	'gx_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"min"',	'gx_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"mean"',	'gx_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"var"',	'gx_augmented_dickey_fuller__attr_"teststat"__autolag_"AIC"',	'gx_augmented_dickey_fuller__attr_"pvalue"__autolag_"AIC"',	'gx_augmented_dickey_fuller__attr_"usedlag"__autolag_"AIC"',	'gx_number_crossing_m__m_0',	'gx_number_crossing_m__m_-1',	'gx_number_crossing_m__m_1',	'gx_energy_ratio_by_chunks__num_segments_10__segment_focus_0',	'gx_energy_ratio_by_chunks__num_segments_10__segment_focus_1',	'gx_energy_ratio_by_chunks__num_segments_10__segment_focus_2',	'gx_energy_ratio_by_chunks__num_segments_10__segment_focus_3',	'gx_energy_ratio_by_chunks__num_segments_10__segment_focus_4',	'gx_energy_ratio_by_chunks__num_segments_10__segment_focus_5',	'gx_energy_ratio_by_chunks__num_segments_10__segment_focus_6',	'gx_energy_ratio_by_chunks__num_segments_10__segment_focus_7',	'gx_energy_ratio_by_chunks__num_segments_10__segment_focus_8',	'gx_energy_ratio_by_chunks__num_segments_10__segment_focus_9',	'gx_ratio_beyond_r_sigma__r_0.5',	'gx_ratio_beyond_r_sigma__r_1',	'gx_ratio_beyond_r_sigma__r_1.5',	'gx_ratio_beyond_r_sigma__r_2',	'gx_ratio_beyond_r_sigma__r_2.5',	'gx_ratio_beyond_r_sigma__r_3',	'gx_ratio_beyond_r_sigma__r_5',	'gx_ratio_beyond_r_sigma__r_6',	'gx_ratio_beyond_r_sigma__r_7',	'gx_ratio_beyond_r_sigma__r_10',	'gx_count_above__t_0',	'gx_count_below__t_0',	'gx_lempel_ziv_complexity__bins_2',	'gx_lempel_ziv_complexity__bins_3',	'gx_lempel_ziv_complexity__bins_5',	'gx_lempel_ziv_complexity__bins_10',	'gx_lempel_ziv_complexity__bins_100',	'gx_fourier_entropy__bins_2',	'gx_fourier_entropy__bins_3',	'gx_fourier_entropy__bins_5',	'gx_fourier_entropy__bins_10',	'gx_fourier_entropy__bins_100',	'gx_permutation_entropy__dimension_3__tau_1',	'gx_permutation_entropy__dimension_4__tau_1',	'gx_permutation_entropy__dimension_5__tau_1',	'gx_permutation_entropy__dimension_6__tau_1',	'gx_permutation_entropy__dimension_7__tau_1',	'gx_query_similarity_count__query_None__threshold_0.0',	'gx_matrix_profile__feature_"min"__threshold_0.98',	'gx_matrix_profile__feature_"max"__threshold_0.98',	'gx_matrix_profile__feature_"mean"__threshold_0.98',	'gx_matrix_profile__feature_"median"__threshold_0.98',	'gx_matrix_profile__feature_"25"__threshold_0.98',	'gx_matrix_profile__feature_"75"__threshold_0.98']
fea_user_1gx=pd.DataFrame(fea_user_1gx.reshape( Gx_list_trans.shape[1], 787))
fea_user_1gx.columns=[header_gx]
fea_user_1gx

,gx_variance_larger_than_standard_deviation,gx_has_duplicate_max,gx_has_duplicate_min,gx_has_duplicate,gx_sum_values,gx_abs_energy,gx_mean_abs_change,gx_mean_change,gx_mean_second_derivative_central,gx_median,...,gx_permutation_entropy__dimension_5__tau_1,gx_permutation_entropy__dimension_6__tau_1,gx_permutation_entropy__dimension_7__tau_1,gx_query_similarity_count__query_None__threshold_0.0,"gx_matrix_profile__feature_""min""__threshold_0.98","gx_matrix_profile__feature_""max""__threshold_0.98","gx_matrix_profile__feature_""mean""__threshold_0.98","gx_matrix_profile__feature_""median""__threshold_0.98","gx_matrix_profile__feature_""25""__threshold_0.98","gx_matrix_profile__feature_""75""__threshold_0.98"
0,0.0,0.0,0.0,1.0,0.9613,0.046208,0.000153,-0.000047,0.000003,0.04810,...,2.685945,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,1.0,0.0,1.0,0.9596,0.046043,0.000153,0.000026,0.000006,0.04800,...,2.306669,2.523211,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,1.0,1.0,0.9666,0.046721,0.000168,0.000095,0.000003,0.04815,...,2.100679,2.338372,2.403640,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,0.0,1.0,0.9902,0.049046,0.000226,0.000111,0.000003,0.04970,...,1.124036,1.172993,1.226556,NaN,0.639830,3.896478,1.297599,0.845513,0.747479,1.241578
4,0.0,0.0,0.0,1.0,1.0104,0.051048,0.000174,0.000079,-0.000008,0.05040,...,1.536650,1.799959,2.064258,NaN,4.107435,5.722644,4.958574,5.094451,4.680235,5.310627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,0.0,1.0,1.0,1.0,-0.2141,0.002301,0.000279,-0.000005,0.000006,-0.01050,...,2.479956,2.615631,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
805,0.0,0.0,0.0,1.0,-0.1986,0.001979,0.000158,0.000095,0.000003,-0.01000,...,1.450006,1.707539,1.871160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
806,0.0,1.0,0.0,1.0,-0.1850,0.001715,0.000158,0.000063,0.000003,-0.00915,...,1.646225,1.799959,1.871160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
807,0.0,1.0,1.0,1.0,-0.1634,0.001357,0.000216,0.000089,-0.000003,-0.00860,...,1.646225,1.799959,1.871160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### removing binary features  

In [11]:
r_gx=fea_user_1gx.dropna(axis='columns',how='all')
r_gx=r_gx.fillna(0)
binary_features_gx=['gx_variance_larger_than_standard_deviation', 'gx_has_duplicate_max',	'gx_has_duplicate_min',	'gx_has_duplicate','gx_symmetry_looking__r_0.0',	'gx_symmetry_looking__r_0.05',	'gx_symmetry_looking__r_0.1',	'gx_symmetry_looking__r_0.15000000000000002',	'gx_symmetry_looking__r_0.2',	'gx_symmetry_looking__r_0.25',	'gx_symmetry_looking__r_0.30000000000000004',	'gx_symmetry_looking__r_0.35000000000000003',	'gx_symmetry_looking__r_0.4',	'gx_symmetry_looking__r_0.45',	'gx_symmetry_looking__r_0.5',	'gx_symmetry_looking__r_0.55',	'gx_symmetry_looking__r_0.6000000000000001',	'gx_symmetry_looking__r_0.65',	'gx_symmetry_looking__r_0.7000000000000001',	'gx_symmetry_looking__r_0.75',	'gx_symmetry_looking__r_0.8',	'gx_symmetry_looking__r_0.8500000000000001',	'gx_symmetry_looking__r_0.9',	'gx_symmetry_looking__r_0.9500000000000001',	'gx_large_standard_deviation__r_0.05',	'gx_large_standard_deviation__r_0.1',	'gx_large_standard_deviation__r_0.15000000000000002',	'gx_large_standard_deviation__r_0.2',	'gx_large_standard_deviation__r_0.25',	'gx_large_standard_deviation__r_0.30000000000000004',	'gx_large_standard_deviation__r_0.35000000000000003',	'gx_large_standard_deviation__r_0.4',	'gx_large_standard_deviation__r_0.45',	'gx_large_standard_deviation__r_0.5',	'gx_large_standard_deviation__r_0.55',	'gx_large_standard_deviation__r_0.6000000000000001',	'gx_large_standard_deviation__r_0.65',	'gx_large_standard_deviation__r_0.7000000000000001',	'gx_large_standard_deviation__r_0.75',	'gx_large_standard_deviation__r_0.8',	'gx_large_standard_deviation__r_0.8500000000000001',	'gx_large_standard_deviation__r_0.9',	'gx_large_standard_deviation__r_0.9500000000000001','gx_value_count__value_0',	'gx_value_count__value_1',	'gx_value_count__value_-1']
r_gx=r_gx.drop(binary_features_gx, axis=1)
r_gx

d:\m-tech-data-science\tools\python 3.7\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,gx_sum_values,gx_abs_energy,gx_mean_abs_change,gx_mean_change,gx_mean_second_derivative_central,gx_median,gx_mean,gx_length,gx_standard_deviation,gx_variation_coefficient,...,gx_permutation_entropy__dimension_4__tau_1,gx_permutation_entropy__dimension_5__tau_1,gx_permutation_entropy__dimension_6__tau_1,gx_permutation_entropy__dimension_7__tau_1,"gx_matrix_profile__feature_""min""__threshold_0.98","gx_matrix_profile__feature_""max""__threshold_0.98","gx_matrix_profile__feature_""mean""__threshold_0.98","gx_matrix_profile__feature_""median""__threshold_0.98","gx_matrix_profile__feature_""25""__threshold_0.98","gx_matrix_profile__feature_""75""__threshold_0.98"
0,0.9613,0.046208,0.000153,-0.000047,0.000003,0.04810,0.048065,20.0,0.000362,0.007538,...,2.282374,2.685945,2.708050,2.639057,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.9596,0.046043,0.000153,0.000026,0.000006,0.04800,0.047980,20.0,0.000268,0.005577,...,1.921336,2.306669,2.523211,2.639057,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.9666,0.046721,0.000168,0.000095,0.000003,0.04815,0.048330,20.0,0.000516,0.010673,...,1.691559,2.100679,2.338372,2.403640,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.9902,0.049046,0.000226,0.000111,0.000003,0.04970,0.049510,20.0,0.001024,0.020676,...,0.997615,1.124036,1.172993,1.226556,0.639830,3.896478,1.297599,0.845513,0.747479,1.241578
4,1.0104,0.051048,0.000174,0.000079,-0.000008,0.05040,0.050520,20.0,0.000392,0.007758,...,1.200087,1.536650,1.799959,2.064258,4.107435,5.722644,4.958574,5.094451,4.680235,5.310627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,-0.2141,0.002301,0.000279,-0.000005,0.000006,-0.01050,-0.010705,20.0,0.000668,-0.062418,...,2.002883,2.479956,2.615631,2.639057,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
805,-0.1986,0.001979,0.000158,0.000095,0.000003,-0.01000,-0.009930,20.0,0.000589,-0.059331,...,1.200087,1.450006,1.707539,1.871160,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
806,-0.1850,0.001715,0.000158,0.000063,0.000003,-0.00915,-0.009250,20.0,0.000451,-0.048769,...,1.284879,1.646225,1.799959,1.871160,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
807,-0.1634,0.001357,0.000216,0.000089,-0.000003,-0.00860,-0.008170,20.0,0.001038,-0.127089,...,1.394557,1.646225,1.799959,1.871160,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Y-axis gyroscope  

In [12]:
Gy_list=pd.DataFrame(Gy_list)
Gy_list=Gy_list.astype(float)
Gy_list_trans=Gy_list.T
Gy_list_trans

,0,1,2,3,4,5,6,7,8,9,...,799,800,801,802,803,804,805,806,807,808
0,0.1655,0.1643,0.1630,0.1642,0.1659,0.1667,0.1667,0.1690,0.1699,0.1687,...,-0.1134,-0.1175,-0.1162,-0.1131,-0.1135,-0.1145,-0.1104,-0.1164,-0.1136,-0.1092
1,0.1644,0.1642,0.1629,0.1645,0.1660,0.1669,0.1669,0.1687,0.1697,0.1684,...,-0.1181,-0.1167,-0.1165,-0.1125,-0.1135,-0.1143,-0.1139,-0.1164,-0.1134,-0.1089
2,0.1643,0.1640,0.1630,0.1647,0.1660,0.1671,0.1674,0.1689,0.1695,0.1682,...,-0.1210,-0.1153,-0.1168,-0.1123,-0.1134,-0.1142,-0.1173,-0.1164,-0.1133,-0.1088
3,0.1649,0.1637,0.1631,0.1650,0.1658,0.1670,0.1676,0.1696,0.1694,0.1681,...,-0.1211,-0.1143,-0.1171,-0.1124,-0.1134,-0.1143,-0.1182,-0.1159,-0.1129,-0.1087
4,0.1651,0.1637,0.1631,0.1652,0.1657,0.1668,0.1673,0.1700,0.1697,0.1680,...,-0.1188,-0.1143,-0.1168,-0.1125,-0.1135,-0.1143,-0.1171,-0.1151,-0.1124,-0.1086
5,0.1649,0.1637,0.1630,0.1652,0.1656,0.1668,0.1672,0.1702,0.1698,0.1680,...,-0.1158,-0.1148,-0.1160,-0.1125,-0.1137,-0.1144,-0.1163,-0.1145,-0.1119,-0.1086
6,0.1648,0.1635,0.1632,0.1653,0.1656,0.1668,0.1674,0.1703,0.1698,0.1681,...,-0.1141,-0.1155,-0.1153,-0.1126,-0.1139,-0.1198,-0.1168,-0.1141,-0.1113,-0.1089
7,0.1648,0.1632,0.1633,0.1654,0.1660,0.1668,0.1679,0.1701,0.1695,0.1682,...,-0.1141,-0.1160,-0.1148,-0.1129,-0.1142,-0.1197,-0.1177,-0.1140,-0.1106,-0.1091
8,0.1649,0.1631,0.1635,0.1656,0.1664,0.1667,0.1683,0.1698,0.1693,0.1681,...,-0.1156,-0.1160,-0.1143,-0.1132,-0.1144,-0.1145,-0.1178,-0.1139,-0.1100,-0.1089
9,0.1647,0.1632,0.1638,0.1657,0.1666,0.1666,0.1689,0.1699,0.1691,0.1678,...,-0.1171,-0.1160,-0.1137,-0.1135,-0.1144,-0.1105,-0.1170,-0.1138,-0.1095,-0.1084


In [13]:
from tsfresh import extract_features
#c=c.fillna(0)
l_gy=[]
z_gy=[]
u_gy=[]
X_gy=[]
for i in range (Gx_list_trans.shape[1]):
    p_gy=Gy_list_trans.iloc[:,i]
    p_gy=pd.DataFrame(p_gy)
    p_gy['ID'] = '1'
    X_gy= extract_features(p_gy, column_id='ID')
    #print(X.shape)
    l_gy.append(X_gy)
    #m.append(l)
    z_gy=l_gy
u_gy=z_gy  

Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.91s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.64s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.50s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.65s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.80s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.70s/it]


In [14]:
fea_user_1gy=np.array(u_gy)
header_gy=['gy_variance_larger_than_standard_deviation',	'gy_has_duplicate_max',	'gy_has_duplicate_min',	'gy_has_duplicate',	'gy_sum_values',	'gy_abs_energy',	'gy_mean_abs_change',	'gy_mean_change',	'gy_mean_second_derivative_central',	'gy_median',	'gy_mean',	'gy_length',	'gy_standard_deviation',	'gy_variation_coefficient',	'gy_variance',	'gy_skewness',	'gy_kurtosis',	'gy_root_mean_square',	'gy_absolute_sum_of_changes',	'gy_longest_strike_below_mean',	'gy_longest_strike_above_mean',	'gy_count_above_mean',	'gy_count_below_mean',	'gy_last_location_of_maximum',	'gy_first_location_of_maximum',	'gy_last_location_of_minimum',	'gy_first_location_of_minimum',	'gy_percentage_of_reoccurring_values_to_all_values',	'gy_percentage_of_reoccurring_datapoints_to_all_datapoints',	'gy_sum_of_reoccurring_values',	'gy_sum_of_reoccurring_data_points',	'gy_ratio_value_number_to_time_series_length',	'gy_sample_entropy',	'gy_maximum',	'gy_minimum',	'gy_benford_correlation',	'gy_time_reversal_asymmetry_statistic__lag_1',	'gy_time_reversal_asymmetry_statistic__lag_2',	'gy_time_reversal_asymmetry_statistic__lag_3',	'gy_c3__lag_1',	'gy_c3__lag_2',	'gy_c3__lag_3',	'gy_cid_ce__normalize_True',	'gy_cid_ce__normalize_False',	'gy_symmetry_looking__r_0.0',	'gy_symmetry_looking__r_0.05',	'gy_symmetry_looking__r_0.1',	'gy_symmetry_looking__r_0.15000000000000002',	'gy_symmetry_looking__r_0.2',	'gy_symmetry_looking__r_0.25',	'gy_symmetry_looking__r_0.30000000000000004',	'gy_symmetry_looking__r_0.35000000000000003',	'gy_symmetry_looking__r_0.4',	'gy_symmetry_looking__r_0.45',	'gy_symmetry_looking__r_0.5',	'gy_symmetry_looking__r_0.55',	'gy_symmetry_looking__r_0.6000000000000001',	'gy_symmetry_looking__r_0.65',	'gy_symmetry_looking__r_0.7000000000000001',	'gy_symmetry_looking__r_0.75',	'gy_symmetry_looking__r_0.8',	'gy_symmetry_looking__r_0.8500000000000001',	'gy_symmetry_looking__r_0.9',	'gy_symmetry_looking__r_0.9500000000000001',	'gy_large_standard_deviation__r_0.05',	'gy_large_standard_deviation__r_0.1',	'gy_large_standard_deviation__r_0.15000000000000002',	'gy_large_standard_deviation__r_0.2',	'gy_large_standard_deviation__r_0.25',	'gy_large_standard_deviation__r_0.30000000000000004',	'gy_large_standard_deviation__r_0.35000000000000003',	'gy_large_standard_deviation__r_0.4',	'gy_large_standard_deviation__r_0.45',	'gy_large_standard_deviation__r_0.5',	'gy_large_standard_deviation__r_0.55',	'gy_large_standard_deviation__r_0.6000000000000001',	'gy_large_standard_deviation__r_0.65',	'gy_large_standard_deviation__r_0.7000000000000001',	'gy_large_standard_deviation__r_0.75',	'gy_large_standard_deviation__r_0.8',	'gy_large_standard_deviation__r_0.8500000000000001',	'gy_large_standard_deviation__r_0.9',	'gy_large_standard_deviation__r_0.9500000000000001',	'gy_quantile__q_0.1',	'gy_quantile__q_0.2',	'gy_quantile__q_0.3',	'gy_quantile__q_0.4',	'gy_quantile__q_0.6',	'gy_quantile__q_0.7',	'gy_quantile__q_0.8',	'gy_quantile__q_0.9',	'gy_autocorrelation__lag_0',	'gy_autocorrelation__lag_1',	'gy_autocorrelation__lag_2',	'gy_autocorrelation__lag_3',	'gy_autocorrelation__lag_4',	'gy_autocorrelation__lag_5',	'gy_autocorrelation__lag_6',	'gy_autocorrelation__lag_7',	'gy_autocorrelation__lag_8',	'gy_autocorrelation__lag_9',	'gy_agg_autocorrelation__f_agg_"mean"__maxlag_40',	'gy_agg_autocorrelation__f_agg_"median"__maxlag_40',	'gy_agg_autocorrelation__f_agg_"var"__maxlag_40',	'gy_partial_autocorrelation__lag_0',	'gy_partial_autocorrelation__lag_1',	'gy_partial_autocorrelation__lag_2',	'gy_partial_autocorrelation__lag_3',	'gy_partial_autocorrelation__lag_4',	'gy_partial_autocorrelation__lag_5',	'gy_partial_autocorrelation__lag_6',	'gy_partial_autocorrelation__lag_7',	'gy_partial_autocorrelation__lag_8',	'gy_partial_autocorrelation__lag_9',	'gy_number_cwt_peaks__n_1',	'gy_number_cwt_peaks__n_5',	'gy_number_peaks__n_1',	'gy_number_peaks__n_3',	'gy_number_peaks__n_5',	'gy_number_peaks__n_10',	'gy_number_peaks__n_50',	'gy_binned_entropy__max_bins_10',	'gy_index_mass_quantile__q_0.1',	'gy_index_mass_quantile__q_0.2',	'gy_index_mass_quantile__q_0.3',	'gy_index_mass_quantile__q_0.4',	'gy_index_mass_quantile__q_0.6',	'gy_index_mass_quantile__q_0.7',	'gy_index_mass_quantile__q_0.8',	'gy_index_mass_quantile__q_0.9',	'gy_cwt_coefficients__coeff_0__w_2__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_0__w_5__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_0__w_10__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_0__w_20__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_w_2__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_w_5__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_w_10__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_w_20__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_2__w_2__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_2__w_5__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_2__w_10__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_2__w_20__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_3__w_2__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_3__w_5__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_3__w_10__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_3__w_20__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_4__w_2__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_4__w_5__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_4__w_10__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_4__w_20__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_5__w_2__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_5__w_5__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_5__w_10__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_5__w_20__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_6__w_2__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_6__w_5__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_6__w_10__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_6__w_20__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_7__w_2__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_7__w_5__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_7__w_10__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_7__w_20__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_8__w_2__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_8__w_5__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_8__w_10__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_8__w_20__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_9__w_2__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_9__w_5__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_9__w_10__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_9__w_20__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_10__w_2__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_10__w_5__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_10__w_10__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_10__w_20__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_1w_2__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_1w_5__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_1w_10__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_1w_20__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_12__w_2__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_12__w_5__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_12__w_10__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_12__w_20__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_13__w_2__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_13__w_5__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_13__w_10__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_13__w_20__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_14__w_2__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_14__w_5__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_14__w_10__widths_(2 5 10 20)',	'gy_cwt_coefficients__coeff_14__w_20__widths_(2 5 10 20)',	'gy_spkt_welch_density__coeff_2',	'gy_spkt_welch_density__coeff_5',	'gy_spkt_welch_density__coeff_8',	'gy_ar_coefficient__coeff_0__k_10',	'gy_ar_coefficient__coeff_k_10',	'gy_ar_coefficient__coeff_2__k_10',	'gy_ar_coefficient__coeff_3__k_10',	'gy_ar_coefficient__coeff_4__k_10',	'gy_ar_coefficient__coeff_5__k_10',	'gy_ar_coefficient__coeff_6__k_10',	'gy_ar_coefficient__coeff_7__k_10',	'gy_ar_coefficient__coeff_8__k_10',	'gy_ar_coefficient__coeff_9__k_10',	'gy_ar_coefficient__coeff_10__k_10',	'gy_change_quantiles__f_agg_"mean"__isabs_False__qh_0.2__ql_0.0',	'gy_change_quantiles__f_agg_"var"__isabs_False__qh_0.2__ql_0.0',	'gy_change_quantiles__f_agg_"mean"__isabs_True__qh_0.2__ql_0.0',	'gy_change_quantiles__f_agg_"var"__isabs_True__qh_0.2__ql_0.0',	'gy_change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.0',	'gy_change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.0',	'gy_change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.0',	'gy_change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.0',	'gy_change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.0',	'gy_change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.0',	'gy_change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0',	'gy_change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.0',	'gy_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.0',	'gy_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0',	'gy_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0',	'gy_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.0',	'gy_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.0',	'gy_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.0',	'gy_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0',	'gy_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.0',	'gy_change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.2',	'gy_change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.2',	'gy_change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.2',	'gy_change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.2',	'gy_change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.2',	'gy_change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.2',	'gy_change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.2',	'gy_change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.2',	'gy_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.2',	'gy_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.2',	'gy_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2',	'gy_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.2',	'gy_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.2',	'gy_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.2',	'gy_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.2',	'gy_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.2',	'gy_change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.4',	'gy_change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.4',	'gy_change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.4',	'gy_change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.4',	'gy_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.4',	'gy_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.4',	'gy_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.4',	'gy_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.4',	'gy_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.4',	'gy_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.4',	'gy_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.4',	'gy_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.4',	'gy_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.6',	'gy_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.6',	'gy_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.6',	'gy_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.6',	'gy_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.6',	'gy_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.6',	'gy_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.6',	'gy_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.6',	'gy_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.8',	'gy_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.8',	'gy_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.8',	'gy_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.8',	'gy_fft_coefficient__attr_"real"__coeff_0',	'gy_fft_coefficient__attr_"real"__coeff_1',	'gy_fft_coefficient__attr_"real"__coeff_2',	'gy_fft_coefficient__attr_"real"__coeff_3',	'gy_fft_coefficient__attr_"real"__coeff_4',	'gy_fft_coefficient__attr_"real"__coeff_5',	'gy_fft_coefficient__attr_"real"__coeff_6',	'gy_fft_coefficient__attr_"real"__coeff_7',	'gy_fft_coefficient__attr_"real"__coeff_8',	'gy_fft_coefficient__attr_"real"__coeff_9',	'gy_fft_coefficient__attr_"real"__coeff_10',	'gy_fft_coefficient__attr_"real"__coeff_11',	'gy_fft_coefficient__attr_"real"__coeff_12',	'gy_fft_coefficient__attr_"real"__coeff_13',	'gy_fft_coefficient__attr_"real"__coeff_14',	'gy_fft_coefficient__attr_"real"__coeff_15',	'gy_fft_coefficient__attr_"real"__coeff_16',	'gy_fft_coefficient__attr_"real"__coeff_17',	'gy_fft_coefficient__attr_"real"__coeff_18',	'gy_fft_coefficient__attr_"real"__coeff_19',	'gy_fft_coefficient__attr_"real"__coeff_20',	'gy_fft_coefficient__attr_"real"__coeff_21',	'gy_fft_coefficient__attr_"real"__coeff_22',	'gy_fft_coefficient__attr_"real"__coeff_23',	'gy_fft_coefficient__attr_"real"__coeff_24',	'gy_fft_coefficient__attr_"real"__coeff_25',	'gy_fft_coefficient__attr_"real"__coeff_26',	'gy_fft_coefficient__attr_"real"__coeff_27',	'gy_fft_coefficient__attr_"real"__coeff_28',	'gy_fft_coefficient__attr_"real"__coeff_29',	'gy_fft_coefficient__attr_"real"__coeff_30',	'gy_fft_coefficient__attr_"real"__coeff_31',	'gy_fft_coefficient__attr_"real"__coeff_32',	'gy_fft_coefficient__attr_"real"__coeff_33',	'gy_fft_coefficient__attr_"real"__coeff_34',	'gy_fft_coefficient__attr_"real"__coeff_35',	'gy_fft_coefficient__attr_"real"__coeff_36',	'gy_fft_coefficient__attr_"real"__coeff_37',	'gy_fft_coefficient__attr_"real"__coeff_38',	'gy_fft_coefficient__attr_"real"__coeff_39',	'gy_fft_coefficient__attr_"real"__coeff_40',	'gy_fft_coefficient__attr_"real"__coeff_41',	'gy_fft_coefficient__attr_"real"__coeff_42',	'gy_fft_coefficient__attr_"real"__coeff_43',	'gy_fft_coefficient__attr_"real"__coeff_44',	'gy_fft_coefficient__attr_"real"__coeff_45',	'gy_fft_coefficient__attr_"real"__coeff_46',	'gy_fft_coefficient__attr_"real"__coeff_47',	'gy_fft_coefficient__attr_"real"__coeff_48',	'gy_fft_coefficient__attr_"real"__coeff_49',	'gy_fft_coefficient__attr_"real"__coeff_50',	'gy_fft_coefficient__attr_"real"__coeff_51',	'gy_fft_coefficient__attr_"real"__coeff_52',	'gy_fft_coefficient__attr_"real"__coeff_53',	'gy_fft_coefficient__attr_"real"__coeff_54',	'gy_fft_coefficient__attr_"real"__coeff_55',	'gy_fft_coefficient__attr_"real"__coeff_56',	'gy_fft_coefficient__attr_"real"__coeff_57',	'gy_fft_coefficient__attr_"real"__coeff_58',	'gy_fft_coefficient__attr_"real"__coeff_59',	'gy_fft_coefficient__attr_"real"__coeff_60',	'gy_fft_coefficient__attr_"real"__coeff_61',	'gy_fft_coefficient__attr_"real"__coeff_62',	'gy_fft_coefficient__attr_"real"__coeff_63',	'gy_fft_coefficient__attr_"real"__coeff_64',	'gy_fft_coefficient__attr_"real"__coeff_65',	'gy_fft_coefficient__attr_"real"__coeff_66',	'gy_fft_coefficient__attr_"real"__coeff_67',	'gy_fft_coefficient__attr_"real"__coeff_68',	'gy_fft_coefficient__attr_"real"__coeff_69',	'gy_fft_coefficient__attr_"real"__coeff_70',	'gy_fft_coefficient__attr_"real"__coeff_71',	'gy_fft_coefficient__attr_"real"__coeff_72',	'gy_fft_coefficient__attr_"real"__coeff_73',	'gy_fft_coefficient__attr_"real"__coeff_74',	'gy_fft_coefficient__attr_"real"__coeff_75',	'gy_fft_coefficient__attr_"real"__coeff_76',	'gy_fft_coefficient__attr_"real"__coeff_77',	'gy_fft_coefficient__attr_"real"__coeff_78',	'gy_fft_coefficient__attr_"real"__coeff_79',	'gy_fft_coefficient__attr_"real"__coeff_80',	'gy_fft_coefficient__attr_"real"__coeff_81',	'gy_fft_coefficient__attr_"real"__coeff_82',	'gy_fft_coefficient__attr_"real"__coeff_83',	'gy_fft_coefficient__attr_"real"__coeff_84',	'gy_fft_coefficient__attr_"real"__coeff_85',	'gy_fft_coefficient__attr_"real"__coeff_86',	'gy_fft_coefficient__attr_"real"__coeff_87',	'gy_fft_coefficient__attr_"real"__coeff_88',	'gy_fft_coefficient__attr_"real"__coeff_89',	'gy_fft_coefficient__attr_"real"__coeff_90',	'gy_fft_coefficient__attr_"real"__coeff_91',	'gy_fft_coefficient__attr_"real"__coeff_92',	'gy_fft_coefficient__attr_"real"__coeff_93',	'gy_fft_coefficient__attr_"real"__coeff_94',	'gy_fft_coefficient__attr_"real"__coeff_95',	'gy_fft_coefficient__attr_"real"__coeff_96',	'gy_fft_coefficient__attr_"real"__coeff_97',	'gy_fft_coefficient__attr_"real"__coeff_98',	'gy_fft_coefficient__attr_"real"__coeff_99',	'gy_fft_coefficient__attr_"imag"__coeff_0',	'gy_fft_coefficient__attr_"imag"__coeff_1',	'gy_fft_coefficient__attr_"imag"__coeff_2',	'gy_fft_coefficient__attr_"imag"__coeff_3',	'gy_fft_coefficient__attr_"imag"__coeff_4',	'gy_fft_coefficient__attr_"imag"__coeff_5',	'gy_fft_coefficient__attr_"imag"__coeff_6',	'gy_fft_coefficient__attr_"imag"__coeff_7',	'gy_fft_coefficient__attr_"imag"__coeff_8',	'gy_fft_coefficient__attr_"imag"__coeff_9',	'gy_fft_coefficient__attr_"imag"__coeff_10',	'gy_fft_coefficient__attr_"imag"__coeff_11',	'gy_fft_coefficient__attr_"imag"__coeff_12',	'gy_fft_coefficient__attr_"imag"__coeff_13',	'gy_fft_coefficient__attr_"imag"__coeff_14',	'gy_fft_coefficient__attr_"imag"__coeff_15',	'gy_fft_coefficient__attr_"imag"__coeff_16',	'gy_fft_coefficient__attr_"imag"__coeff_17',	'gy_fft_coefficient__attr_"imag"__coeff_18',	'gy_fft_coefficient__attr_"imag"__coeff_19',	'gy_fft_coefficient__attr_"imag"__coeff_20',	'gy_fft_coefficient__attr_"imag"__coeff_21',	'gy_fft_coefficient__attr_"imag"__coeff_22',	'gy_fft_coefficient__attr_"imag"__coeff_23',	'gy_fft_coefficient__attr_"imag"__coeff_24',	'gy_fft_coefficient__attr_"imag"__coeff_25',	'gy_fft_coefficient__attr_"imag"__coeff_26',	'gy_fft_coefficient__attr_"imag"__coeff_27',	'gy_fft_coefficient__attr_"imag"__coeff_28',	'gy_fft_coefficient__attr_"imag"__coeff_29',	'gy_fft_coefficient__attr_"imag"__coeff_30',	'gy_fft_coefficient__attr_"imag"__coeff_31',	'gy_fft_coefficient__attr_"imag"__coeff_32',	'gy_fft_coefficient__attr_"imag"__coeff_33',	'gy_fft_coefficient__attr_"imag"__coeff_34',	'gy_fft_coefficient__attr_"imag"__coeff_35',	'gy_fft_coefficient__attr_"imag"__coeff_36',	'gy_fft_coefficient__attr_"imag"__coeff_37',	'gy_fft_coefficient__attr_"imag"__coeff_38',	'gy_fft_coefficient__attr_"imag"__coeff_39',	'gy_fft_coefficient__attr_"imag"__coeff_40',	'gy_fft_coefficient__attr_"imag"__coeff_41',	'gy_fft_coefficient__attr_"imag"__coeff_42',	'gy_fft_coefficient__attr_"imag"__coeff_43',	'gy_fft_coefficient__attr_"imag"__coeff_44',	'gy_fft_coefficient__attr_"imag"__coeff_45',	'gy_fft_coefficient__attr_"imag"__coeff_46',	'gy_fft_coefficient__attr_"imag"__coeff_47',	'gy_fft_coefficient__attr_"imag"__coeff_48',	'gy_fft_coefficient__attr_"imag"__coeff_49',	'gy_fft_coefficient__attr_"imag"__coeff_50',	'gy_fft_coefficient__attr_"imag"__coeff_51',	'gy_fft_coefficient__attr_"imag"__coeff_52',	'gy_fft_coefficient__attr_"imag"__coeff_53',	'gy_fft_coefficient__attr_"imag"__coeff_54',	'gy_fft_coefficient__attr_"imag"__coeff_55',	'gy_fft_coefficient__attr_"imag"__coeff_56',	'gy_fft_coefficient__attr_"imag"__coeff_57',	'gy_fft_coefficient__attr_"imag"__coeff_58',	'gy_fft_coefficient__attr_"imag"__coeff_59',	'gy_fft_coefficient__attr_"imag"__coeff_60',	'gy_fft_coefficient__attr_"imag"__coeff_61',	'gy_fft_coefficient__attr_"imag"__coeff_62',	'gy_fft_coefficient__attr_"imag"__coeff_63',	'gy_fft_coefficient__attr_"imag"__coeff_64',	'gy_fft_coefficient__attr_"imag"__coeff_65',	'gy_fft_coefficient__attr_"imag"__coeff_66',	'gy_fft_coefficient__attr_"imag"__coeff_67',	'gy_fft_coefficient__attr_"imag"__coeff_68',	'gy_fft_coefficient__attr_"imag"__coeff_69',	'gy_fft_coefficient__attr_"imag"__coeff_70',	'gy_fft_coefficient__attr_"imag"__coeff_71',	'gy_fft_coefficient__attr_"imag"__coeff_72',	'gy_fft_coefficient__attr_"imag"__coeff_73',	'gy_fft_coefficient__attr_"imag"__coeff_74',	'gy_fft_coefficient__attr_"imag"__coeff_75',	'gy_fft_coefficient__attr_"imag"__coeff_76',	'gy_fft_coefficient__attr_"imag"__coeff_77',	'gy_fft_coefficient__attr_"imag"__coeff_78',	'gy_fft_coefficient__attr_"imag"__coeff_79',	'gy_fft_coefficient__attr_"imag"__coeff_80',	'gy_fft_coefficient__attr_"imag"__coeff_81',	'gy_fft_coefficient__attr_"imag"__coeff_82',	'gy_fft_coefficient__attr_"imag"__coeff_83',	'gy_fft_coefficient__attr_"imag"__coeff_84',	'gy_fft_coefficient__attr_"imag"__coeff_85',	'gy_fft_coefficient__attr_"imag"__coeff_86',	'gy_fft_coefficient__attr_"imag"__coeff_87',	'gy_fft_coefficient__attr_"imag"__coeff_88',	'gy_fft_coefficient__attr_"imag"__coeff_89',	'gy_fft_coefficient__attr_"imag"__coeff_90',	'gy_fft_coefficient__attr_"imag"__coeff_91',	'gy_fft_coefficient__attr_"imag"__coeff_92',	'gy_fft_coefficient__attr_"imag"__coeff_93',	'gy_fft_coefficient__attr_"imag"__coeff_94',	'gy_fft_coefficient__attr_"imag"__coeff_95',	'gy_fft_coefficient__attr_"imag"__coeff_96',	'gy_fft_coefficient__attr_"imag"__coeff_97',	'gy_fft_coefficient__attr_"imag"__coeff_98',	'gy_fft_coefficient__attr_"imag"__coeff_99',	'gy_fft_coefficient__attr_"abs"__coeff_0',	'gy_fft_coefficient__attr_"abs"__coeff_1',	'gy_fft_coefficient__attr_"abs"__coeff_2',	'gy_fft_coefficient__attr_"abs"__coeff_3',	'gy_fft_coefficient__attr_"abs"__coeff_4',	'gy_fft_coefficient__attr_"abs"__coeff_5',	'gy_fft_coefficient__attr_"abs"__coeff_6',	'gy_fft_coefficient__attr_"abs"__coeff_7',	'gy_fft_coefficient__attr_"abs"__coeff_8',	'gy_fft_coefficient__attr_"abs"__coeff_9',	'gy_fft_coefficient__attr_"abs"__coeff_10',	'gy_fft_coefficient__attr_"abs"__coeff_11',	'gy_fft_coefficient__attr_"abs"__coeff_12',	'gy_fft_coefficient__attr_"abs"__coeff_13',	'gy_fft_coefficient__attr_"abs"__coeff_14',	'gy_fft_coefficient__attr_"abs"__coeff_15',	'gy_fft_coefficient__attr_"abs"__coeff_16',	'gy_fft_coefficient__attr_"abs"__coeff_17',	'gy_fft_coefficient__attr_"abs"__coeff_18',	'gy_fft_coefficient__attr_"abs"__coeff_19',	'gy_fft_coefficient__attr_"abs"__coeff_20',	'gy_fft_coefficient__attr_"abs"__coeff_21',	'gy_fft_coefficient__attr_"abs"__coeff_22',	'gy_fft_coefficient__attr_"abs"__coeff_23',	'gy_fft_coefficient__attr_"abs"__coeff_24',	'gy_fft_coefficient__attr_"abs"__coeff_25',	'gy_fft_coefficient__attr_"abs"__coeff_26',	'gy_fft_coefficient__attr_"abs"__coeff_27',	'gy_fft_coefficient__attr_"abs"__coeff_28',	'gy_fft_coefficient__attr_"abs"__coeff_29',	'gy_fft_coefficient__attr_"abs"__coeff_30',	'gy_fft_coefficient__attr_"abs"__coeff_31',	'gy_fft_coefficient__attr_"abs"__coeff_32',	'gy_fft_coefficient__attr_"abs"__coeff_33',	'gy_fft_coefficient__attr_"abs"__coeff_34',	'gy_fft_coefficient__attr_"abs"__coeff_35',	'gy_fft_coefficient__attr_"abs"__coeff_36',	'gy_fft_coefficient__attr_"abs"__coeff_37',	'gy_fft_coefficient__attr_"abs"__coeff_38',	'gy_fft_coefficient__attr_"abs"__coeff_39',	'gy_fft_coefficient__attr_"abs"__coeff_40',	'gy_fft_coefficient__attr_"abs"__coeff_41',	'gy_fft_coefficient__attr_"abs"__coeff_42',	'gy_fft_coefficient__attr_"abs"__coeff_43',	'gy_fft_coefficient__attr_"abs"__coeff_44',	'gy_fft_coefficient__attr_"abs"__coeff_45',	'gy_fft_coefficient__attr_"abs"__coeff_46',	'gy_fft_coefficient__attr_"abs"__coeff_47',	'gy_fft_coefficient__attr_"abs"__coeff_48',	'gy_fft_coefficient__attr_"abs"__coeff_49',	'gy_fft_coefficient__attr_"abs"__coeff_50',	'gy_fft_coefficient__attr_"abs"__coeff_51',	'gy_fft_coefficient__attr_"abs"__coeff_52',	'gy_fft_coefficient__attr_"abs"__coeff_53',	'gy_fft_coefficient__attr_"abs"__coeff_54',	'gy_fft_coefficient__attr_"abs"__coeff_55',	'gy_fft_coefficient__attr_"abs"__coeff_56',	'gy_fft_coefficient__attr_"abs"__coeff_57',	'gy_fft_coefficient__attr_"abs"__coeff_58',	'gy_fft_coefficient__attr_"abs"__coeff_59',	'gy_fft_coefficient__attr_"abs"__coeff_60',	'gy_fft_coefficient__attr_"abs"__coeff_61',	'gy_fft_coefficient__attr_"abs"__coeff_62',	'gy_fft_coefficient__attr_"abs"__coeff_63',	'gy_fft_coefficient__attr_"abs"__coeff_64',	'gy_fft_coefficient__attr_"abs"__coeff_65',	'gy_fft_coefficient__attr_"abs"__coeff_66',	'gy_fft_coefficient__attr_"abs"__coeff_67',	'gy_fft_coefficient__attr_"abs"__coeff_68',	'gy_fft_coefficient__attr_"abs"__coeff_69',	'gy_fft_coefficient__attr_"abs"__coeff_70',	'gy_fft_coefficient__attr_"abs"__coeff_71',	'gy_fft_coefficient__attr_"abs"__coeff_72',	'gy_fft_coefficient__attr_"abs"__coeff_73',	'gy_fft_coefficient__attr_"abs"__coeff_74',	'gy_fft_coefficient__attr_"abs"__coeff_75',	'gy_fft_coefficient__attr_"abs"__coeff_76',	'gy_fft_coefficient__attr_"abs"__coeff_77',	'gy_fft_coefficient__attr_"abs"__coeff_78',	'gy_fft_coefficient__attr_"abs"__coeff_79',	'gy_fft_coefficient__attr_"abs"__coeff_80',	'gy_fft_coefficient__attr_"abs"__coeff_81',	'gy_fft_coefficient__attr_"abs"__coeff_82',	'gy_fft_coefficient__attr_"abs"__coeff_83',	'gy_fft_coefficient__attr_"abs"__coeff_84',	'gy_fft_coefficient__attr_"abs"__coeff_85',	'gy_fft_coefficient__attr_"abs"__coeff_86',	'gy_fft_coefficient__attr_"abs"__coeff_87',	'gy_fft_coefficient__attr_"abs"__coeff_88',	'gy_fft_coefficient__attr_"abs"__coeff_89',	'gy_fft_coefficient__attr_"abs"__coeff_90',	'gy_fft_coefficient__attr_"abs"__coeff_91',	'gy_fft_coefficient__attr_"abs"__coeff_92',	'gy_fft_coefficient__attr_"abs"__coeff_93',	'gy_fft_coefficient__attr_"abs"__coeff_94',	'gy_fft_coefficient__attr_"abs"__coeff_95',	'gy_fft_coefficient__attr_"abs"__coeff_96',	'gy_fft_coefficient__attr_"abs"__coeff_97',	'gy_fft_coefficient__attr_"abs"__coeff_98',	'gy_fft_coefficient__attr_"abs"__coeff_99',	'gy_fft_coefficient__attr_"angle"__coeff_0',	'gy_fft_coefficient__attr_"angle"__coeff_1',	'gy_fft_coefficient__attr_"angle"__coeff_2',	'gy_fft_coefficient__attr_"angle"__coeff_3',	'gy_fft_coefficient__attr_"angle"__coeff_4',	'gy_fft_coefficient__attr_"angle"__coeff_5',	'gy_fft_coefficient__attr_"angle"__coeff_6',	'gy_fft_coefficient__attr_"angle"__coeff_7',	'gy_fft_coefficient__attr_"angle"__coeff_8',	'gy_fft_coefficient__attr_"angle"__coeff_9',	'gy_fft_coefficient__attr_"angle"__coeff_10',	'gy_fft_coefficient__attr_"angle"__coeff_11',	'gy_fft_coefficient__attr_"angle"__coeff_12',	'gy_fft_coefficient__attr_"angle"__coeff_13',	'gy_fft_coefficient__attr_"angle"__coeff_14',	'gy_fft_coefficient__attr_"angle"__coeff_15',	'gy_fft_coefficient__attr_"angle"__coeff_16',	'gy_fft_coefficient__attr_"angle"__coeff_17',	'gy_fft_coefficient__attr_"angle"__coeff_18',	'gy_fft_coefficient__attr_"angle"__coeff_19',	'gy_fft_coefficient__attr_"angle"__coeff_20',	'gy_fft_coefficient__attr_"angle"__coeff_21',	'gy_fft_coefficient__attr_"angle"__coeff_22',	'gy_fft_coefficient__attr_"angle"__coeff_23',	'gy_fft_coefficient__attr_"angle"__coeff_24',	'gy_fft_coefficient__attr_"angle"__coeff_25',	'gy_fft_coefficient__attr_"angle"__coeff_26',	'gy_fft_coefficient__attr_"angle"__coeff_27',	'gy_fft_coefficient__attr_"angle"__coeff_28',	'gy_fft_coefficient__attr_"angle"__coeff_29',	'gy_fft_coefficient__attr_"angle"__coeff_30',	'gy_fft_coefficient__attr_"angle"__coeff_31',	'gy_fft_coefficient__attr_"angle"__coeff_32',	'gy_fft_coefficient__attr_"angle"__coeff_33',	'gy_fft_coefficient__attr_"angle"__coeff_34',	'gy_fft_coefficient__attr_"angle"__coeff_35',	'gy_fft_coefficient__attr_"angle"__coeff_36',	'gy_fft_coefficient__attr_"angle"__coeff_37',	'gy_fft_coefficient__attr_"angle"__coeff_38',	'gy_fft_coefficient__attr_"angle"__coeff_39',	'gy_fft_coefficient__attr_"angle"__coeff_40',	'gy_fft_coefficient__attr_"angle"__coeff_41',	'gy_fft_coefficient__attr_"angle"__coeff_42',	'gy_fft_coefficient__attr_"angle"__coeff_43',	'gy_fft_coefficient__attr_"angle"__coeff_44',	'gy_fft_coefficient__attr_"angle"__coeff_45',	'gy_fft_coefficient__attr_"angle"__coeff_46',	'gy_fft_coefficient__attr_"angle"__coeff_47',	'gy_fft_coefficient__attr_"angle"__coeff_48',	'gy_fft_coefficient__attr_"angle"__coeff_49',	'gy_fft_coefficient__attr_"angle"__coeff_50',	'gy_fft_coefficient__attr_"angle"__coeff_51',	'gy_fft_coefficient__attr_"angle"__coeff_52',	'gy_fft_coefficient__attr_"angle"__coeff_53',	'gy_fft_coefficient__attr_"angle"__coeff_54',	'gy_fft_coefficient__attr_"angle"__coeff_55',	'gy_fft_coefficient__attr_"angle"__coeff_56',	'gy_fft_coefficient__attr_"angle"__coeff_57',	'gy_fft_coefficient__attr_"angle"__coeff_58',	'gy_fft_coefficient__attr_"angle"__coeff_59',	'gy_fft_coefficient__attr_"angle"__coeff_60',	'gy_fft_coefficient__attr_"angle"__coeff_61',	'gy_fft_coefficient__attr_"angle"__coeff_62',	'gy_fft_coefficient__attr_"angle"__coeff_63',	'gy_fft_coefficient__attr_"angle"__coeff_64',	'gy_fft_coefficient__attr_"angle"__coeff_65',	'gy_fft_coefficient__attr_"angle"__coeff_66',	'gy_fft_coefficient__attr_"angle"__coeff_67',	'gy_fft_coefficient__attr_"angle"__coeff_68',	'gy_fft_coefficient__attr_"angle"__coeff_69',	'gy_fft_coefficient__attr_"angle"__coeff_70',	'gy_fft_coefficient__attr_"angle"__coeff_71',	'gy_fft_coefficient__attr_"angle"__coeff_72',	'gy_fft_coefficient__attr_"angle"__coeff_73',	'gy_fft_coefficient__attr_"angle"__coeff_74',	'gy_fft_coefficient__attr_"angle"__coeff_75',	'gy_fft_coefficient__attr_"angle"__coeff_76',	'gy_fft_coefficient__attr_"angle"__coeff_77',	'gy_fft_coefficient__attr_"angle"__coeff_78',	'gy_fft_coefficient__attr_"angle"__coeff_79',	'gy_fft_coefficient__attr_"angle"__coeff_80',	'gy_fft_coefficient__attr_"angle"__coeff_81',	'gy_fft_coefficient__attr_"angle"__coeff_82',	'gy_fft_coefficient__attr_"angle"__coeff_83',	'gy_fft_coefficient__attr_"angle"__coeff_84',	'gy_fft_coefficient__attr_"angle"__coeff_85',	'gy_fft_coefficient__attr_"angle"__coeff_86',	'gy_fft_coefficient__attr_"angle"__coeff_87',	'gy_fft_coefficient__attr_"angle"__coeff_88',	'gy_fft_coefficient__attr_"angle"__coeff_89',	'gy_fft_coefficient__attr_"angle"__coeff_90',	'gy_fft_coefficient__attr_"angle"__coeff_91',	'gy_fft_coefficient__attr_"angle"__coeff_92',	'gy_fft_coefficient__attr_"angle"__coeff_93',	'gy_fft_coefficient__attr_"angle"__coeff_94',	'gy_fft_coefficient__attr_"angle"__coeff_95',	'gy_fft_coefficient__attr_"angle"__coeff_96',	'gy_fft_coefficient__attr_"angle"__coeff_97',	'gy_fft_coefficient__attr_"angle"__coeff_98',	'gy_fft_coefficient__attr_"angle"__coeff_99',	'gy_fft_aggregated__aggtype_"centroid"',	'gy_fft_aggregated__aggtype_"variance"',	'gy_fft_aggregated__aggtype_"skew"',	'gy_fft_aggregated__aggtype_"kurtosis"',	'gy_value_count__value_0',	'gy_value_count__value_1',	'gy_value_count__value_-1',	'gy_range_count__max_min_-1',	'gy_range_count__max_0__min_1000000000000.0',	'gy_range_count__max_1000000000000.0__min_0',	'gy_approximate_entropy__m_2__r_0.1',	'gy_approximate_entropy__m_2__r_0.3',	'gy_approximate_entropy__m_2__r_0.5',	'gy_approximate_entropy__m_2__r_0.7',	'gy_approximate_entropy__m_2__r_0.9',	'gy_friedrich_coefficients__coeff_0__m_3__r_30',	'gy_friedrich_coefficients__coeff_m_3__r_30',	'gy_friedrich_coefficients__coeff_2__m_3__r_30',	'gy_friedrich_coefficients__coeff_3__m_3__r_30',	'gy_max_langevin_fixed_point__m_3__r_30',	'gy_linear_trend__attr_"pvalue"',	'gy_linear_trend__attr_"rvalue"',	'gy_linear_trend__attr_"intercept"',	'gy_linear_trend__attr_"slope"',	'gy_linear_trend__attr_"stderr"',	'gy_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"max"',	'gy_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"min"',	'gy_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"mean"',	'gy_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"var"',	'gy_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"max"',	'gy_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"min"',	'gy_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"mean"',	'gy_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"var"',	'gy_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"max"',	'gy_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"min"',	'gy_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"mean"',	'gy_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"var"',	'gy_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"max"',	'gy_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"min"',	'gy_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"mean"',	'gy_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"var"',	'gy_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"max"',	'gy_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"min"',	'gy_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"mean"',	'gy_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"var"',	'gy_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"max"',	'gy_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"min"',	'gy_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"mean"',	'gy_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"var"',	'gy_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"max"',	'gy_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"min"',	'gy_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"mean"',	'gy_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"var"',	'gy_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"max"',	'gy_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"min"',	'gy_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"mean"',	'gy_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"var"',	'gy_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"max"',	'gy_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"min"',	'gy_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"mean"',	'gy_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"var"',	'gy_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"max"',	'gy_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"min"',	'gy_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"mean"',	'gy_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"var"',	'gy_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"max"',	'gy_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"min"',	'gy_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"mean"',	'gy_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"var"',	'gy_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"max"',	'gy_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"min"',	'gy_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"mean"',	'gy_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"var"',	'gy_augmented_dickey_fuller__attr_"teststat"__autolag_"AIC"',	'gy_augmented_dickey_fuller__attr_"pvalue"__autolag_"AIC"',	'gy_augmented_dickey_fuller__attr_"usedlag"__autolag_"AIC"',	'gy_number_crossing_m__m_0',	'gy_number_crossing_m__m_-1',	'gy_number_crossing_m__m_1',	'gy_energy_ratio_by_chunks__num_segments_10__segment_focus_0',	'gy_energy_ratio_by_chunks__num_segments_10__segment_focus_1',	'gy_energy_ratio_by_chunks__num_segments_10__segment_focus_2',	'gy_energy_ratio_by_chunks__num_segments_10__segment_focus_3',	'gy_energy_ratio_by_chunks__num_segments_10__segment_focus_4',	'gy_energy_ratio_by_chunks__num_segments_10__segment_focus_5',	'gy_energy_ratio_by_chunks__num_segments_10__segment_focus_6',	'gy_energy_ratio_by_chunks__num_segments_10__segment_focus_7',	'gy_energy_ratio_by_chunks__num_segments_10__segment_focus_8',	'gy_energy_ratio_by_chunks__num_segments_10__segment_focus_9',	'gy_ratio_beyond_r_sigma__r_0.5',	'gy_ratio_beyond_r_sigma__r_1',	'gy_ratio_beyond_r_sigma__r_1.5',	'gy_ratio_beyond_r_sigma__r_2',	'gy_ratio_beyond_r_sigma__r_2.5',	'gy_ratio_beyond_r_sigma__r_3',	'gy_ratio_beyond_r_sigma__r_5',	'gy_ratio_beyond_r_sigma__r_6',	'gy_ratio_beyond_r_sigma__r_7',	'gy_ratio_beyond_r_sigma__r_10',	'gy_count_above__t_0',	'gy_count_below__t_0',	'gy_lempel_ziv_complexity__bins_2',	'gy_lempel_ziv_complexity__bins_3',	'gy_lempel_ziv_complexity__bins_5',	'gy_lempel_ziv_complexity__bins_10',	'gy_lempel_ziv_complexity__bins_100',	'gy_fourier_entropy__bins_2',	'gy_fourier_entropy__bins_3',	'gy_fourier_entropy__bins_5',	'gy_fourier_entropy__bins_10',	'gy_fourier_entropy__bins_100',	'gy_permutation_entropy__dimension_3__tau_1',	'gy_permutation_entropy__dimension_4__tau_1',	'gy_permutation_entropy__dimension_5__tau_1',	'gy_permutation_entropy__dimension_6__tau_1',	'gy_permutation_entropy__dimension_7__tau_1',	'gy_query_similarity_count__query_None__threshold_0.0',	'gy_matrix_profile__feature_"min"__threshold_0.98',	'gy_matrix_profile__feature_"max"__threshold_0.98',	'gy_matrix_profile__feature_"mean"__threshold_0.98',	'gy_matrix_profile__feature_"median"__threshold_0.98',	'gy_matrix_profile__feature_"25"__threshold_0.98',	'gy_matrix_profile__feature_"75"__threshold_0.98']
fea_user_1gy=pd.DataFrame(fea_user_1gy.reshape(Gx_list_trans.shape[1], 787))
fea_user_1gy.columns=[header_gy]
fea_user_1gy

,gy_variance_larger_than_standard_deviation,gy_has_duplicate_max,gy_has_duplicate_min,gy_has_duplicate,gy_sum_values,gy_abs_energy,gy_mean_abs_change,gy_mean_change,gy_mean_second_derivative_central,gy_median,...,gy_permutation_entropy__dimension_5__tau_1,gy_permutation_entropy__dimension_6__tau_1,gy_permutation_entropy__dimension_7__tau_1,gy_query_similarity_count__query_None__threshold_0.0,"gy_matrix_profile__feature_""min""__threshold_0.98","gy_matrix_profile__feature_""max""__threshold_0.98","gy_matrix_profile__feature_""mean""__threshold_0.98","gy_matrix_profile__feature_""median""__threshold_0.98","gy_matrix_profile__feature_""25""__threshold_0.98","gy_matrix_profile__feature_""75""__threshold_0.98"
0,0.0,0.0,0.0,1.0,3.2849,0.539538,0.000226,-0.000121,0.000033,0.16430,...,2.512659,2.615631,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,1.0,3.2685,0.534158,0.000142,-0.000026,0.000011,0.16325,...,2.685945,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,1.0,3.2827,0.538827,0.000163,0.000142,0.000006,0.16400,...,1.037392,1.172993,1.226556,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,0.0,1.0,3.3104,0.547944,0.000168,0.000126,-0.000003,0.16560,...,1.333473,1.599015,1.666102,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,1.0,1.0,3.3278,0.553717,0.000132,0.000037,-0.000006,0.16660,...,2.339372,2.488328,2.540036,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,0.0,0.0,0.0,1.0,-2.3130,0.267625,0.001437,-0.000132,0.000017,-0.11540,...,2.685945,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
805,0.0,0.0,0.0,1.0,-2.3130,0.267568,0.000800,-0.000179,0.000100,-0.11635,...,1.646225,1.799959,1.871160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
806,0.0,0.0,1.0,1.0,-2.2694,0.257590,0.000363,0.000363,0.000014,-0.11370,...,-0.000000,-0.000000,-0.000000,NaN,0.655702,1.738159,1.153516,1.278667,0.659904,1.475961
807,0.0,0.0,0.0,1.0,-2.2070,0.243610,0.000326,0.000274,0.000008,-0.10935,...,0.908909,0.949594,0.994354,NaN,1.148715,3.223797,1.593679,1.235337,1.217092,1.612342


In [15]:
r_gy=fea_user_1gy.dropna(axis='columns',how='all')
r_gy=r_gy.fillna(0)
binary_features_gy=['gy_variance_larger_than_standard_deviation','gy_has_duplicate_max',	'gy_has_duplicate_min',	'gy_has_duplicate','gy_symmetry_looking__r_0.0',	'gy_symmetry_looking__r_0.05',	'gy_symmetry_looking__r_0.1',	'gy_symmetry_looking__r_0.15000000000000002',	'gy_symmetry_looking__r_0.2',	'gy_symmetry_looking__r_0.25',	'gy_symmetry_looking__r_0.30000000000000004',	'gy_symmetry_looking__r_0.35000000000000003',	'gy_symmetry_looking__r_0.4',	'gy_symmetry_looking__r_0.45',	'gy_symmetry_looking__r_0.5',	'gy_symmetry_looking__r_0.55',	'gy_symmetry_looking__r_0.6000000000000001',	'gy_symmetry_looking__r_0.65',	'gy_symmetry_looking__r_0.7000000000000001',	'gy_symmetry_looking__r_0.75',	'gy_symmetry_looking__r_0.8',	'gy_symmetry_looking__r_0.8500000000000001',	'gy_symmetry_looking__r_0.9',	'gy_symmetry_looking__r_0.9500000000000001',	'gy_large_standard_deviation__r_0.05',	'gy_large_standard_deviation__r_0.1',	'gy_large_standard_deviation__r_0.15000000000000002',	'gy_large_standard_deviation__r_0.2',	'gy_large_standard_deviation__r_0.25',	'gy_large_standard_deviation__r_0.30000000000000004',	'gy_large_standard_deviation__r_0.35000000000000003',	'gy_large_standard_deviation__r_0.4',	'gy_large_standard_deviation__r_0.45',	'gy_large_standard_deviation__r_0.5',	'gy_large_standard_deviation__r_0.55',	'gy_large_standard_deviation__r_0.6000000000000001',	'gy_large_standard_deviation__r_0.65',	'gy_large_standard_deviation__r_0.7000000000000001',	'gy_large_standard_deviation__r_0.75',	'gy_large_standard_deviation__r_0.8',	'gy_large_standard_deviation__r_0.8500000000000001',	'gy_large_standard_deviation__r_0.9',	'gy_large_standard_deviation__r_0.9500000000000001','gy_value_count__value_0',	'gy_value_count__value_1',	'gy_value_count__value_-1']
r_gy=r_gy.drop(binary_features_gy, axis=1)
r_gy

d:\m-tech-data-science\tools\python 3.7\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,gy_sum_values,gy_abs_energy,gy_mean_abs_change,gy_mean_change,gy_mean_second_derivative_central,gy_median,gy_mean,gy_length,gy_standard_deviation,gy_variation_coefficient,...,gy_permutation_entropy__dimension_4__tau_1,gy_permutation_entropy__dimension_5__tau_1,gy_permutation_entropy__dimension_6__tau_1,gy_permutation_entropy__dimension_7__tau_1,"gy_matrix_profile__feature_""min""__threshold_0.98","gy_matrix_profile__feature_""max""__threshold_0.98","gy_matrix_profile__feature_""mean""__threshold_0.98","gy_matrix_profile__feature_""median""__threshold_0.98","gy_matrix_profile__feature_""25""__threshold_0.98","gy_matrix_profile__feature_""75""__threshold_0.98"
0,3.2849,0.539538,0.000226,-0.000121,0.000033,0.16430,0.164245,20.0,0.000687,0.004185,...,2.180840,2.512659,2.615631,2.639057,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.2685,0.534158,0.000142,-0.000026,0.000011,0.16325,0.163425,20.0,0.000411,0.002515,...,2.313154,2.685945,2.708050,2.639057,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,3.2827,0.538827,0.000163,0.000142,0.000006,0.16400,0.164135,20.0,0.001016,0.006190,...,0.790235,1.037392,1.172993,1.226556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3.3104,0.547944,0.000168,0.000126,-0.000003,0.16560,0.165520,20.0,0.000589,0.003557,...,1.079162,1.333473,1.599015,1.666102,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,3.3278,0.553717,0.000132,0.000037,-0.000006,0.16660,0.166390,20.0,0.000493,0.002962,...,1.956187,2.339372,2.488328,2.540036,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,-2.3130,0.267625,0.001437,-0.000132,0.000017,-0.11540,-0.115650,20.0,0.002513,-0.021726,...,2.425480,2.685945,2.708050,2.639057,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
805,-2.3130,0.267568,0.000800,-0.000179,0.000100,-0.11635,-0.115650,20.0,0.001862,-0.016099,...,1.425337,1.646225,1.799959,1.871160,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
806,-2.2694,0.257590,0.000363,0.000363,0.000014,-0.11370,-0.113470,20.0,0.002013,-0.017745,...,-0.000000,-0.000000,-0.000000,-0.000000,0.655702,1.738159,1.153516,1.278667,0.659904,1.475961
807,-2.2070,0.243610,0.000326,0.000274,0.000008,-0.10935,-0.110350,20.0,0.001840,-0.016676,...,0.871781,0.908909,0.949594,0.994354,1.148715,3.223797,1.593679,1.235337,1.217092,1.612342


### Z-axis gyroscope  

In [16]:
Gz_list=pd.DataFrame(Gz_list)
Gz_list=Gz_list.astype(float)
Gz_list_trans=Gz_list.T
Gz_list_trans

,0,1,2,3,4,5,6,7,8,9,...,799,800,801,802,803,804,805,806,807,808
0,0.9703,0.9712,0.9707,0.9703,0.9694,0.9686,0.9683,0.9685,0.9686,0.9678,...,0.9694,0.9701,0.9717,0.9728,0.9723,0.9710,0.9715,0.9701,0.9698,0.9699
1,0.9705,0.9713,0.9706,0.9702,0.9693,0.9686,0.9683,0.9685,0.9686,0.9678,...,0.9690,0.9703,0.9719,0.9729,0.9723,0.9710,0.9709,0.9701,0.9697,0.9699
2,0.9706,0.9713,0.9705,0.9701,0.9692,0.9686,0.9682,0.9685,0.9685,0.9678,...,0.9689,0.9706,0.9719,0.9730,0.9722,0.9709,0.9704,0.9701,0.9696,0.9698
3,0.9706,0.9714,0.9704,0.9700,0.9692,0.9686,0.9683,0.9684,0.9684,0.9677,...,0.9691,0.9709,0.9720,0.9730,0.9721,0.9708,0.9702,0.9702,0.9696,0.9697
4,0.9706,0.9714,0.9704,0.9699,0.9691,0.9686,0.9684,0.9684,0.9683,0.9677,...,0.9696,0.9710,0.9720,0.9730,0.9720,0.9708,0.9702,0.9702,0.9696,0.9696
5,0.9707,0.9713,0.9704,0.9698,0.9690,0.9686,0.9685,0.9684,0.9681,0.9677,...,0.9700,0.9711,0.9722,0.9729,0.9718,0.9707,0.9703,0.9703,0.9696,0.9695
6,0.9707,0.9712,0.9704,0.9697,0.9689,0.9686,0.9685,0.9684,0.9680,0.9677,...,0.9703,0.9712,0.9723,0.9728,0.9717,0.9702,0.9701,0.9702,0.9697,0.9694
7,0.9708,0.9711,0.9704,0.9697,0.9688,0.9685,0.9685,0.9685,0.9680,0.9676,...,0.9703,0.9714,0.9724,0.9727,0.9715,0.9704,0.9699,0.9701,0.9698,0.9693
8,0.9709,0.9710,0.9704,0.9696,0.9687,0.9684,0.9685,0.9686,0.9679,0.9676,...,0.9702,0.9714,0.9725,0.9725,0.9713,0.9711,0.9699,0.9700,0.9698,0.9692
9,0.9710,0.9708,0.9704,0.9695,0.9687,0.9684,0.9685,0.9686,0.9679,0.9675,...,0.9701,0.9716,0.9727,0.9724,0.9712,0.9716,0.9700,0.9699,0.9699,0.9692


In [17]:
from tsfresh import extract_features
#c=c.fillna(0)
l_gz=[]
z_gz=[]
u_gz=[]
X_gz=[]
for i in range (Gx_list_trans.shape[1]):
    p_gz=Gz_list_trans.iloc[:,i]
    p_gz=pd.DataFrame(p_gz)
    p_gz['ID'] = '1'
    X_gz= extract_features(p_gz, column_id='ID')
    #print(X.shape)
    l_gz.append(X_gz)
    #m.append(l)
    z_gz=l_gz
u_gz=z_gz  

Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.65s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.58s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.73s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.68s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.62s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.61s/it]


In [18]:
fea_user_1gz=np.array(u_gz)
header_gz=['gz_variance_larger_than_standard_deviation',	'gz_has_duplicate_max',	'gz_has_duplicate_min',	'gz_has_duplicate',	'gz_sum_values',	'gz_abs_energz',	'gz_mean_abs_change',	'gz_mean_change',	'gz_mean_second_derivative_central',	'gz_median',	'gz_mean',	'gz_length',	'gz_standard_deviation',	'gz_variation_coefficient',	'gz_variance',	'gz_skewness',	'gz_kurtosis',	'gz_root_mean_square',	'gz_absolute_sum_of_changes',	'gz_longest_strike_below_mean',	'gz_longest_strike_above_mean',	'gz_count_above_mean',	'gz_count_below_mean',	'gz_last_location_of_maximum',	'gz_first_location_of_maximum',	'gz_last_location_of_minimum',	'gz_first_location_of_minimum',	'gz_percentage_of_reoccurring_values_to_all_values',	'gz_percentage_of_reoccurring_datapoints_to_all_datapoints',	'gz_sum_of_reoccurring_values',	'gz_sum_of_reoccurring_data_points',	'gz_ratio_value_number_to_time_series_length',	'gz_sample_entropy',	'gz_maximum',	'gz_minimum',	'gz_benford_correlation',	'gz_time_reversal_asymmetry_statistic__lag_1',	'gz_time_reversal_asymmetry_statistic__lag_2',	'gz_time_reversal_asymmetry_statistic__lag_3',	'gz_c3__lag_1',	'gz_c3__lag_2',	'gz_c3__lag_3',	'gz_cid_ce__normalize_True',	'gz_cid_ce__normalize_False',	'gz_symmetry_looking__r_0.0',	'gz_symmetry_looking__r_0.05',	'gz_symmetry_looking__r_0.1',	'gz_symmetry_looking__r_0.15000000000000002',	'gz_symmetry_looking__r_0.2',	'gz_symmetry_looking__r_0.25',	'gz_symmetry_looking__r_0.30000000000000004',	'gz_symmetry_looking__r_0.35000000000000003',	'gz_symmetry_looking__r_0.4',	'gz_symmetry_looking__r_0.45',	'gz_symmetry_looking__r_0.5',	'gz_symmetry_looking__r_0.55',	'gz_symmetry_looking__r_0.6000000000000001',	'gz_symmetry_looking__r_0.65',	'gz_symmetry_looking__r_0.7000000000000001',	'gz_symmetry_looking__r_0.75',	'gz_symmetry_looking__r_0.8',	'gz_symmetry_looking__r_0.8500000000000001',	'gz_symmetry_looking__r_0.9',	'gz_symmetry_looking__r_0.9500000000000001',	'gz_large_standard_deviation__r_0.05',	'gz_large_standard_deviation__r_0.1',	'gz_large_standard_deviation__r_0.15000000000000002',	'gz_large_standard_deviation__r_0.2',	'gz_large_standard_deviation__r_0.25',	'gz_large_standard_deviation__r_0.30000000000000004',	'gz_large_standard_deviation__r_0.35000000000000003',	'gz_large_standard_deviation__r_0.4',	'gz_large_standard_deviation__r_0.45',	'gz_large_standard_deviation__r_0.5',	'gz_large_standard_deviation__r_0.55',	'gz_large_standard_deviation__r_0.6000000000000001',	'gz_large_standard_deviation__r_0.65',	'gz_large_standard_deviation__r_0.7000000000000001',	'gz_large_standard_deviation__r_0.75',	'gz_large_standard_deviation__r_0.8',	'gz_large_standard_deviation__r_0.8500000000000001',	'gz_large_standard_deviation__r_0.9',	'gz_large_standard_deviation__r_0.9500000000000001',	'gz_quantile__q_0.1',	'gz_quantile__q_0.2',	'gz_quantile__q_0.3',	'gz_quantile__q_0.4',	'gz_quantile__q_0.6',	'gz_quantile__q_0.7',	'gz_quantile__q_0.8',	'gz_quantile__q_0.9',	'gz_autocorrelation__lag_0',	'gz_autocorrelation__lag_1',	'gz_autocorrelation__lag_2',	'gz_autocorrelation__lag_3',	'gz_autocorrelation__lag_4',	'gz_autocorrelation__lag_5',	'gz_autocorrelation__lag_6',	'gz_autocorrelation__lag_7',	'gz_autocorrelation__lag_8',	'gz_autocorrelation__lag_9',	'gz_agg_autocorrelation__f_agg_"mean"__maxlag_40',	'gz_agg_autocorrelation__f_agg_"median"__maxlag_40',	'gz_agg_autocorrelation__f_agg_"var"__maxlag_40',	'gz_partial_autocorrelation__lag_0',	'gz_partial_autocorrelation__lag_1',	'gz_partial_autocorrelation__lag_2',	'gz_partial_autocorrelation__lag_3',	'gz_partial_autocorrelation__lag_4',	'gz_partial_autocorrelation__lag_5',	'gz_partial_autocorrelation__lag_6',	'gz_partial_autocorrelation__lag_7',	'gz_partial_autocorrelation__lag_8',	'gz_partial_autocorrelation__lag_9',	'gz_number_cwt_peaks__n_1',	'gz_number_cwt_peaks__n_5',	'gz_number_peaks__n_1',	'gz_number_peaks__n_3',	'gz_number_peaks__n_5',	'gz_number_peaks__n_10',	'gz_number_peaks__n_50',	'gz_binned_entropy__max_bins_10',	'gz_index_mass_quantile__q_0.1',	'gz_index_mass_quantile__q_0.2',	'gz_index_mass_quantile__q_0.3',	'gz_index_mass_quantile__q_0.4',	'gz_index_mass_quantile__q_0.6',	'gz_index_mass_quantile__q_0.7',	'gz_index_mass_quantile__q_0.8',	'gz_index_mass_quantile__q_0.9',	'gz_cwt_coefficients__coeff_0__w_2__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_0__w_5__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_0__w_10__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_0__w_20__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_w_2__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_w_5__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_w_10__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_w_20__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_2__w_2__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_2__w_5__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_2__w_10__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_2__w_20__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_3__w_2__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_3__w_5__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_3__w_10__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_3__w_20__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_4__w_2__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_4__w_5__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_4__w_10__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_4__w_20__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_5__w_2__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_5__w_5__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_5__w_10__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_5__w_20__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_6__w_2__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_6__w_5__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_6__w_10__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_6__w_20__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_7__w_2__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_7__w_5__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_7__w_10__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_7__w_20__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_8__w_2__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_8__w_5__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_8__w_10__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_8__w_20__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_9__w_2__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_9__w_5__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_9__w_10__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_9__w_20__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_10__w_2__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_10__w_5__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_10__w_10__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_10__w_20__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_1w_2__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_1w_5__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_1w_10__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_1w_20__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_12__w_2__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_12__w_5__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_12__w_10__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_12__w_20__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_13__w_2__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_13__w_5__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_13__w_10__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_13__w_20__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_14__w_2__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_14__w_5__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_14__w_10__widths_(2 5 10 20)',	'gz_cwt_coefficients__coeff_14__w_20__widths_(2 5 10 20)',	'gz_spkt_welch_density__coeff_2',	'gz_spkt_welch_density__coeff_5',	'gz_spkt_welch_density__coeff_8',	'gz_ar_coefficient__coeff_0__k_10',	'gz_ar_coefficient__coeff_k_10',	'gz_ar_coefficient__coeff_2__k_10',	'gz_ar_coefficient__coeff_3__k_10',	'gz_ar_coefficient__coeff_4__k_10',	'gz_ar_coefficient__coeff_5__k_10',	'gz_ar_coefficient__coeff_6__k_10',	'gz_ar_coefficient__coeff_7__k_10',	'gz_ar_coefficient__coeff_8__k_10',	'gz_ar_coefficient__coeff_9__k_10',	'gz_ar_coefficient__coeff_10__k_10',	'gz_change_quantiles__f_agg_"mean"__isabs_False__qh_0.2__ql_0.0',	'gz_change_quantiles__f_agg_"var"__isabs_False__qh_0.2__ql_0.0',	'gz_change_quantiles__f_agg_"mean"__isabs_True__qh_0.2__ql_0.0',	'gz_change_quantiles__f_agg_"var"__isabs_True__qh_0.2__ql_0.0',	'gz_change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.0',	'gz_change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.0',	'gz_change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.0',	'gz_change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.0',	'gz_change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.0',	'gz_change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.0',	'gz_change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0',	'gz_change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.0',	'gz_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.0',	'gz_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0',	'gz_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0',	'gz_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.0',	'gz_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.0',	'gz_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.0',	'gz_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0',	'gz_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.0',	'gz_change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.2',	'gz_change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.2',	'gz_change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.2',	'gz_change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.2',	'gz_change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.2',	'gz_change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.2',	'gz_change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.2',	'gz_change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.2',	'gz_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.2',	'gz_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.2',	'gz_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2',	'gz_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.2',	'gz_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.2',	'gz_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.2',	'gz_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.2',	'gz_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.2',	'gz_change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.4',	'gz_change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.4',	'gz_change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.4',	'gz_change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.4',	'gz_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.4',	'gz_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.4',	'gz_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.4',	'gz_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.4',	'gz_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.4',	'gz_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.4',	'gz_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.4',	'gz_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.4',	'gz_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.6',	'gz_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.6',	'gz_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.6',	'gz_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.6',	'gz_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.6',	'gz_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.6',	'gz_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.6',	'gz_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.6',	'gz_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.8',	'gz_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.8',	'gz_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.8',	'gz_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.8',	'gz_fft_coefficient__attr_"real"__coeff_0',	'gz_fft_coefficient__attr_"real"__coeff_1',	'gz_fft_coefficient__attr_"real"__coeff_2',	'gz_fft_coefficient__attr_"real"__coeff_3',	'gz_fft_coefficient__attr_"real"__coeff_4',	'gz_fft_coefficient__attr_"real"__coeff_5',	'gz_fft_coefficient__attr_"real"__coeff_6',	'gz_fft_coefficient__attr_"real"__coeff_7',	'gz_fft_coefficient__attr_"real"__coeff_8',	'gz_fft_coefficient__attr_"real"__coeff_9',	'gz_fft_coefficient__attr_"real"__coeff_10',	'gz_fft_coefficient__attr_"real"__coeff_11',	'gz_fft_coefficient__attr_"real"__coeff_12',	'gz_fft_coefficient__attr_"real"__coeff_13',	'gz_fft_coefficient__attr_"real"__coeff_14',	'gz_fft_coefficient__attr_"real"__coeff_15',	'gz_fft_coefficient__attr_"real"__coeff_16',	'gz_fft_coefficient__attr_"real"__coeff_17',	'gz_fft_coefficient__attr_"real"__coeff_18',	'gz_fft_coefficient__attr_"real"__coeff_19',	'gz_fft_coefficient__attr_"real"__coeff_20',	'gz_fft_coefficient__attr_"real"__coeff_21',	'gz_fft_coefficient__attr_"real"__coeff_22',	'gz_fft_coefficient__attr_"real"__coeff_23',	'gz_fft_coefficient__attr_"real"__coeff_24',	'gz_fft_coefficient__attr_"real"__coeff_25',	'gz_fft_coefficient__attr_"real"__coeff_26',	'gz_fft_coefficient__attr_"real"__coeff_27',	'gz_fft_coefficient__attr_"real"__coeff_28',	'gz_fft_coefficient__attr_"real"__coeff_29',	'gz_fft_coefficient__attr_"real"__coeff_30',	'gz_fft_coefficient__attr_"real"__coeff_31',	'gz_fft_coefficient__attr_"real"__coeff_32',	'gz_fft_coefficient__attr_"real"__coeff_33',	'gz_fft_coefficient__attr_"real"__coeff_34',	'gz_fft_coefficient__attr_"real"__coeff_35',	'gz_fft_coefficient__attr_"real"__coeff_36',	'gz_fft_coefficient__attr_"real"__coeff_37',	'gz_fft_coefficient__attr_"real"__coeff_38',	'gz_fft_coefficient__attr_"real"__coeff_39',	'gz_fft_coefficient__attr_"real"__coeff_40',	'gz_fft_coefficient__attr_"real"__coeff_41',	'gz_fft_coefficient__attr_"real"__coeff_42',	'gz_fft_coefficient__attr_"real"__coeff_43',	'gz_fft_coefficient__attr_"real"__coeff_44',	'gz_fft_coefficient__attr_"real"__coeff_45',	'gz_fft_coefficient__attr_"real"__coeff_46',	'gz_fft_coefficient__attr_"real"__coeff_47',	'gz_fft_coefficient__attr_"real"__coeff_48',	'gz_fft_coefficient__attr_"real"__coeff_49',	'gz_fft_coefficient__attr_"real"__coeff_50',	'gz_fft_coefficient__attr_"real"__coeff_51',	'gz_fft_coefficient__attr_"real"__coeff_52',	'gz_fft_coefficient__attr_"real"__coeff_53',	'gz_fft_coefficient__attr_"real"__coeff_54',	'gz_fft_coefficient__attr_"real"__coeff_55',	'gz_fft_coefficient__attr_"real"__coeff_56',	'gz_fft_coefficient__attr_"real"__coeff_57',	'gz_fft_coefficient__attr_"real"__coeff_58',	'gz_fft_coefficient__attr_"real"__coeff_59',	'gz_fft_coefficient__attr_"real"__coeff_60',	'gz_fft_coefficient__attr_"real"__coeff_61',	'gz_fft_coefficient__attr_"real"__coeff_62',	'gz_fft_coefficient__attr_"real"__coeff_63',	'gz_fft_coefficient__attr_"real"__coeff_64',	'gz_fft_coefficient__attr_"real"__coeff_65',	'gz_fft_coefficient__attr_"real"__coeff_66',	'gz_fft_coefficient__attr_"real"__coeff_67',	'gz_fft_coefficient__attr_"real"__coeff_68',	'gz_fft_coefficient__attr_"real"__coeff_69',	'gz_fft_coefficient__attr_"real"__coeff_70',	'gz_fft_coefficient__attr_"real"__coeff_71',	'gz_fft_coefficient__attr_"real"__coeff_72',	'gz_fft_coefficient__attr_"real"__coeff_73',	'gz_fft_coefficient__attr_"real"__coeff_74',	'gz_fft_coefficient__attr_"real"__coeff_75',	'gz_fft_coefficient__attr_"real"__coeff_76',	'gz_fft_coefficient__attr_"real"__coeff_77',	'gz_fft_coefficient__attr_"real"__coeff_78',	'gz_fft_coefficient__attr_"real"__coeff_79',	'gz_fft_coefficient__attr_"real"__coeff_80',	'gz_fft_coefficient__attr_"real"__coeff_81',	'gz_fft_coefficient__attr_"real"__coeff_82',	'gz_fft_coefficient__attr_"real"__coeff_83',	'gz_fft_coefficient__attr_"real"__coeff_84',	'gz_fft_coefficient__attr_"real"__coeff_85',	'gz_fft_coefficient__attr_"real"__coeff_86',	'gz_fft_coefficient__attr_"real"__coeff_87',	'gz_fft_coefficient__attr_"real"__coeff_88',	'gz_fft_coefficient__attr_"real"__coeff_89',	'gz_fft_coefficient__attr_"real"__coeff_90',	'gz_fft_coefficient__attr_"real"__coeff_91',	'gz_fft_coefficient__attr_"real"__coeff_92',	'gz_fft_coefficient__attr_"real"__coeff_93',	'gz_fft_coefficient__attr_"real"__coeff_94',	'gz_fft_coefficient__attr_"real"__coeff_95',	'gz_fft_coefficient__attr_"real"__coeff_96',	'gz_fft_coefficient__attr_"real"__coeff_97',	'gz_fft_coefficient__attr_"real"__coeff_98',	'gz_fft_coefficient__attr_"real"__coeff_99',	'gz_fft_coefficient__attr_"imag"__coeff_0',	'gz_fft_coefficient__attr_"imag"__coeff_1',	'gz_fft_coefficient__attr_"imag"__coeff_2',	'gz_fft_coefficient__attr_"imag"__coeff_3',	'gz_fft_coefficient__attr_"imag"__coeff_4',	'gz_fft_coefficient__attr_"imag"__coeff_5',	'gz_fft_coefficient__attr_"imag"__coeff_6',	'gz_fft_coefficient__attr_"imag"__coeff_7',	'gz_fft_coefficient__attr_"imag"__coeff_8',	'gz_fft_coefficient__attr_"imag"__coeff_9',	'gz_fft_coefficient__attr_"imag"__coeff_10',	'gz_fft_coefficient__attr_"imag"__coeff_11',	'gz_fft_coefficient__attr_"imag"__coeff_12',	'gz_fft_coefficient__attr_"imag"__coeff_13',	'gz_fft_coefficient__attr_"imag"__coeff_14',	'gz_fft_coefficient__attr_"imag"__coeff_15',	'gz_fft_coefficient__attr_"imag"__coeff_16',	'gz_fft_coefficient__attr_"imag"__coeff_17',	'gz_fft_coefficient__attr_"imag"__coeff_18',	'gz_fft_coefficient__attr_"imag"__coeff_19',	'gz_fft_coefficient__attr_"imag"__coeff_20',	'gz_fft_coefficient__attr_"imag"__coeff_21',	'gz_fft_coefficient__attr_"imag"__coeff_22',	'gz_fft_coefficient__attr_"imag"__coeff_23',	'gz_fft_coefficient__attr_"imag"__coeff_24',	'gz_fft_coefficient__attr_"imag"__coeff_25',	'gz_fft_coefficient__attr_"imag"__coeff_26',	'gz_fft_coefficient__attr_"imag"__coeff_27',	'gz_fft_coefficient__attr_"imag"__coeff_28',	'gz_fft_coefficient__attr_"imag"__coeff_29',	'gz_fft_coefficient__attr_"imag"__coeff_30',	'gz_fft_coefficient__attr_"imag"__coeff_31',	'gz_fft_coefficient__attr_"imag"__coeff_32',	'gz_fft_coefficient__attr_"imag"__coeff_33',	'gz_fft_coefficient__attr_"imag"__coeff_34',	'gz_fft_coefficient__attr_"imag"__coeff_35',	'gz_fft_coefficient__attr_"imag"__coeff_36',	'gz_fft_coefficient__attr_"imag"__coeff_37',	'gz_fft_coefficient__attr_"imag"__coeff_38',	'gz_fft_coefficient__attr_"imag"__coeff_39',	'gz_fft_coefficient__attr_"imag"__coeff_40',	'gz_fft_coefficient__attr_"imag"__coeff_41',	'gz_fft_coefficient__attr_"imag"__coeff_42',	'gz_fft_coefficient__attr_"imag"__coeff_43',	'gz_fft_coefficient__attr_"imag"__coeff_44',	'gz_fft_coefficient__attr_"imag"__coeff_45',	'gz_fft_coefficient__attr_"imag"__coeff_46',	'gz_fft_coefficient__attr_"imag"__coeff_47',	'gz_fft_coefficient__attr_"imag"__coeff_48',	'gz_fft_coefficient__attr_"imag"__coeff_49',	'gz_fft_coefficient__attr_"imag"__coeff_50',	'gz_fft_coefficient__attr_"imag"__coeff_51',	'gz_fft_coefficient__attr_"imag"__coeff_52',	'gz_fft_coefficient__attr_"imag"__coeff_53',	'gz_fft_coefficient__attr_"imag"__coeff_54',	'gz_fft_coefficient__attr_"imag"__coeff_55',	'gz_fft_coefficient__attr_"imag"__coeff_56',	'gz_fft_coefficient__attr_"imag"__coeff_57',	'gz_fft_coefficient__attr_"imag"__coeff_58',	'gz_fft_coefficient__attr_"imag"__coeff_59',	'gz_fft_coefficient__attr_"imag"__coeff_60',	'gz_fft_coefficient__attr_"imag"__coeff_61',	'gz_fft_coefficient__attr_"imag"__coeff_62',	'gz_fft_coefficient__attr_"imag"__coeff_63',	'gz_fft_coefficient__attr_"imag"__coeff_64',	'gz_fft_coefficient__attr_"imag"__coeff_65',	'gz_fft_coefficient__attr_"imag"__coeff_66',	'gz_fft_coefficient__attr_"imag"__coeff_67',	'gz_fft_coefficient__attr_"imag"__coeff_68',	'gz_fft_coefficient__attr_"imag"__coeff_69',	'gz_fft_coefficient__attr_"imag"__coeff_70',	'gz_fft_coefficient__attr_"imag"__coeff_71',	'gz_fft_coefficient__attr_"imag"__coeff_72',	'gz_fft_coefficient__attr_"imag"__coeff_73',	'gz_fft_coefficient__attr_"imag"__coeff_74',	'gz_fft_coefficient__attr_"imag"__coeff_75',	'gz_fft_coefficient__attr_"imag"__coeff_76',	'gz_fft_coefficient__attr_"imag"__coeff_77',	'gz_fft_coefficient__attr_"imag"__coeff_78',	'gz_fft_coefficient__attr_"imag"__coeff_79',	'gz_fft_coefficient__attr_"imag"__coeff_80',	'gz_fft_coefficient__attr_"imag"__coeff_81',	'gz_fft_coefficient__attr_"imag"__coeff_82',	'gz_fft_coefficient__attr_"imag"__coeff_83',	'gz_fft_coefficient__attr_"imag"__coeff_84',	'gz_fft_coefficient__attr_"imag"__coeff_85',	'gz_fft_coefficient__attr_"imag"__coeff_86',	'gz_fft_coefficient__attr_"imag"__coeff_87',	'gz_fft_coefficient__attr_"imag"__coeff_88',	'gz_fft_coefficient__attr_"imag"__coeff_89',	'gz_fft_coefficient__attr_"imag"__coeff_90',	'gz_fft_coefficient__attr_"imag"__coeff_91',	'gz_fft_coefficient__attr_"imag"__coeff_92',	'gz_fft_coefficient__attr_"imag"__coeff_93',	'gz_fft_coefficient__attr_"imag"__coeff_94',	'gz_fft_coefficient__attr_"imag"__coeff_95',	'gz_fft_coefficient__attr_"imag"__coeff_96',	'gz_fft_coefficient__attr_"imag"__coeff_97',	'gz_fft_coefficient__attr_"imag"__coeff_98',	'gz_fft_coefficient__attr_"imag"__coeff_99',	'gz_fft_coefficient__attr_"abs"__coeff_0',	'gz_fft_coefficient__attr_"abs"__coeff_1',	'gz_fft_coefficient__attr_"abs"__coeff_2',	'gz_fft_coefficient__attr_"abs"__coeff_3',	'gz_fft_coefficient__attr_"abs"__coeff_4',	'gz_fft_coefficient__attr_"abs"__coeff_5',	'gz_fft_coefficient__attr_"abs"__coeff_6',	'gz_fft_coefficient__attr_"abs"__coeff_7',	'gz_fft_coefficient__attr_"abs"__coeff_8',	'gz_fft_coefficient__attr_"abs"__coeff_9',	'gz_fft_coefficient__attr_"abs"__coeff_10',	'gz_fft_coefficient__attr_"abs"__coeff_11',	'gz_fft_coefficient__attr_"abs"__coeff_12',	'gz_fft_coefficient__attr_"abs"__coeff_13',	'gz_fft_coefficient__attr_"abs"__coeff_14',	'gz_fft_coefficient__attr_"abs"__coeff_15',	'gz_fft_coefficient__attr_"abs"__coeff_16',	'gz_fft_coefficient__attr_"abs"__coeff_17',	'gz_fft_coefficient__attr_"abs"__coeff_18',	'gz_fft_coefficient__attr_"abs"__coeff_19',	'gz_fft_coefficient__attr_"abs"__coeff_20',	'gz_fft_coefficient__attr_"abs"__coeff_21',	'gz_fft_coefficient__attr_"abs"__coeff_22',	'gz_fft_coefficient__attr_"abs"__coeff_23',	'gz_fft_coefficient__attr_"abs"__coeff_24',	'gz_fft_coefficient__attr_"abs"__coeff_25',	'gz_fft_coefficient__attr_"abs"__coeff_26',	'gz_fft_coefficient__attr_"abs"__coeff_27',	'gz_fft_coefficient__attr_"abs"__coeff_28',	'gz_fft_coefficient__attr_"abs"__coeff_29',	'gz_fft_coefficient__attr_"abs"__coeff_30',	'gz_fft_coefficient__attr_"abs"__coeff_31',	'gz_fft_coefficient__attr_"abs"__coeff_32',	'gz_fft_coefficient__attr_"abs"__coeff_33',	'gz_fft_coefficient__attr_"abs"__coeff_34',	'gz_fft_coefficient__attr_"abs"__coeff_35',	'gz_fft_coefficient__attr_"abs"__coeff_36',	'gz_fft_coefficient__attr_"abs"__coeff_37',	'gz_fft_coefficient__attr_"abs"__coeff_38',	'gz_fft_coefficient__attr_"abs"__coeff_39',	'gz_fft_coefficient__attr_"abs"__coeff_40',	'gz_fft_coefficient__attr_"abs"__coeff_41',	'gz_fft_coefficient__attr_"abs"__coeff_42',	'gz_fft_coefficient__attr_"abs"__coeff_43',	'gz_fft_coefficient__attr_"abs"__coeff_44',	'gz_fft_coefficient__attr_"abs"__coeff_45',	'gz_fft_coefficient__attr_"abs"__coeff_46',	'gz_fft_coefficient__attr_"abs"__coeff_47',	'gz_fft_coefficient__attr_"abs"__coeff_48',	'gz_fft_coefficient__attr_"abs"__coeff_49',	'gz_fft_coefficient__attr_"abs"__coeff_50',	'gz_fft_coefficient__attr_"abs"__coeff_51',	'gz_fft_coefficient__attr_"abs"__coeff_52',	'gz_fft_coefficient__attr_"abs"__coeff_53',	'gz_fft_coefficient__attr_"abs"__coeff_54',	'gz_fft_coefficient__attr_"abs"__coeff_55',	'gz_fft_coefficient__attr_"abs"__coeff_56',	'gz_fft_coefficient__attr_"abs"__coeff_57',	'gz_fft_coefficient__attr_"abs"__coeff_58',	'gz_fft_coefficient__attr_"abs"__coeff_59',	'gz_fft_coefficient__attr_"abs"__coeff_60',	'gz_fft_coefficient__attr_"abs"__coeff_61',	'gz_fft_coefficient__attr_"abs"__coeff_62',	'gz_fft_coefficient__attr_"abs"__coeff_63',	'gz_fft_coefficient__attr_"abs"__coeff_64',	'gz_fft_coefficient__attr_"abs"__coeff_65',	'gz_fft_coefficient__attr_"abs"__coeff_66',	'gz_fft_coefficient__attr_"abs"__coeff_67',	'gz_fft_coefficient__attr_"abs"__coeff_68',	'gz_fft_coefficient__attr_"abs"__coeff_69',	'gz_fft_coefficient__attr_"abs"__coeff_70',	'gz_fft_coefficient__attr_"abs"__coeff_71',	'gz_fft_coefficient__attr_"abs"__coeff_72',	'gz_fft_coefficient__attr_"abs"__coeff_73',	'gz_fft_coefficient__attr_"abs"__coeff_74',	'gz_fft_coefficient__attr_"abs"__coeff_75',	'gz_fft_coefficient__attr_"abs"__coeff_76',	'gz_fft_coefficient__attr_"abs"__coeff_77',	'gz_fft_coefficient__attr_"abs"__coeff_78',	'gz_fft_coefficient__attr_"abs"__coeff_79',	'gz_fft_coefficient__attr_"abs"__coeff_80',	'gz_fft_coefficient__attr_"abs"__coeff_81',	'gz_fft_coefficient__attr_"abs"__coeff_82',	'gz_fft_coefficient__attr_"abs"__coeff_83',	'gz_fft_coefficient__attr_"abs"__coeff_84',	'gz_fft_coefficient__attr_"abs"__coeff_85',	'gz_fft_coefficient__attr_"abs"__coeff_86',	'gz_fft_coefficient__attr_"abs"__coeff_87',	'gz_fft_coefficient__attr_"abs"__coeff_88',	'gz_fft_coefficient__attr_"abs"__coeff_89',	'gz_fft_coefficient__attr_"abs"__coeff_90',	'gz_fft_coefficient__attr_"abs"__coeff_91',	'gz_fft_coefficient__attr_"abs"__coeff_92',	'gz_fft_coefficient__attr_"abs"__coeff_93',	'gz_fft_coefficient__attr_"abs"__coeff_94',	'gz_fft_coefficient__attr_"abs"__coeff_95',	'gz_fft_coefficient__attr_"abs"__coeff_96',	'gz_fft_coefficient__attr_"abs"__coeff_97',	'gz_fft_coefficient__attr_"abs"__coeff_98',	'gz_fft_coefficient__attr_"abs"__coeff_99',	'gz_fft_coefficient__attr_"angle"__coeff_0',	'gz_fft_coefficient__attr_"angle"__coeff_1',	'gz_fft_coefficient__attr_"angle"__coeff_2',	'gz_fft_coefficient__attr_"angle"__coeff_3',	'gz_fft_coefficient__attr_"angle"__coeff_4',	'gz_fft_coefficient__attr_"angle"__coeff_5',	'gz_fft_coefficient__attr_"angle"__coeff_6',	'gz_fft_coefficient__attr_"angle"__coeff_7',	'gz_fft_coefficient__attr_"angle"__coeff_8',	'gz_fft_coefficient__attr_"angle"__coeff_9',	'gz_fft_coefficient__attr_"angle"__coeff_10',	'gz_fft_coefficient__attr_"angle"__coeff_11',	'gz_fft_coefficient__attr_"angle"__coeff_12',	'gz_fft_coefficient__attr_"angle"__coeff_13',	'gz_fft_coefficient__attr_"angle"__coeff_14',	'gz_fft_coefficient__attr_"angle"__coeff_15',	'gz_fft_coefficient__attr_"angle"__coeff_16',	'gz_fft_coefficient__attr_"angle"__coeff_17',	'gz_fft_coefficient__attr_"angle"__coeff_18',	'gz_fft_coefficient__attr_"angle"__coeff_19',	'gz_fft_coefficient__attr_"angle"__coeff_20',	'gz_fft_coefficient__attr_"angle"__coeff_21',	'gz_fft_coefficient__attr_"angle"__coeff_22',	'gz_fft_coefficient__attr_"angle"__coeff_23',	'gz_fft_coefficient__attr_"angle"__coeff_24',	'gz_fft_coefficient__attr_"angle"__coeff_25',	'gz_fft_coefficient__attr_"angle"__coeff_26',	'gz_fft_coefficient__attr_"angle"__coeff_27',	'gz_fft_coefficient__attr_"angle"__coeff_28',	'gz_fft_coefficient__attr_"angle"__coeff_29',	'gz_fft_coefficient__attr_"angle"__coeff_30',	'gz_fft_coefficient__attr_"angle"__coeff_31',	'gz_fft_coefficient__attr_"angle"__coeff_32',	'gz_fft_coefficient__attr_"angle"__coeff_33',	'gz_fft_coefficient__attr_"angle"__coeff_34',	'gz_fft_coefficient__attr_"angle"__coeff_35',	'gz_fft_coefficient__attr_"angle"__coeff_36',	'gz_fft_coefficient__attr_"angle"__coeff_37',	'gz_fft_coefficient__attr_"angle"__coeff_38',	'gz_fft_coefficient__attr_"angle"__coeff_39',	'gz_fft_coefficient__attr_"angle"__coeff_40',	'gz_fft_coefficient__attr_"angle"__coeff_41',	'gz_fft_coefficient__attr_"angle"__coeff_42',	'gz_fft_coefficient__attr_"angle"__coeff_43',	'gz_fft_coefficient__attr_"angle"__coeff_44',	'gz_fft_coefficient__attr_"angle"__coeff_45',	'gz_fft_coefficient__attr_"angle"__coeff_46',	'gz_fft_coefficient__attr_"angle"__coeff_47',	'gz_fft_coefficient__attr_"angle"__coeff_48',	'gz_fft_coefficient__attr_"angle"__coeff_49',	'gz_fft_coefficient__attr_"angle"__coeff_50',	'gz_fft_coefficient__attr_"angle"__coeff_51',	'gz_fft_coefficient__attr_"angle"__coeff_52',	'gz_fft_coefficient__attr_"angle"__coeff_53',	'gz_fft_coefficient__attr_"angle"__coeff_54',	'gz_fft_coefficient__attr_"angle"__coeff_55',	'gz_fft_coefficient__attr_"angle"__coeff_56',	'gz_fft_coefficient__attr_"angle"__coeff_57',	'gz_fft_coefficient__attr_"angle"__coeff_58',	'gz_fft_coefficient__attr_"angle"__coeff_59',	'gz_fft_coefficient__attr_"angle"__coeff_60',	'gz_fft_coefficient__attr_"angle"__coeff_61',	'gz_fft_coefficient__attr_"angle"__coeff_62',	'gz_fft_coefficient__attr_"angle"__coeff_63',	'gz_fft_coefficient__attr_"angle"__coeff_64',	'gz_fft_coefficient__attr_"angle"__coeff_65',	'gz_fft_coefficient__attr_"angle"__coeff_66',	'gz_fft_coefficient__attr_"angle"__coeff_67',	'gz_fft_coefficient__attr_"angle"__coeff_68',	'gz_fft_coefficient__attr_"angle"__coeff_69',	'gz_fft_coefficient__attr_"angle"__coeff_70',	'gz_fft_coefficient__attr_"angle"__coeff_71',	'gz_fft_coefficient__attr_"angle"__coeff_72',	'gz_fft_coefficient__attr_"angle"__coeff_73',	'gz_fft_coefficient__attr_"angle"__coeff_74',	'gz_fft_coefficient__attr_"angle"__coeff_75',	'gz_fft_coefficient__attr_"angle"__coeff_76',	'gz_fft_coefficient__attr_"angle"__coeff_77',	'gz_fft_coefficient__attr_"angle"__coeff_78',	'gz_fft_coefficient__attr_"angle"__coeff_79',	'gz_fft_coefficient__attr_"angle"__coeff_80',	'gz_fft_coefficient__attr_"angle"__coeff_81',	'gz_fft_coefficient__attr_"angle"__coeff_82',	'gz_fft_coefficient__attr_"angle"__coeff_83',	'gz_fft_coefficient__attr_"angle"__coeff_84',	'gz_fft_coefficient__attr_"angle"__coeff_85',	'gz_fft_coefficient__attr_"angle"__coeff_86',	'gz_fft_coefficient__attr_"angle"__coeff_87',	'gz_fft_coefficient__attr_"angle"__coeff_88',	'gz_fft_coefficient__attr_"angle"__coeff_89',	'gz_fft_coefficient__attr_"angle"__coeff_90',	'gz_fft_coefficient__attr_"angle"__coeff_91',	'gz_fft_coefficient__attr_"angle"__coeff_92',	'gz_fft_coefficient__attr_"angle"__coeff_93',	'gz_fft_coefficient__attr_"angle"__coeff_94',	'gz_fft_coefficient__attr_"angle"__coeff_95',	'gz_fft_coefficient__attr_"angle"__coeff_96',	'gz_fft_coefficient__attr_"angle"__coeff_97',	'gz_fft_coefficient__attr_"angle"__coeff_98',	'gz_fft_coefficient__attr_"angle"__coeff_99',	'gz_fft_aggregated__aggtype_"centroid"',	'gz_fft_aggregated__aggtype_"variance"',	'gz_fft_aggregated__aggtype_"skew"',	'gz_fft_aggregated__aggtype_"kurtosis"',	'gz_value_count__value_0',	'gz_value_count__value_1',	'gz_value_count__value_-1',	'gz_range_count__max_min_-1',	'gz_range_count__max_0__min_1000000000000.0',	'gz_range_count__max_1000000000000.0__min_0',	'gz_approximate_entropy__m_2__r_0.1',	'gz_approximate_entropy__m_2__r_0.3',	'gz_approximate_entropy__m_2__r_0.5',	'gz_approximate_entropy__m_2__r_0.7',	'gz_approximate_entropy__m_2__r_0.9',	'gz_friedrich_coefficients__coeff_0__m_3__r_30',	'gz_friedrich_coefficients__coeff_m_3__r_30',	'gz_friedrich_coefficients__coeff_2__m_3__r_30',	'gz_friedrich_coefficients__coeff_3__m_3__r_30',	'gz_max_langevin_fixed_point__m_3__r_30',	'gz_linear_trend__attr_"pvalue"',	'gz_linear_trend__attr_"rvalue"',	'gz_linear_trend__attr_"intercept"',	'gz_linear_trend__attr_"slope"',	'gz_linear_trend__attr_"stderr"',	'gz_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"max"',	'gz_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"min"',	'gz_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"mean"',	'gz_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"var"',	'gz_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"max"',	'gz_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"min"',	'gz_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"mean"',	'gz_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"var"',	'gz_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"max"',	'gz_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"min"',	'gz_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"mean"',	'gz_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"var"',	'gz_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"max"',	'gz_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"min"',	'gz_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"mean"',	'gz_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"var"',	'gz_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"max"',	'gz_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"min"',	'gz_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"mean"',	'gz_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"var"',	'gz_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"max"',	'gz_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"min"',	'gz_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"mean"',	'gz_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"var"',	'gz_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"max"',	'gz_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"min"',	'gz_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"mean"',	'gz_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"var"',	'gz_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"max"',	'gz_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"min"',	'gz_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"mean"',	'gz_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"var"',	'gz_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"max"',	'gz_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"min"',	'gz_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"mean"',	'gz_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"var"',	'gz_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"max"',	'gz_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"min"',	'gz_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"mean"',	'gz_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"var"',	'gz_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"max"',	'gz_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"min"',	'gz_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"mean"',	'gz_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"var"',	'gz_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"max"',	'gz_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"min"',	'gz_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"mean"',	'gz_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"var"',	'gz_augmented_dickey_fuller__attr_"teststat"__autolag_"AIC"',	'gz_augmented_dickey_fuller__attr_"pvalue"__autolag_"AIC"',	'gz_augmented_dickey_fuller__attr_"usedlag"__autolag_"AIC"',	'gz_number_crossing_m__m_0',	'gz_number_crossing_m__m_-1',	'gz_number_crossing_m__m_1',	'gz_energz_ratio_by_chunks__num_segments_10__segment_focus_0',	'gz_energz_ratio_by_chunks__num_segments_10__segment_focus_1',	'gz_energz_ratio_by_chunks__num_segments_10__segment_focus_2',	'gz_energz_ratio_by_chunks__num_segments_10__segment_focus_3',	'gz_energz_ratio_by_chunks__num_segments_10__segment_focus_4',	'gz_energz_ratio_by_chunks__num_segments_10__segment_focus_5',	'gz_energz_ratio_by_chunks__num_segments_10__segment_focus_6',	'gz_energz_ratio_by_chunks__num_segments_10__segment_focus_7',	'gz_energz_ratio_by_chunks__num_segments_10__segment_focus_8',	'gz_energz_ratio_by_chunks__num_segments_10__segment_focus_9',	'gz_ratio_beyond_r_sigma__r_0.5',	'gz_ratio_beyond_r_sigma__r_1',	'gz_ratio_beyond_r_sigma__r_1.5',	'gz_ratio_beyond_r_sigma__r_2',	'gz_ratio_beyond_r_sigma__r_2.5',	'gz_ratio_beyond_r_sigma__r_3',	'gz_ratio_beyond_r_sigma__r_5',	'gz_ratio_beyond_r_sigma__r_6',	'gz_ratio_beyond_r_sigma__r_7',	'gz_ratio_beyond_r_sigma__r_10',	'gz_count_above__t_0',	'gz_count_below__t_0',	'gz_lempel_ziv_complexity__bins_2',	'gz_lempel_ziv_complexity__bins_3',	'gz_lempel_ziv_complexity__bins_5',	'gz_lempel_ziv_complexity__bins_10',	'gz_lempel_ziv_complexity__bins_100',	'gz_fourier_entropy__bins_2',	'gz_fourier_entropy__bins_3',	'gz_fourier_entropy__bins_5',	'gz_fourier_entropy__bins_10',	'gz_fourier_entropy__bins_100',	'gz_permutation_entropy__dimension_3__tau_1',	'gz_permutation_entropy__dimension_4__tau_1',	'gz_permutation_entropy__dimension_5__tau_1',	'gz_permutation_entropy__dimension_6__tau_1',	'gz_permutation_entropy__dimension_7__tau_1',	'gz_query_similarity_count__query_None__threshold_0.0',	'gz_matrix_profile__feature_"min"__threshold_0.98',	'gz_matrix_profile__feature_"max"__threshold_0.98',	'gz_matrix_profile__feature_"mean"__threshold_0.98',	'gz_matrix_profile__feature_"median"__threshold_0.98',	'gz_matrix_profile__feature_"25"__threshold_0.98',	'gz_matrix_profile__feature_"75"__threshold_0.98']
fea_user_1gz=pd.DataFrame(fea_user_1gz.reshape(Gx_list_trans.shape[1], 787))
fea_user_1gz.columns=[header_gz]
fea_user_1gz

,gz_variance_larger_than_standard_deviation,gz_has_duplicate_max,gz_has_duplicate_min,gz_has_duplicate,gz_sum_values,gz_abs_energz,gz_mean_abs_change,gz_mean_change,gz_mean_second_derivative_central,gz_median,...,gz_permutation_entropy__dimension_5__tau_1,gz_permutation_entropy__dimension_6__tau_1,gz_permutation_entropy__dimension_7__tau_1,gz_query_similarity_count__query_None__threshold_0.0,"gz_matrix_profile__feature_""min""__threshold_0.98","gz_matrix_profile__feature_""max""__threshold_0.98","gz_matrix_profile__feature_""mean""__threshold_0.98","gz_matrix_profile__feature_""median""__threshold_0.98","gz_matrix_profile__feature_""25""__threshold_0.98","gz_matrix_profile__feature_""75""__threshold_0.98"
0,0.0,1.0,0.0,1.0,19.4187,18.854298,0.000089,0.000026,-0.000011,0.97095,...,1.037392,1.172993,1.226556,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,1.0,1.0,1.0,19.4166,18.850221,0.000063,-0.000042,-0.000003,0.97075,...,1.754105,2.079151,2.242973,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,1.0,19.4034,18.824599,0.000063,-0.000063,0.000000,0.97035,...,2.133382,2.395908,2.540036,NaN,1.182025,2.050256,1.667833,1.731859,1.535379,1.838409
3,0.0,0.0,1.0,1.0,19.3891,18.796865,0.000084,-0.000084,0.000003,0.96945,...,1.455333,1.748971,1.908535,NaN,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
4,0.0,0.0,1.0,1.0,19.3758,18.771083,0.000053,-0.000053,0.000003,0.96865,...,2.393312,2.615631,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,0.0,0.0,1.0,1.0,19.4119,18.841098,0.000221,-0.000053,0.000003,0.97055,...,2.479956,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
805,0.0,0.0,1.0,1.0,19.4046,18.826928,0.000137,-0.000084,0.000014,0.97015,...,2.182996,2.338372,2.403640,NaN,NaN,NaN,NaN,NaN,NaN,NaN
806,0.0,0.0,1.0,1.0,19.3983,18.814703,0.000063,-0.000011,0.000003,0.96990,...,1.754105,2.118649,2.441015,NaN,NaN,NaN,NaN,NaN,NaN,NaN
807,0.0,1.0,1.0,1.0,19.3926,18.803648,0.000063,-0.000032,0.000003,0.96965,...,1.754105,1.951848,2.143952,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
r_gz=fea_user_1gz.dropna(axis='columns',how='all')
r_gz=r_gz.fillna(0)
binary_features_gz=['gz_variance_larger_than_standard_deviation','gz_has_duplicate_max',	'gz_has_duplicate_min',	'gz_has_duplicate','gz_symmetry_looking__r_0.0',	'gz_symmetry_looking__r_0.05',	'gz_symmetry_looking__r_0.1',	'gz_symmetry_looking__r_0.15000000000000002',	'gz_symmetry_looking__r_0.2',	'gz_symmetry_looking__r_0.25',	'gz_symmetry_looking__r_0.30000000000000004',	'gz_symmetry_looking__r_0.35000000000000003',	'gz_symmetry_looking__r_0.4',	'gz_symmetry_looking__r_0.45',	'gz_symmetry_looking__r_0.5',	'gz_symmetry_looking__r_0.55',	'gz_symmetry_looking__r_0.6000000000000001',	'gz_symmetry_looking__r_0.65',	'gz_symmetry_looking__r_0.7000000000000001',	'gz_symmetry_looking__r_0.75',	'gz_symmetry_looking__r_0.8',	'gz_symmetry_looking__r_0.8500000000000001',	'gz_symmetry_looking__r_0.9',	'gz_symmetry_looking__r_0.9500000000000001',	'gz_large_standard_deviation__r_0.05',	'gz_large_standard_deviation__r_0.1',	'gz_large_standard_deviation__r_0.15000000000000002',	'gz_large_standard_deviation__r_0.2',	'gz_large_standard_deviation__r_0.25',	'gz_large_standard_deviation__r_0.30000000000000004',	'gz_large_standard_deviation__r_0.35000000000000003',	'gz_large_standard_deviation__r_0.4',	'gz_large_standard_deviation__r_0.45',	'gz_large_standard_deviation__r_0.5',	'gz_large_standard_deviation__r_0.55',	'gz_large_standard_deviation__r_0.6000000000000001',	'gz_large_standard_deviation__r_0.65',	'gz_large_standard_deviation__r_0.7000000000000001',	'gz_large_standard_deviation__r_0.75',	'gz_large_standard_deviation__r_0.8',	'gz_large_standard_deviation__r_0.8500000000000001',	'gz_large_standard_deviation__r_0.9',	'gz_large_standard_deviation__r_0.9500000000000001','gz_value_count__value_0',	'gz_value_count__value_1',	'gz_value_count__value_-1']
r_gz=r_gz.drop(binary_features_gz, axis=1)
r_gz

d:\m-tech-data-science\tools\python 3.7\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,gz_sum_values,gz_abs_energz,gz_mean_abs_change,gz_mean_change,gz_mean_second_derivative_central,gz_median,gz_mean,gz_length,gz_standard_deviation,gz_variation_coefficient,...,gz_permutation_entropy__dimension_4__tau_1,gz_permutation_entropy__dimension_5__tau_1,gz_permutation_entropy__dimension_6__tau_1,gz_permutation_entropy__dimension_7__tau_1,"gz_matrix_profile__feature_""min""__threshold_0.98","gz_matrix_profile__feature_""max""__threshold_0.98","gz_matrix_profile__feature_""mean""__threshold_0.98","gz_matrix_profile__feature_""median""__threshold_0.98","gz_matrix_profile__feature_""25""__threshold_0.98","gz_matrix_profile__feature_""75""__threshold_0.98"
0,19.4187,18.854298,0.000089,0.000026,-0.000011,0.97095,0.970935,20.0,0.000323,0.000333,...,0.885289,1.037392,1.172993,1.226556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,19.4166,18.850221,0.000063,-0.000042,-0.000003,0.97075,0.970830,20.0,0.000398,0.000410,...,1.399571,1.754105,2.079151,2.242973,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,19.4034,18.824599,0.000063,-0.000063,0.000000,0.97035,0.970170,20.0,0.000348,0.000359,...,1.711547,2.133382,2.395908,2.540036,1.182025,2.050256,1.667833,1.731859,1.535379,1.838409
3,19.3891,18.796865,0.000084,-0.000084,0.000003,0.96945,0.969455,20.0,0.000490,0.000506,...,1.121785,1.455333,1.748971,1.908535,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
4,19.3758,18.771083,0.000053,-0.000053,0.000003,0.96865,0.968790,20.0,0.000298,0.000308,...,1.956187,2.393312,2.615631,2.639057,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,19.4119,18.841098,0.000221,-0.000053,0.000003,0.97055,0.970595,20.0,0.000494,0.000509,...,2.150060,2.479956,2.708050,2.639057,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
805,19.4046,18.826928,0.000137,-0.000084,0.000014,0.97015,0.970230,20.0,0.000362,0.000373,...,1.843861,2.182996,2.338372,2.403640,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
806,19.3983,18.814703,0.000063,-0.000011,0.000003,0.96990,0.969915,20.0,0.000231,0.000238,...,1.381289,1.754105,2.118649,2.441015,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
807,19.3926,18.803648,0.000063,-0.000032,0.000003,0.96965,0.969630,20.0,0.000212,0.000219,...,1.477045,1.754105,1.951848,2.143952,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### X-axis accelerometer 

In [20]:
Ax_list=pd.DataFrame(Ax_list)
Ax_list=Ax_list.astype(float)
Ax_list_trans=Ax_list.T
Ax_list_trans

,0,1,2,3,4,5,6,7,8,9,...,799,800,801,802,803,804,805,806,807,808
0,-0.0140,-0.0096,0.0053,-0.0553,0.0234,0.0105,-0.0541,-0.1604,0.0253,0.0604,...,0.0754,0.0066,0.0623,0.0027,-0.0434,0.0097,0.1199,0.0435,-0.0207,0.1015
1,0.0769,0.0789,0.0013,0.0496,0.0039,0.0458,-0.1278,-0.0986,0.0553,0.0294,...,0.0639,-0.0518,-0.0340,0.0284,-0.0555,0.0011,0.0726,-0.0033,-0.0444,0.1291
2,-0.0602,0.0508,0.0733,-0.0303,0.0109,0.0203,-0.0332,-0.0832,0.0326,-0.0036,...,0.0922,-0.0254,-0.0256,0.0456,0.0155,0.0030,0.0142,-0.0042,-0.0443,0.0579
3,-0.0434,-0.0246,0.0179,-0.0162,-0.0033,0.0482,-0.0511,-0.0115,-0.0423,0.0132,...,0.0129,0.0018,-0.0987,0.0462,-0.0102,-0.0417,0.0444,0.0112,-0.0753,0.0441
4,-0.0332,0.0108,0.0173,0.0267,0.0237,0.0319,-0.0463,0.0183,-0.0512,-0.0314,...,0.0006,-0.0485,-0.0720,0.0028,0.0404,-0.0440,0.0077,0.0589,-0.0481,0.0226
5,0.0445,-0.0105,0.0087,0.0272,-0.0129,0.0063,-0.0242,-0.0399,0.0001,-0.0711,...,0.0054,-0.0550,-0.0382,0.0056,-0.0016,-0.1471,0.0067,0.0040,-0.0125,0.0033
6,-0.0242,-0.0199,-0.0104,0.0305,-0.0834,-0.0094,0.0101,-0.0146,0.0668,-0.0274,...,-0.0010,0.0421,-0.0605,0.0188,0.0167,0.1411,-0.0401,0.0063,0.0178,-0.0531
7,-0.0258,-0.0290,-0.0118,0.0252,-0.0098,0.0421,-0.1063,0.0551,0.0473,0.0236,...,-0.0068,0.0450,-0.0476,-0.0015,0.0071,-0.1723,-0.0615,0.0414,0.0771,-0.0497
8,0.0862,-0.0214,-0.0350,0.0125,-0.0590,0.0386,0.1824,0.0123,-0.0398,0.0156,...,-0.0029,0.0522,-0.0056,-0.0160,-0.0061,0.1565,0.0508,0.0222,0.0055,-0.0305
9,0.0076,-0.1128,-0.0359,0.0126,-0.0144,-0.0307,0.1093,0.0841,0.0016,0.0003,...,0.0120,0.1141,-0.0011,-0.0057,-0.0351,0.0404,0.0070,0.0209,-0.0102,-0.0070


In [21]:
from tsfresh import extract_features
#c=c.fillna(0)
l_ax=[]
z_ax=[]
u_ax=[]
X_ax=[]
for i in range (Gx_list_trans.shape[1]):
    p_ax=Ax_list_trans.iloc[:,i]
    p_ax=pd.DataFrame(p_ax)
    p_ax['ID'] = '1'
    X_ax= extract_features(p_ax, column_id='ID')
    #print(X.shape)
    l_ax.append(X_ax)
    #m.append(l)
    z_ax=l_ax
u_ax=z_ax  

Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.68s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.33s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.78s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.02s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.16s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.62s/it]


In [22]:
fea_user_1ax=np.array(u_ax)
header_ax=['ax_variance_larger_than_standard_deviation',	'ax_has_duplicate_max',	'ax_has_duplicate_min',	'ax_has_duplicate',	'ax_sum_values',	'ax_abs_enerax',	'ax_mean_abs_change',	'ax_mean_change',	'ax_mean_second_derivative_central',	'ax_median',	'ax_mean',	'ax_length',	'ax_standard_deviation',	'ax_variation_coefficient',	'ax_variance',	'ax_skewness',	'ax_kurtosis',	'ax_root_mean_square',	'ax_absolute_sum_of_changes',	'ax_longest_strike_below_mean',	'ax_longest_strike_above_mean',	'ax_count_above_mean',	'ax_count_below_mean',	'ax_last_location_of_maximum',	'ax_first_location_of_maximum',	'ax_last_location_of_minimum',	'ax_first_location_of_minimum',	'ax_percentage_of_reoccurring_values_to_all_values',	'ax_percentage_of_reoccurring_datapoints_to_all_datapoints',	'ax_sum_of_reoccurring_values',	'ax_sum_of_reoccurring_data_points',	'ax_ratio_value_number_to_time_series_length',	'ax_sample_entropy',	'ax_maximum',	'ax_minimum',	'ax_benford_correlation',	'ax_time_reversal_asymmetry_statistic__lag_1',	'ax_time_reversal_asymmetry_statistic__lag_2',	'ax_time_reversal_asymmetry_statistic__lag_3',	'ax_c3__lag_1',	'ax_c3__lag_2',	'ax_c3__lag_3',	'ax_cid_ce__normalize_True',	'ax_cid_ce__normalize_False',	'ax_symmetry_looking__r_0.0',	'ax_symmetry_looking__r_0.05',	'ax_symmetry_looking__r_0.1',	'ax_symmetry_looking__r_0.15000000000000002',	'ax_symmetry_looking__r_0.2',	'ax_symmetry_looking__r_0.25',	'ax_symmetry_looking__r_0.30000000000000004',	'ax_symmetry_looking__r_0.35000000000000003',	'ax_symmetry_looking__r_0.4',	'ax_symmetry_looking__r_0.45',	'ax_symmetry_looking__r_0.5',	'ax_symmetry_looking__r_0.55',	'ax_symmetry_looking__r_0.6000000000000001',	'ax_symmetry_looking__r_0.65',	'ax_symmetry_looking__r_0.7000000000000001',	'ax_symmetry_looking__r_0.75',	'ax_symmetry_looking__r_0.8',	'ax_symmetry_looking__r_0.8500000000000001',	'ax_symmetry_looking__r_0.9',	'ax_symmetry_looking__r_0.9500000000000001',	'ax_large_standard_deviation__r_0.05',	'ax_large_standard_deviation__r_0.1',	'ax_large_standard_deviation__r_0.15000000000000002',	'ax_large_standard_deviation__r_0.2',	'ax_large_standard_deviation__r_0.25',	'ax_large_standard_deviation__r_0.30000000000000004',	'ax_large_standard_deviation__r_0.35000000000000003',	'ax_large_standard_deviation__r_0.4',	'ax_large_standard_deviation__r_0.45',	'ax_large_standard_deviation__r_0.5',	'ax_large_standard_deviation__r_0.55',	'ax_large_standard_deviation__r_0.6000000000000001',	'ax_large_standard_deviation__r_0.65',	'ax_large_standard_deviation__r_0.7000000000000001',	'ax_large_standard_deviation__r_0.75',	'ax_large_standard_deviation__r_0.8',	'ax_large_standard_deviation__r_0.8500000000000001',	'ax_large_standard_deviation__r_0.9',	'ax_large_standard_deviation__r_0.9500000000000001',	'ax_quantile__q_0.1',	'ax_quantile__q_0.2',	'ax_quantile__q_0.3',	'ax_quantile__q_0.4',	'ax_quantile__q_0.6',	'ax_quantile__q_0.7',	'ax_quantile__q_0.8',	'ax_quantile__q_0.9',	'ax_autocorrelation__lag_0',	'ax_autocorrelation__lag_1',	'ax_autocorrelation__lag_2',	'ax_autocorrelation__lag_3',	'ax_autocorrelation__lag_4',	'ax_autocorrelation__lag_5',	'ax_autocorrelation__lag_6',	'ax_autocorrelation__lag_7',	'ax_autocorrelation__lag_8',	'ax_autocorrelation__lag_9',	'ax_agg_autocorrelation__f_agg_"mean"__maxlag_40',	'ax_agg_autocorrelation__f_agg_"median"__maxlag_40',	'ax_agg_autocorrelation__f_agg_"var"__maxlag_40',	'ax_partial_autocorrelation__lag_0',	'ax_partial_autocorrelation__lag_1',	'ax_partial_autocorrelation__lag_2',	'ax_partial_autocorrelation__lag_3',	'ax_partial_autocorrelation__lag_4',	'ax_partial_autocorrelation__lag_5',	'ax_partial_autocorrelation__lag_6',	'ax_partial_autocorrelation__lag_7',	'ax_partial_autocorrelation__lag_8',	'ax_partial_autocorrelation__lag_9',	'ax_number_cwt_peaks__n_1',	'ax_number_cwt_peaks__n_5',	'ax_number_peaks__n_1',	'ax_number_peaks__n_3',	'ax_number_peaks__n_5',	'ax_number_peaks__n_10',	'ax_number_peaks__n_50',	'ax_binned_entropy__max_bins_10',	'ax_index_mass_quantile__q_0.1',	'ax_index_mass_quantile__q_0.2',	'ax_index_mass_quantile__q_0.3',	'ax_index_mass_quantile__q_0.4',	'ax_index_mass_quantile__q_0.6',	'ax_index_mass_quantile__q_0.7',	'ax_index_mass_quantile__q_0.8',	'ax_index_mass_quantile__q_0.9',	'ax_cwt_coefficients__coeff_0__w_2__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_0__w_5__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_0__w_10__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_0__w_20__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_w_2__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_w_5__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_w_10__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_w_20__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_2__w_2__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_2__w_5__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_2__w_10__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_2__w_20__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_3__w_2__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_3__w_5__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_3__w_10__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_3__w_20__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_4__w_2__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_4__w_5__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_4__w_10__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_4__w_20__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_5__w_2__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_5__w_5__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_5__w_10__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_5__w_20__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_6__w_2__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_6__w_5__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_6__w_10__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_6__w_20__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_7__w_2__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_7__w_5__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_7__w_10__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_7__w_20__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_8__w_2__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_8__w_5__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_8__w_10__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_8__w_20__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_9__w_2__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_9__w_5__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_9__w_10__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_9__w_20__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_10__w_2__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_10__w_5__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_10__w_10__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_10__w_20__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_1w_2__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_1w_5__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_1w_10__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_1w_20__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_12__w_2__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_12__w_5__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_12__w_10__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_12__w_20__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_13__w_2__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_13__w_5__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_13__w_10__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_13__w_20__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_14__w_2__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_14__w_5__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_14__w_10__widths_(2 5 10 20)',	'ax_cwt_coefficients__coeff_14__w_20__widths_(2 5 10 20)',	'ax_spkt_welch_density__coeff_2',	'ax_spkt_welch_density__coeff_5',	'ax_spkt_welch_density__coeff_8',	'ax_ar_coefficient__coeff_0__k_10',	'ax_ar_coefficient__coeff_k_10',	'ax_ar_coefficient__coeff_2__k_10',	'ax_ar_coefficient__coeff_3__k_10',	'ax_ar_coefficient__coeff_4__k_10',	'ax_ar_coefficient__coeff_5__k_10',	'ax_ar_coefficient__coeff_6__k_10',	'ax_ar_coefficient__coeff_7__k_10',	'ax_ar_coefficient__coeff_8__k_10',	'ax_ar_coefficient__coeff_9__k_10',	'ax_ar_coefficient__coeff_10__k_10',	'ax_change_quantiles__f_agg_"mean"__isabs_False__qh_0.2__ql_0.0',	'ax_change_quantiles__f_agg_"var"__isabs_False__qh_0.2__ql_0.0',	'ax_change_quantiles__f_agg_"mean"__isabs_True__qh_0.2__ql_0.0',	'ax_change_quantiles__f_agg_"var"__isabs_True__qh_0.2__ql_0.0',	'ax_change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.0',	'ax_change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.0',	'ax_change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.0',	'ax_change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.0',	'ax_change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.0',	'ax_change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.0',	'ax_change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0',	'ax_change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.0',	'ax_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.0',	'ax_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0',	'ax_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0',	'ax_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.0',	'ax_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.0',	'ax_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.0',	'ax_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0',	'ax_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.0',	'ax_change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.2',	'ax_change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.2',	'ax_change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.2',	'ax_change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.2',	'ax_change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.2',	'ax_change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.2',	'ax_change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.2',	'ax_change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.2',	'ax_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.2',	'ax_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.2',	'ax_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2',	'ax_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.2',	'ax_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.2',	'ax_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.2',	'ax_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.2',	'ax_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.2',	'ax_change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.4',	'ax_change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.4',	'ax_change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.4',	'ax_change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.4',	'ax_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.4',	'ax_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.4',	'ax_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.4',	'ax_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.4',	'ax_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.4',	'ax_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.4',	'ax_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.4',	'ax_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.4',	'ax_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.6',	'ax_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.6',	'ax_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.6',	'ax_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.6',	'ax_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.6',	'ax_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.6',	'ax_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.6',	'ax_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.6',	'ax_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.8',	'ax_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.8',	'ax_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.8',	'ax_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.8',	'ax_fft_coefficient__attr_"real"__coeff_0',	'ax_fft_coefficient__attr_"real"__coeff_1',	'ax_fft_coefficient__attr_"real"__coeff_2',	'ax_fft_coefficient__attr_"real"__coeff_3',	'ax_fft_coefficient__attr_"real"__coeff_4',	'ax_fft_coefficient__attr_"real"__coeff_5',	'ax_fft_coefficient__attr_"real"__coeff_6',	'ax_fft_coefficient__attr_"real"__coeff_7',	'ax_fft_coefficient__attr_"real"__coeff_8',	'ax_fft_coefficient__attr_"real"__coeff_9',	'ax_fft_coefficient__attr_"real"__coeff_10',	'ax_fft_coefficient__attr_"real"__coeff_11',	'ax_fft_coefficient__attr_"real"__coeff_12',	'ax_fft_coefficient__attr_"real"__coeff_13',	'ax_fft_coefficient__attr_"real"__coeff_14',	'ax_fft_coefficient__attr_"real"__coeff_15',	'ax_fft_coefficient__attr_"real"__coeff_16',	'ax_fft_coefficient__attr_"real"__coeff_17',	'ax_fft_coefficient__attr_"real"__coeff_18',	'ax_fft_coefficient__attr_"real"__coeff_19',	'ax_fft_coefficient__attr_"real"__coeff_20',	'ax_fft_coefficient__attr_"real"__coeff_21',	'ax_fft_coefficient__attr_"real"__coeff_22',	'ax_fft_coefficient__attr_"real"__coeff_23',	'ax_fft_coefficient__attr_"real"__coeff_24',	'ax_fft_coefficient__attr_"real"__coeff_25',	'ax_fft_coefficient__attr_"real"__coeff_26',	'ax_fft_coefficient__attr_"real"__coeff_27',	'ax_fft_coefficient__attr_"real"__coeff_28',	'ax_fft_coefficient__attr_"real"__coeff_29',	'ax_fft_coefficient__attr_"real"__coeff_30',	'ax_fft_coefficient__attr_"real"__coeff_31',	'ax_fft_coefficient__attr_"real"__coeff_32',	'ax_fft_coefficient__attr_"real"__coeff_33',	'ax_fft_coefficient__attr_"real"__coeff_34',	'ax_fft_coefficient__attr_"real"__coeff_35',	'ax_fft_coefficient__attr_"real"__coeff_36',	'ax_fft_coefficient__attr_"real"__coeff_37',	'ax_fft_coefficient__attr_"real"__coeff_38',	'ax_fft_coefficient__attr_"real"__coeff_39',	'ax_fft_coefficient__attr_"real"__coeff_40',	'ax_fft_coefficient__attr_"real"__coeff_41',	'ax_fft_coefficient__attr_"real"__coeff_42',	'ax_fft_coefficient__attr_"real"__coeff_43',	'ax_fft_coefficient__attr_"real"__coeff_44',	'ax_fft_coefficient__attr_"real"__coeff_45',	'ax_fft_coefficient__attr_"real"__coeff_46',	'ax_fft_coefficient__attr_"real"__coeff_47',	'ax_fft_coefficient__attr_"real"__coeff_48',	'ax_fft_coefficient__attr_"real"__coeff_49',	'ax_fft_coefficient__attr_"real"__coeff_50',	'ax_fft_coefficient__attr_"real"__coeff_51',	'ax_fft_coefficient__attr_"real"__coeff_52',	'ax_fft_coefficient__attr_"real"__coeff_53',	'ax_fft_coefficient__attr_"real"__coeff_54',	'ax_fft_coefficient__attr_"real"__coeff_55',	'ax_fft_coefficient__attr_"real"__coeff_56',	'ax_fft_coefficient__attr_"real"__coeff_57',	'ax_fft_coefficient__attr_"real"__coeff_58',	'ax_fft_coefficient__attr_"real"__coeff_59',	'ax_fft_coefficient__attr_"real"__coeff_60',	'ax_fft_coefficient__attr_"real"__coeff_61',	'ax_fft_coefficient__attr_"real"__coeff_62',	'ax_fft_coefficient__attr_"real"__coeff_63',	'ax_fft_coefficient__attr_"real"__coeff_64',	'ax_fft_coefficient__attr_"real"__coeff_65',	'ax_fft_coefficient__attr_"real"__coeff_66',	'ax_fft_coefficient__attr_"real"__coeff_67',	'ax_fft_coefficient__attr_"real"__coeff_68',	'ax_fft_coefficient__attr_"real"__coeff_69',	'ax_fft_coefficient__attr_"real"__coeff_70',	'ax_fft_coefficient__attr_"real"__coeff_71',	'ax_fft_coefficient__attr_"real"__coeff_72',	'ax_fft_coefficient__attr_"real"__coeff_73',	'ax_fft_coefficient__attr_"real"__coeff_74',	'ax_fft_coefficient__attr_"real"__coeff_75',	'ax_fft_coefficient__attr_"real"__coeff_76',	'ax_fft_coefficient__attr_"real"__coeff_77',	'ax_fft_coefficient__attr_"real"__coeff_78',	'ax_fft_coefficient__attr_"real"__coeff_79',	'ax_fft_coefficient__attr_"real"__coeff_80',	'ax_fft_coefficient__attr_"real"__coeff_81',	'ax_fft_coefficient__attr_"real"__coeff_82',	'ax_fft_coefficient__attr_"real"__coeff_83',	'ax_fft_coefficient__attr_"real"__coeff_84',	'ax_fft_coefficient__attr_"real"__coeff_85',	'ax_fft_coefficient__attr_"real"__coeff_86',	'ax_fft_coefficient__attr_"real"__coeff_87',	'ax_fft_coefficient__attr_"real"__coeff_88',	'ax_fft_coefficient__attr_"real"__coeff_89',	'ax_fft_coefficient__attr_"real"__coeff_90',	'ax_fft_coefficient__attr_"real"__coeff_91',	'ax_fft_coefficient__attr_"real"__coeff_92',	'ax_fft_coefficient__attr_"real"__coeff_93',	'ax_fft_coefficient__attr_"real"__coeff_94',	'ax_fft_coefficient__attr_"real"__coeff_95',	'ax_fft_coefficient__attr_"real"__coeff_96',	'ax_fft_coefficient__attr_"real"__coeff_97',	'ax_fft_coefficient__attr_"real"__coeff_98',	'ax_fft_coefficient__attr_"real"__coeff_99',	'ax_fft_coefficient__attr_"imag"__coeff_0',	'ax_fft_coefficient__attr_"imag"__coeff_1',	'ax_fft_coefficient__attr_"imag"__coeff_2',	'ax_fft_coefficient__attr_"imag"__coeff_3',	'ax_fft_coefficient__attr_"imag"__coeff_4',	'ax_fft_coefficient__attr_"imag"__coeff_5',	'ax_fft_coefficient__attr_"imag"__coeff_6',	'ax_fft_coefficient__attr_"imag"__coeff_7',	'ax_fft_coefficient__attr_"imag"__coeff_8',	'ax_fft_coefficient__attr_"imag"__coeff_9',	'ax_fft_coefficient__attr_"imag"__coeff_10',	'ax_fft_coefficient__attr_"imag"__coeff_11',	'ax_fft_coefficient__attr_"imag"__coeff_12',	'ax_fft_coefficient__attr_"imag"__coeff_13',	'ax_fft_coefficient__attr_"imag"__coeff_14',	'ax_fft_coefficient__attr_"imag"__coeff_15',	'ax_fft_coefficient__attr_"imag"__coeff_16',	'ax_fft_coefficient__attr_"imag"__coeff_17',	'ax_fft_coefficient__attr_"imag"__coeff_18',	'ax_fft_coefficient__attr_"imag"__coeff_19',	'ax_fft_coefficient__attr_"imag"__coeff_20',	'ax_fft_coefficient__attr_"imag"__coeff_21',	'ax_fft_coefficient__attr_"imag"__coeff_22',	'ax_fft_coefficient__attr_"imag"__coeff_23',	'ax_fft_coefficient__attr_"imag"__coeff_24',	'ax_fft_coefficient__attr_"imag"__coeff_25',	'ax_fft_coefficient__attr_"imag"__coeff_26',	'ax_fft_coefficient__attr_"imag"__coeff_27',	'ax_fft_coefficient__attr_"imag"__coeff_28',	'ax_fft_coefficient__attr_"imag"__coeff_29',	'ax_fft_coefficient__attr_"imag"__coeff_30',	'ax_fft_coefficient__attr_"imag"__coeff_31',	'ax_fft_coefficient__attr_"imag"__coeff_32',	'ax_fft_coefficient__attr_"imag"__coeff_33',	'ax_fft_coefficient__attr_"imag"__coeff_34',	'ax_fft_coefficient__attr_"imag"__coeff_35',	'ax_fft_coefficient__attr_"imag"__coeff_36',	'ax_fft_coefficient__attr_"imag"__coeff_37',	'ax_fft_coefficient__attr_"imag"__coeff_38',	'ax_fft_coefficient__attr_"imag"__coeff_39',	'ax_fft_coefficient__attr_"imag"__coeff_40',	'ax_fft_coefficient__attr_"imag"__coeff_41',	'ax_fft_coefficient__attr_"imag"__coeff_42',	'ax_fft_coefficient__attr_"imag"__coeff_43',	'ax_fft_coefficient__attr_"imag"__coeff_44',	'ax_fft_coefficient__attr_"imag"__coeff_45',	'ax_fft_coefficient__attr_"imag"__coeff_46',	'ax_fft_coefficient__attr_"imag"__coeff_47',	'ax_fft_coefficient__attr_"imag"__coeff_48',	'ax_fft_coefficient__attr_"imag"__coeff_49',	'ax_fft_coefficient__attr_"imag"__coeff_50',	'ax_fft_coefficient__attr_"imag"__coeff_51',	'ax_fft_coefficient__attr_"imag"__coeff_52',	'ax_fft_coefficient__attr_"imag"__coeff_53',	'ax_fft_coefficient__attr_"imag"__coeff_54',	'ax_fft_coefficient__attr_"imag"__coeff_55',	'ax_fft_coefficient__attr_"imag"__coeff_56',	'ax_fft_coefficient__attr_"imag"__coeff_57',	'ax_fft_coefficient__attr_"imag"__coeff_58',	'ax_fft_coefficient__attr_"imag"__coeff_59',	'ax_fft_coefficient__attr_"imag"__coeff_60',	'ax_fft_coefficient__attr_"imag"__coeff_61',	'ax_fft_coefficient__attr_"imag"__coeff_62',	'ax_fft_coefficient__attr_"imag"__coeff_63',	'ax_fft_coefficient__attr_"imag"__coeff_64',	'ax_fft_coefficient__attr_"imag"__coeff_65',	'ax_fft_coefficient__attr_"imag"__coeff_66',	'ax_fft_coefficient__attr_"imag"__coeff_67',	'ax_fft_coefficient__attr_"imag"__coeff_68',	'ax_fft_coefficient__attr_"imag"__coeff_69',	'ax_fft_coefficient__attr_"imag"__coeff_70',	'ax_fft_coefficient__attr_"imag"__coeff_71',	'ax_fft_coefficient__attr_"imag"__coeff_72',	'ax_fft_coefficient__attr_"imag"__coeff_73',	'ax_fft_coefficient__attr_"imag"__coeff_74',	'ax_fft_coefficient__attr_"imag"__coeff_75',	'ax_fft_coefficient__attr_"imag"__coeff_76',	'ax_fft_coefficient__attr_"imag"__coeff_77',	'ax_fft_coefficient__attr_"imag"__coeff_78',	'ax_fft_coefficient__attr_"imag"__coeff_79',	'ax_fft_coefficient__attr_"imag"__coeff_80',	'ax_fft_coefficient__attr_"imag"__coeff_81',	'ax_fft_coefficient__attr_"imag"__coeff_82',	'ax_fft_coefficient__attr_"imag"__coeff_83',	'ax_fft_coefficient__attr_"imag"__coeff_84',	'ax_fft_coefficient__attr_"imag"__coeff_85',	'ax_fft_coefficient__attr_"imag"__coeff_86',	'ax_fft_coefficient__attr_"imag"__coeff_87',	'ax_fft_coefficient__attr_"imag"__coeff_88',	'ax_fft_coefficient__attr_"imag"__coeff_89',	'ax_fft_coefficient__attr_"imag"__coeff_90',	'ax_fft_coefficient__attr_"imag"__coeff_91',	'ax_fft_coefficient__attr_"imag"__coeff_92',	'ax_fft_coefficient__attr_"imag"__coeff_93',	'ax_fft_coefficient__attr_"imag"__coeff_94',	'ax_fft_coefficient__attr_"imag"__coeff_95',	'ax_fft_coefficient__attr_"imag"__coeff_96',	'ax_fft_coefficient__attr_"imag"__coeff_97',	'ax_fft_coefficient__attr_"imag"__coeff_98',	'ax_fft_coefficient__attr_"imag"__coeff_99',	'ax_fft_coefficient__attr_"abs"__coeff_0',	'ax_fft_coefficient__attr_"abs"__coeff_1',	'ax_fft_coefficient__attr_"abs"__coeff_2',	'ax_fft_coefficient__attr_"abs"__coeff_3',	'ax_fft_coefficient__attr_"abs"__coeff_4',	'ax_fft_coefficient__attr_"abs"__coeff_5',	'ax_fft_coefficient__attr_"abs"__coeff_6',	'ax_fft_coefficient__attr_"abs"__coeff_7',	'ax_fft_coefficient__attr_"abs"__coeff_8',	'ax_fft_coefficient__attr_"abs"__coeff_9',	'ax_fft_coefficient__attr_"abs"__coeff_10',	'ax_fft_coefficient__attr_"abs"__coeff_11',	'ax_fft_coefficient__attr_"abs"__coeff_12',	'ax_fft_coefficient__attr_"abs"__coeff_13',	'ax_fft_coefficient__attr_"abs"__coeff_14',	'ax_fft_coefficient__attr_"abs"__coeff_15',	'ax_fft_coefficient__attr_"abs"__coeff_16',	'ax_fft_coefficient__attr_"abs"__coeff_17',	'ax_fft_coefficient__attr_"abs"__coeff_18',	'ax_fft_coefficient__attr_"abs"__coeff_19',	'ax_fft_coefficient__attr_"abs"__coeff_20',	'ax_fft_coefficient__attr_"abs"__coeff_21',	'ax_fft_coefficient__attr_"abs"__coeff_22',	'ax_fft_coefficient__attr_"abs"__coeff_23',	'ax_fft_coefficient__attr_"abs"__coeff_24',	'ax_fft_coefficient__attr_"abs"__coeff_25',	'ax_fft_coefficient__attr_"abs"__coeff_26',	'ax_fft_coefficient__attr_"abs"__coeff_27',	'ax_fft_coefficient__attr_"abs"__coeff_28',	'ax_fft_coefficient__attr_"abs"__coeff_29',	'ax_fft_coefficient__attr_"abs"__coeff_30',	'ax_fft_coefficient__attr_"abs"__coeff_31',	'ax_fft_coefficient__attr_"abs"__coeff_32',	'ax_fft_coefficient__attr_"abs"__coeff_33',	'ax_fft_coefficient__attr_"abs"__coeff_34',	'ax_fft_coefficient__attr_"abs"__coeff_35',	'ax_fft_coefficient__attr_"abs"__coeff_36',	'ax_fft_coefficient__attr_"abs"__coeff_37',	'ax_fft_coefficient__attr_"abs"__coeff_38',	'ax_fft_coefficient__attr_"abs"__coeff_39',	'ax_fft_coefficient__attr_"abs"__coeff_40',	'ax_fft_coefficient__attr_"abs"__coeff_41',	'ax_fft_coefficient__attr_"abs"__coeff_42',	'ax_fft_coefficient__attr_"abs"__coeff_43',	'ax_fft_coefficient__attr_"abs"__coeff_44',	'ax_fft_coefficient__attr_"abs"__coeff_45',	'ax_fft_coefficient__attr_"abs"__coeff_46',	'ax_fft_coefficient__attr_"abs"__coeff_47',	'ax_fft_coefficient__attr_"abs"__coeff_48',	'ax_fft_coefficient__attr_"abs"__coeff_49',	'ax_fft_coefficient__attr_"abs"__coeff_50',	'ax_fft_coefficient__attr_"abs"__coeff_51',	'ax_fft_coefficient__attr_"abs"__coeff_52',	'ax_fft_coefficient__attr_"abs"__coeff_53',	'ax_fft_coefficient__attr_"abs"__coeff_54',	'ax_fft_coefficient__attr_"abs"__coeff_55',	'ax_fft_coefficient__attr_"abs"__coeff_56',	'ax_fft_coefficient__attr_"abs"__coeff_57',	'ax_fft_coefficient__attr_"abs"__coeff_58',	'ax_fft_coefficient__attr_"abs"__coeff_59',	'ax_fft_coefficient__attr_"abs"__coeff_60',	'ax_fft_coefficient__attr_"abs"__coeff_61',	'ax_fft_coefficient__attr_"abs"__coeff_62',	'ax_fft_coefficient__attr_"abs"__coeff_63',	'ax_fft_coefficient__attr_"abs"__coeff_64',	'ax_fft_coefficient__attr_"abs"__coeff_65',	'ax_fft_coefficient__attr_"abs"__coeff_66',	'ax_fft_coefficient__attr_"abs"__coeff_67',	'ax_fft_coefficient__attr_"abs"__coeff_68',	'ax_fft_coefficient__attr_"abs"__coeff_69',	'ax_fft_coefficient__attr_"abs"__coeff_70',	'ax_fft_coefficient__attr_"abs"__coeff_71',	'ax_fft_coefficient__attr_"abs"__coeff_72',	'ax_fft_coefficient__attr_"abs"__coeff_73',	'ax_fft_coefficient__attr_"abs"__coeff_74',	'ax_fft_coefficient__attr_"abs"__coeff_75',	'ax_fft_coefficient__attr_"abs"__coeff_76',	'ax_fft_coefficient__attr_"abs"__coeff_77',	'ax_fft_coefficient__attr_"abs"__coeff_78',	'ax_fft_coefficient__attr_"abs"__coeff_79',	'ax_fft_coefficient__attr_"abs"__coeff_80',	'ax_fft_coefficient__attr_"abs"__coeff_81',	'ax_fft_coefficient__attr_"abs"__coeff_82',	'ax_fft_coefficient__attr_"abs"__coeff_83',	'ax_fft_coefficient__attr_"abs"__coeff_84',	'ax_fft_coefficient__attr_"abs"__coeff_85',	'ax_fft_coefficient__attr_"abs"__coeff_86',	'ax_fft_coefficient__attr_"abs"__coeff_87',	'ax_fft_coefficient__attr_"abs"__coeff_88',	'ax_fft_coefficient__attr_"abs"__coeff_89',	'ax_fft_coefficient__attr_"abs"__coeff_90',	'ax_fft_coefficient__attr_"abs"__coeff_91',	'ax_fft_coefficient__attr_"abs"__coeff_92',	'ax_fft_coefficient__attr_"abs"__coeff_93',	'ax_fft_coefficient__attr_"abs"__coeff_94',	'ax_fft_coefficient__attr_"abs"__coeff_95',	'ax_fft_coefficient__attr_"abs"__coeff_96',	'ax_fft_coefficient__attr_"abs"__coeff_97',	'ax_fft_coefficient__attr_"abs"__coeff_98',	'ax_fft_coefficient__attr_"abs"__coeff_99',	'ax_fft_coefficient__attr_"angle"__coeff_0',	'ax_fft_coefficient__attr_"angle"__coeff_1',	'ax_fft_coefficient__attr_"angle"__coeff_2',	'ax_fft_coefficient__attr_"angle"__coeff_3',	'ax_fft_coefficient__attr_"angle"__coeff_4',	'ax_fft_coefficient__attr_"angle"__coeff_5',	'ax_fft_coefficient__attr_"angle"__coeff_6',	'ax_fft_coefficient__attr_"angle"__coeff_7',	'ax_fft_coefficient__attr_"angle"__coeff_8',	'ax_fft_coefficient__attr_"angle"__coeff_9',	'ax_fft_coefficient__attr_"angle"__coeff_10',	'ax_fft_coefficient__attr_"angle"__coeff_11',	'ax_fft_coefficient__attr_"angle"__coeff_12',	'ax_fft_coefficient__attr_"angle"__coeff_13',	'ax_fft_coefficient__attr_"angle"__coeff_14',	'ax_fft_coefficient__attr_"angle"__coeff_15',	'ax_fft_coefficient__attr_"angle"__coeff_16',	'ax_fft_coefficient__attr_"angle"__coeff_17',	'ax_fft_coefficient__attr_"angle"__coeff_18',	'ax_fft_coefficient__attr_"angle"__coeff_19',	'ax_fft_coefficient__attr_"angle"__coeff_20',	'ax_fft_coefficient__attr_"angle"__coeff_21',	'ax_fft_coefficient__attr_"angle"__coeff_22',	'ax_fft_coefficient__attr_"angle"__coeff_23',	'ax_fft_coefficient__attr_"angle"__coeff_24',	'ax_fft_coefficient__attr_"angle"__coeff_25',	'ax_fft_coefficient__attr_"angle"__coeff_26',	'ax_fft_coefficient__attr_"angle"__coeff_27',	'ax_fft_coefficient__attr_"angle"__coeff_28',	'ax_fft_coefficient__attr_"angle"__coeff_29',	'ax_fft_coefficient__attr_"angle"__coeff_30',	'ax_fft_coefficient__attr_"angle"__coeff_31',	'ax_fft_coefficient__attr_"angle"__coeff_32',	'ax_fft_coefficient__attr_"angle"__coeff_33',	'ax_fft_coefficient__attr_"angle"__coeff_34',	'ax_fft_coefficient__attr_"angle"__coeff_35',	'ax_fft_coefficient__attr_"angle"__coeff_36',	'ax_fft_coefficient__attr_"angle"__coeff_37',	'ax_fft_coefficient__attr_"angle"__coeff_38',	'ax_fft_coefficient__attr_"angle"__coeff_39',	'ax_fft_coefficient__attr_"angle"__coeff_40',	'ax_fft_coefficient__attr_"angle"__coeff_41',	'ax_fft_coefficient__attr_"angle"__coeff_42',	'ax_fft_coefficient__attr_"angle"__coeff_43',	'ax_fft_coefficient__attr_"angle"__coeff_44',	'ax_fft_coefficient__attr_"angle"__coeff_45',	'ax_fft_coefficient__attr_"angle"__coeff_46',	'ax_fft_coefficient__attr_"angle"__coeff_47',	'ax_fft_coefficient__attr_"angle"__coeff_48',	'ax_fft_coefficient__attr_"angle"__coeff_49',	'ax_fft_coefficient__attr_"angle"__coeff_50',	'ax_fft_coefficient__attr_"angle"__coeff_51',	'ax_fft_coefficient__attr_"angle"__coeff_52',	'ax_fft_coefficient__attr_"angle"__coeff_53',	'ax_fft_coefficient__attr_"angle"__coeff_54',	'ax_fft_coefficient__attr_"angle"__coeff_55',	'ax_fft_coefficient__attr_"angle"__coeff_56',	'ax_fft_coefficient__attr_"angle"__coeff_57',	'ax_fft_coefficient__attr_"angle"__coeff_58',	'ax_fft_coefficient__attr_"angle"__coeff_59',	'ax_fft_coefficient__attr_"angle"__coeff_60',	'ax_fft_coefficient__attr_"angle"__coeff_61',	'ax_fft_coefficient__attr_"angle"__coeff_62',	'ax_fft_coefficient__attr_"angle"__coeff_63',	'ax_fft_coefficient__attr_"angle"__coeff_64',	'ax_fft_coefficient__attr_"angle"__coeff_65',	'ax_fft_coefficient__attr_"angle"__coeff_66',	'ax_fft_coefficient__attr_"angle"__coeff_67',	'ax_fft_coefficient__attr_"angle"__coeff_68',	'ax_fft_coefficient__attr_"angle"__coeff_69',	'ax_fft_coefficient__attr_"angle"__coeff_70',	'ax_fft_coefficient__attr_"angle"__coeff_71',	'ax_fft_coefficient__attr_"angle"__coeff_72',	'ax_fft_coefficient__attr_"angle"__coeff_73',	'ax_fft_coefficient__attr_"angle"__coeff_74',	'ax_fft_coefficient__attr_"angle"__coeff_75',	'ax_fft_coefficient__attr_"angle"__coeff_76',	'ax_fft_coefficient__attr_"angle"__coeff_77',	'ax_fft_coefficient__attr_"angle"__coeff_78',	'ax_fft_coefficient__attr_"angle"__coeff_79',	'ax_fft_coefficient__attr_"angle"__coeff_80',	'ax_fft_coefficient__attr_"angle"__coeff_81',	'ax_fft_coefficient__attr_"angle"__coeff_82',	'ax_fft_coefficient__attr_"angle"__coeff_83',	'ax_fft_coefficient__attr_"angle"__coeff_84',	'ax_fft_coefficient__attr_"angle"__coeff_85',	'ax_fft_coefficient__attr_"angle"__coeff_86',	'ax_fft_coefficient__attr_"angle"__coeff_87',	'ax_fft_coefficient__attr_"angle"__coeff_88',	'ax_fft_coefficient__attr_"angle"__coeff_89',	'ax_fft_coefficient__attr_"angle"__coeff_90',	'ax_fft_coefficient__attr_"angle"__coeff_91',	'ax_fft_coefficient__attr_"angle"__coeff_92',	'ax_fft_coefficient__attr_"angle"__coeff_93',	'ax_fft_coefficient__attr_"angle"__coeff_94',	'ax_fft_coefficient__attr_"angle"__coeff_95',	'ax_fft_coefficient__attr_"angle"__coeff_96',	'ax_fft_coefficient__attr_"angle"__coeff_97',	'ax_fft_coefficient__attr_"angle"__coeff_98',	'ax_fft_coefficient__attr_"angle"__coeff_99',	'ax_fft_aggregated__aggtype_"centroid"',	'ax_fft_aggregated__aggtype_"variance"',	'ax_fft_aggregated__aggtype_"skew"',	'ax_fft_aggregated__aggtype_"kurtosis"',	'ax_value_count__value_0',	'ax_value_count__value_1',	'ax_value_count__value_-1',	'ax_range_count__max_min_-1',	'ax_range_count__max_0__min_1000000000000.0',	'ax_range_count__max_1000000000000.0__min_0',	'ax_approximate_entropy__m_2__r_0.1',	'ax_approximate_entropy__m_2__r_0.3',	'ax_approximate_entropy__m_2__r_0.5',	'ax_approximate_entropy__m_2__r_0.7',	'ax_approximate_entropy__m_2__r_0.9',	'ax_friedrich_coefficients__coeff_0__m_3__r_30',	'ax_friedrich_coefficients__coeff_m_3__r_30',	'ax_friedrich_coefficients__coeff_2__m_3__r_30',	'ax_friedrich_coefficients__coeff_3__m_3__r_30',	'ax_max_langevin_fixed_point__m_3__r_30',	'ax_linear_trend__attr_"pvalue"',	'ax_linear_trend__attr_"rvalue"',	'ax_linear_trend__attr_"intercept"',	'ax_linear_trend__attr_"slope"',	'ax_linear_trend__attr_"stderr"',	'ax_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"max"',	'ax_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"min"',	'ax_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"mean"',	'ax_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"var"',	'ax_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"max"',	'ax_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"min"',	'ax_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"mean"',	'ax_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"var"',	'ax_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"max"',	'ax_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"min"',	'ax_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"mean"',	'ax_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"var"',	'ax_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"max"',	'ax_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"min"',	'ax_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"mean"',	'ax_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"var"',	'ax_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"max"',	'ax_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"min"',	'ax_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"mean"',	'ax_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"var"',	'ax_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"max"',	'ax_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"min"',	'ax_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"mean"',	'ax_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"var"',	'ax_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"max"',	'ax_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"min"',	'ax_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"mean"',	'ax_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"var"',	'ax_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"max"',	'ax_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"min"',	'ax_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"mean"',	'ax_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"var"',	'ax_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"max"',	'ax_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"min"',	'ax_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"mean"',	'ax_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"var"',	'ax_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"max"',	'ax_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"min"',	'ax_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"mean"',	'ax_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"var"',	'ax_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"max"',	'ax_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"min"',	'ax_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"mean"',	'ax_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"var"',	'ax_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"max"',	'ax_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"min"',	'ax_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"mean"',	'ax_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"var"',	'ax_augmented_dickey_fuller__attr_"teststat"__autolag_"AIC"',	'ax_augmented_dickey_fuller__attr_"pvalue"__autolag_"AIC"',	'ax_augmented_dickey_fuller__attr_"usedlag"__autolag_"AIC"',	'ax_number_crossing_m__m_0',	'ax_number_crossing_m__m_-1',	'ax_number_crossing_m__m_1',	'ax_enerax_ratio_by_chunks__num_segments_10__segment_focus_0',	'ax_enerax_ratio_by_chunks__num_segments_10__segment_focus_1',	'ax_enerax_ratio_by_chunks__num_segments_10__segment_focus_2',	'ax_enerax_ratio_by_chunks__num_segments_10__segment_focus_3',	'ax_enerax_ratio_by_chunks__num_segments_10__segment_focus_4',	'ax_enerax_ratio_by_chunks__num_segments_10__segment_focus_5',	'ax_enerax_ratio_by_chunks__num_segments_10__segment_focus_6',	'ax_enerax_ratio_by_chunks__num_segments_10__segment_focus_7',	'ax_enerax_ratio_by_chunks__num_segments_10__segment_focus_8',	'ax_enerax_ratio_by_chunks__num_segments_10__segment_focus_9',	'ax_ratio_beyond_r_sigma__r_0.5',	'ax_ratio_beyond_r_sigma__r_1',	'ax_ratio_beyond_r_sigma__r_1.5',	'ax_ratio_beyond_r_sigma__r_2',	'ax_ratio_beyond_r_sigma__r_2.5',	'ax_ratio_beyond_r_sigma__r_3',	'ax_ratio_beyond_r_sigma__r_5',	'ax_ratio_beyond_r_sigma__r_6',	'ax_ratio_beyond_r_sigma__r_7',	'ax_ratio_beyond_r_sigma__r_10',	'ax_count_above__t_0',	'ax_count_below__t_0',	'ax_lempel_ziv_complexity__bins_2',	'ax_lempel_ziv_complexity__bins_3',	'ax_lempel_ziv_complexity__bins_5',	'ax_lempel_ziv_complexity__bins_10',	'ax_lempel_ziv_complexity__bins_100',	'ax_fourier_entropy__bins_2',	'ax_fourier_entropy__bins_3',	'ax_fourier_entropy__bins_5',	'ax_fourier_entropy__bins_10',	'ax_fourier_entropy__bins_100',	'ax_permutation_entropy__dimension_3__tau_1',	'ax_permutation_entropy__dimension_4__tau_1',	'ax_permutation_entropy__dimension_5__tau_1',	'ax_permutation_entropy__dimension_6__tau_1',	'ax_permutation_entropy__dimension_7__tau_1',	'ax_query_similarity_count__query_None__threshold_0.0',	'ax_matrix_profile__feature_"min"__threshold_0.98',	'ax_matrix_profile__feature_"max"__threshold_0.98',	'ax_matrix_profile__feature_"mean"__threshold_0.98',	'ax_matrix_profile__feature_"median"__threshold_0.98',	'ax_matrix_profile__feature_"25"__threshold_0.98',	'ax_matrix_profile__feature_"75"__threshold_0.98']
fea_user_1ax=pd.DataFrame(fea_user_1ax.reshape(Gx_list_trans.shape[1], 787))
fea_user_1ax.columns=[header_ax]
fea_user_1ax

,ax_variance_larger_than_standard_deviation,ax_has_duplicate_max,ax_has_duplicate_min,ax_has_duplicate,ax_sum_values,ax_abs_enerax,ax_mean_abs_change,ax_mean_change,ax_mean_second_derivative_central,ax_median,...,ax_permutation_entropy__dimension_5__tau_1,ax_permutation_entropy__dimension_6__tau_1,ax_permutation_entropy__dimension_7__tau_1,ax_query_similarity_count__query_None__threshold_0.0,"ax_matrix_profile__feature_""min""__threshold_0.98","ax_matrix_profile__feature_""max""__threshold_0.98","ax_matrix_profile__feature_""mean""__threshold_0.98","ax_matrix_profile__feature_""median""__threshold_0.98","ax_matrix_profile__feature_""25""__threshold_0.98","ax_matrix_profile__feature_""75""__threshold_0.98"
0,0.0,0.0,0.0,0.0,-0.0729,0.047588,0.051421,-0.005200,-0.005064,-0.01695,...,2.772589,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,-0.0566,0.033008,0.035237,-0.001384,-0.002483,-0.01000,...,2.426015,2.488328,2.540036,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,0.0,0.1132,0.018891,0.024647,0.000384,0.000114,0.01060,...,2.269639,2.338372,2.403640,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,0.0,0.0,-0.0384,0.022192,0.032458,0.002153,-0.001675,0.00740,...,2.772589,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,0.0,0.0,0.0827,0.022454,0.033563,-0.002847,-0.001383,0.00840,...,2.772589,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,0.0,0.0,0.0,0.0,0.1684,0.131082,0.088658,-0.000142,-0.000978,0.00735,...,2.512659,2.615631,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
805,0.0,0.0,0.0,0.0,0.4226,0.038159,0.034632,-0.005211,0.001278,0.01270,...,2.685945,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
806,0.0,0.0,0.0,0.0,0.0458,0.027113,0.029458,-0.002826,0.000864,0.00475,...,2.772589,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
807,0.0,0.0,0.0,0.0,0.0631,0.057955,0.034847,0.000721,0.001311,-0.00860,...,2.599302,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
r_ax=fea_user_1ax.dropna(axis='columns',how='all')
r_ax=r_ax.fillna(0)
binary_features_ax=['ax_variance_larger_than_standard_deviation','ax_has_duplicate_max',	'ax_has_duplicate_min',	'ax_has_duplicate','ax_symmetry_looking__r_0.0',	'ax_symmetry_looking__r_0.05',	'ax_symmetry_looking__r_0.1',	'ax_symmetry_looking__r_0.15000000000000002',	'ax_symmetry_looking__r_0.2',	'ax_symmetry_looking__r_0.25',	'ax_symmetry_looking__r_0.30000000000000004',	'ax_symmetry_looking__r_0.35000000000000003',	'ax_symmetry_looking__r_0.4',	'ax_symmetry_looking__r_0.45',	'ax_symmetry_looking__r_0.5',	'ax_symmetry_looking__r_0.55',	'ax_symmetry_looking__r_0.6000000000000001',	'ax_symmetry_looking__r_0.65',	'ax_symmetry_looking__r_0.7000000000000001',	'ax_symmetry_looking__r_0.75',	'ax_symmetry_looking__r_0.8',	'ax_symmetry_looking__r_0.8500000000000001',	'ax_symmetry_looking__r_0.9',	'ax_symmetry_looking__r_0.9500000000000001',	'ax_large_standard_deviation__r_0.05',	'ax_large_standard_deviation__r_0.1',	'ax_large_standard_deviation__r_0.15000000000000002',	'ax_large_standard_deviation__r_0.2',	'ax_large_standard_deviation__r_0.25',	'ax_large_standard_deviation__r_0.30000000000000004',	'ax_large_standard_deviation__r_0.35000000000000003',	'ax_large_standard_deviation__r_0.4',	'ax_large_standard_deviation__r_0.45',	'ax_large_standard_deviation__r_0.5',	'ax_large_standard_deviation__r_0.55',	'ax_large_standard_deviation__r_0.6000000000000001',	'ax_large_standard_deviation__r_0.65',	'ax_large_standard_deviation__r_0.7000000000000001',	'ax_large_standard_deviation__r_0.75',	'ax_large_standard_deviation__r_0.8',	'ax_large_standard_deviation__r_0.8500000000000001',	'ax_large_standard_deviation__r_0.9',	'ax_large_standard_deviation__r_0.9500000000000001','ax_value_count__value_0',	'ax_value_count__value_1',	'ax_value_count__value_-1']
r_ax=r_ax.drop(binary_features_ax, axis=1)
r_ax

d:\m-tech-data-science\tools\python 3.7\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,ax_sum_values,ax_abs_enerax,ax_mean_abs_change,ax_mean_change,ax_mean_second_derivative_central,ax_median,ax_mean,ax_length,ax_standard_deviation,ax_variation_coefficient,...,ax_permutation_entropy__dimension_4__tau_1,ax_permutation_entropy__dimension_5__tau_1,ax_permutation_entropy__dimension_6__tau_1,ax_permutation_entropy__dimension_7__tau_1,"ax_matrix_profile__feature_""min""__threshold_0.98","ax_matrix_profile__feature_""max""__threshold_0.98","ax_matrix_profile__feature_""mean""__threshold_0.98","ax_matrix_profile__feature_""median""__threshold_0.98","ax_matrix_profile__feature_""25""__threshold_0.98","ax_matrix_profile__feature_""75""__threshold_0.98"
0,-0.0729,0.047588,0.051421,-0.005200,-0.005064,-0.01695,-0.003645,20.0,0.048643,-13.345060,...,2.476247,2.772589,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.0566,0.033008,0.035237,-0.001384,-0.002483,-0.01000,-0.002830,20.0,0.040527,-14.320322,...,1.956187,2.426015,2.488328,2.540036,0.0,0.0,0.0,0.0,0.0,0.0
2,0.1132,0.018891,0.024647,0.000384,0.000114,0.01060,0.005660,20.0,0.030208,5.337088,...,2.037734,2.269639,2.338372,2.403640,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.0384,0.022192,0.032458,0.002153,-0.001675,0.00740,-0.001920,20.0,0.033256,-17.320591,...,2.425480,2.772589,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0827,0.022454,0.033563,-0.002847,-0.001383,0.00840,0.004135,20.0,0.033251,8.041304,...,2.670120,2.772589,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,0.1684,0.131082,0.088658,-0.000142,-0.000978,0.00735,0.008420,20.0,0.080518,9.562762,...,2.394700,2.512659,2.615631,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
805,0.4226,0.038159,0.034632,-0.005211,0.001278,0.01270,0.021130,20.0,0.038229,1.809226,...,2.588573,2.685945,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
806,0.0458,0.027113,0.029458,-0.002826,0.000864,0.00475,0.002290,20.0,0.036748,16.046974,...,2.507026,2.772589,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
807,0.0631,0.057955,0.034847,0.000721,0.001311,-0.00860,0.003155,20.0,0.053738,17.032673,...,2.282374,2.599302,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0


### Y-axis accelerometer 

In [24]:
Ay_list=pd.DataFrame(Ay_list)
Ay_list=Ay_list.astype(float)
Ay_list_trans=Ay_list.T
Ay_list_trans

,0,1,2,3,4,5,6,7,8,9,...,799,800,801,802,803,804,805,806,807,808
0,0.0626,-0.0061,0.0157,0.0152,-0.0277,-0.0230,0.0277,0.0318,-0.0502,-0.0322,...,-0.0690,0.0506,-0.0529,0.0445,-0.0091,0.0354,-0.1443,-0.0205,0.0255,0.0577
1,-0.0396,-0.0183,0.0078,0.0226,-0.0604,0.0002,0.0040,-0.0162,0.0321,0.0021,...,0.0220,0.0306,-0.0394,0.0234,-0.0195,-0.0096,-0.0572,-0.0011,-0.0168,0.0061
2,0.0530,-0.0347,-0.0102,-0.0150,-0.0074,-0.0164,0.0270,0.0325,0.0191,0.0094,...,0.1738,0.0204,-0.0307,-0.0195,0.0133,-0.0195,0.0485,0.0063,-0.0131,-0.0134
3,-0.0259,-0.0375,-0.0666,-0.0089,0.0085,-0.0076,-0.0619,-0.0018,0.0087,-0.0268,...,0.1575,-0.0175,0.0402,0.0005,0.0218,-0.0279,0.0953,0.0193,-0.0167,0.0047
4,-0.0379,-0.0413,-0.0660,-0.0492,0.0007,-0.0049,-0.0562,-0.0141,-0.0268,-0.0348,...,0.0941,-0.0078,0.0762,-0.0099,-0.0106,-0.0262,-0.0435,0.0089,-0.0405,-0.0160
5,-0.0156,-0.0643,-0.0003,-0.0196,0.0166,-0.0331,-0.1026,-0.0344,-0.0243,-0.0460,...,-0.0736,0.0154,0.0397,-0.0704,-0.0277,-0.2809,-0.0560,-0.0612,0.0187,-0.0327
6,-0.0361,-0.0446,0.0041,-0.0259,0.0375,-0.0549,-0.0911,-0.0339,-0.0270,-0.0056,...,-0.1119,0.0039,0.0282,-0.0777,-0.0304,0.2014,-0.0388,-0.0234,0.0936,-0.0001
7,-0.0677,-0.0105,0.0042,-0.0292,0.0493,-0.0146,-0.0603,-0.0437,-0.0500,0.0241,...,-0.1035,0.0128,0.0343,-0.0718,-0.0274,0.0990,-0.0212,-0.0080,0.0250,0.0265
8,-0.0700,0.0183,0.0170,-0.0442,0.0263,-0.0138,0.0672,-0.0308,-0.0632,-0.0223,...,-0.0470,-0.0208,0.0348,-0.0411,-0.0170,-0.1272,0.0356,0.0074,0.0174,0.0300
9,-0.0940,-0.0232,0.0506,-0.0081,0.0006,-0.0325,0.1672,0.0006,-0.0221,0.0039,...,0.0449,-0.0516,0.0959,0.0034,-0.0232,-0.2516,-0.0017,0.0230,-0.0148,0.0186


In [25]:
from tsfresh import extract_features
#c=c.fillna(0)
l_ay=[]
z_ay=[]
u_ay=[]
X_ay=[]
for i in range (Gx_list_trans.shape[1]):
    p_ay=Ay_list_trans.iloc[:,i]
    p_ay=pd.DataFrame(p_ay)
    p_ay['ID'] = '1'
    X_ay= extract_features(p_ay, column_id='ID')
    #print(X.shape)
    l_ay.append(X_ay)
    #m.append(l)
    z_ay=l_ay
u_ay=z_ay  

Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.76s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.82s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.47s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.34s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.18s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.13s/it]


In [26]:
fea_user_1ay=np.array(u_ay)
header_ay=['ay_variance_larger_than_standard_deviation',	'ay_has_duplicate_max',	'ay_has_duplicate_min',	'ay_has_duplicate',	'ay_sum_values',	'ay_abs_eneray',	'ay_mean_abs_change',	'ay_mean_change',	'ay_mean_second_derivative_central',	'ay_median',	'ay_mean',	'ay_length',	'ay_standard_deviation',	'ay_variation_coefficient',	'ay_variance',	'ay_skewness',	'ay_kurtosis',	'ay_root_mean_square',	'ay_absolute_sum_of_changes',	'ay_longest_strike_below_mean',	'ay_longest_strike_above_mean',	'ay_count_above_mean',	'ay_count_below_mean',	'ay_last_location_of_mayimum',	'ay_first_location_of_mayimum',	'ay_last_location_of_minimum',	'ay_first_location_of_minimum',	'ay_percentage_of_reoccurring_values_to_all_values',	'ay_percentage_of_reoccurring_datapoints_to_all_datapoints',	'ay_sum_of_reoccurring_values',	'ay_sum_of_reoccurring_data_points',	'ay_ratio_value_number_to_time_series_length',	'ay_sample_entropy',	'ay_mayimum',	'ay_minimum',	'ay_benford_correlation',	'ay_time_reversal_asymmetry_statistic__lag_1',	'ay_time_reversal_asymmetry_statistic__lag_2',	'ay_time_reversal_asymmetry_statistic__lag_3',	'ay_c3__lag_1',	'ay_c3__lag_2',	'ay_c3__lag_3',	'ay_cid_ce__normalize_True',	'ay_cid_ce__normalize_False',	'ay_symmetry_looking__r_0.0',	'ay_symmetry_looking__r_0.05',	'ay_symmetry_looking__r_0.1',	'ay_symmetry_looking__r_0.15000000000000002',	'ay_symmetry_looking__r_0.2',	'ay_symmetry_looking__r_0.25',	'ay_symmetry_looking__r_0.30000000000000004',	'ay_symmetry_looking__r_0.35000000000000003',	'ay_symmetry_looking__r_0.4',	'ay_symmetry_looking__r_0.45',	'ay_symmetry_looking__r_0.5',	'ay_symmetry_looking__r_0.55',	'ay_symmetry_looking__r_0.6000000000000001',	'ay_symmetry_looking__r_0.65',	'ay_symmetry_looking__r_0.7000000000000001',	'ay_symmetry_looking__r_0.75',	'ay_symmetry_looking__r_0.8',	'ay_symmetry_looking__r_0.8500000000000001',	'ay_symmetry_looking__r_0.9',	'ay_symmetry_looking__r_0.9500000000000001',	'ay_large_standard_deviation__r_0.05',	'ay_large_standard_deviation__r_0.1',	'ay_large_standard_deviation__r_0.15000000000000002',	'ay_large_standard_deviation__r_0.2',	'ay_large_standard_deviation__r_0.25',	'ay_large_standard_deviation__r_0.30000000000000004',	'ay_large_standard_deviation__r_0.35000000000000003',	'ay_large_standard_deviation__r_0.4',	'ay_large_standard_deviation__r_0.45',	'ay_large_standard_deviation__r_0.5',	'ay_large_standard_deviation__r_0.55',	'ay_large_standard_deviation__r_0.6000000000000001',	'ay_large_standard_deviation__r_0.65',	'ay_large_standard_deviation__r_0.7000000000000001',	'ay_large_standard_deviation__r_0.75',	'ay_large_standard_deviation__r_0.8',	'ay_large_standard_deviation__r_0.8500000000000001',	'ay_large_standard_deviation__r_0.9',	'ay_large_standard_deviation__r_0.9500000000000001',	'ay_quantile__q_0.1',	'ay_quantile__q_0.2',	'ay_quantile__q_0.3',	'ay_quantile__q_0.4',	'ay_quantile__q_0.6',	'ay_quantile__q_0.7',	'ay_quantile__q_0.8',	'ay_quantile__q_0.9',	'ay_autocorrelation__lag_0',	'ay_autocorrelation__lag_1',	'ay_autocorrelation__lag_2',	'ay_autocorrelation__lag_3',	'ay_autocorrelation__lag_4',	'ay_autocorrelation__lag_5',	'ay_autocorrelation__lag_6',	'ay_autocorrelation__lag_7',	'ay_autocorrelation__lag_8',	'ay_autocorrelation__lag_9',	'ay_agg_autocorrelation__f_agg_"mean"__maylag_40',	'ay_agg_autocorrelation__f_agg_"median"__maylag_40',	'ay_agg_autocorrelation__f_agg_"var"__maylag_40',	'ay_partial_autocorrelation__lag_0',	'ay_partial_autocorrelation__lag_1',	'ay_partial_autocorrelation__lag_2',	'ay_partial_autocorrelation__lag_3',	'ay_partial_autocorrelation__lag_4',	'ay_partial_autocorrelation__lag_5',	'ay_partial_autocorrelation__lag_6',	'ay_partial_autocorrelation__lag_7',	'ay_partial_autocorrelation__lag_8',	'ay_partial_autocorrelation__lag_9',	'ay_number_cwt_peaks__n_1',	'ay_number_cwt_peaks__n_5',	'ay_number_peaks__n_1',	'ay_number_peaks__n_3',	'ay_number_peaks__n_5',	'ay_number_peaks__n_10',	'ay_number_peaks__n_50',	'ay_binned_entropy__may_bins_10',	'ay_index_mass_quantile__q_0.1',	'ay_index_mass_quantile__q_0.2',	'ay_index_mass_quantile__q_0.3',	'ay_index_mass_quantile__q_0.4',	'ay_index_mass_quantile__q_0.6',	'ay_index_mass_quantile__q_0.7',	'ay_index_mass_quantile__q_0.8',	'ay_index_mass_quantile__q_0.9',	'ay_cwt_coefficients__coeff_0__w_2__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_0__w_5__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_0__w_10__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_0__w_20__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_w_2__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_w_5__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_w_10__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_w_20__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_2__w_2__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_2__w_5__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_2__w_10__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_2__w_20__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_3__w_2__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_3__w_5__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_3__w_10__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_3__w_20__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_4__w_2__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_4__w_5__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_4__w_10__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_4__w_20__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_5__w_2__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_5__w_5__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_5__w_10__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_5__w_20__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_6__w_2__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_6__w_5__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_6__w_10__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_6__w_20__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_7__w_2__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_7__w_5__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_7__w_10__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_7__w_20__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_8__w_2__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_8__w_5__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_8__w_10__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_8__w_20__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_9__w_2__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_9__w_5__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_9__w_10__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_9__w_20__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_10__w_2__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_10__w_5__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_10__w_10__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_10__w_20__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_1w_2__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_1w_5__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_1w_10__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_1w_20__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_12__w_2__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_12__w_5__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_12__w_10__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_12__w_20__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_13__w_2__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_13__w_5__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_13__w_10__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_13__w_20__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_14__w_2__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_14__w_5__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_14__w_10__widths_(2 5 10 20)',	'ay_cwt_coefficients__coeff_14__w_20__widths_(2 5 10 20)',	'ay_spkt_welch_density__coeff_2',	'ay_spkt_welch_density__coeff_5',	'ay_spkt_welch_density__coeff_8',	'ay_ar_coefficient__coeff_0__k_10',	'ay_ar_coefficient__coeff_k_10',	'ay_ar_coefficient__coeff_2__k_10',	'ay_ar_coefficient__coeff_3__k_10',	'ay_ar_coefficient__coeff_4__k_10',	'ay_ar_coefficient__coeff_5__k_10',	'ay_ar_coefficient__coeff_6__k_10',	'ay_ar_coefficient__coeff_7__k_10',	'ay_ar_coefficient__coeff_8__k_10',	'ay_ar_coefficient__coeff_9__k_10',	'ay_ar_coefficient__coeff_10__k_10',	'ay_change_quantiles__f_agg_"mean"__isabs_False__qh_0.2__ql_0.0',	'ay_change_quantiles__f_agg_"var"__isabs_False__qh_0.2__ql_0.0',	'ay_change_quantiles__f_agg_"mean"__isabs_True__qh_0.2__ql_0.0',	'ay_change_quantiles__f_agg_"var"__isabs_True__qh_0.2__ql_0.0',	'ay_change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.0',	'ay_change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.0',	'ay_change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.0',	'ay_change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.0',	'ay_change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.0',	'ay_change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.0',	'ay_change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0',	'ay_change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.0',	'ay_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.0',	'ay_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0',	'ay_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0',	'ay_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.0',	'ay_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.0',	'ay_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.0',	'ay_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0',	'ay_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.0',	'ay_change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.2',	'ay_change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.2',	'ay_change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.2',	'ay_change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.2',	'ay_change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.2',	'ay_change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.2',	'ay_change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.2',	'ay_change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.2',	'ay_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.2',	'ay_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.2',	'ay_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2',	'ay_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.2',	'ay_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.2',	'ay_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.2',	'ay_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.2',	'ay_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.2',	'ay_change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.4',	'ay_change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.4',	'ay_change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.4',	'ay_change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.4',	'ay_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.4',	'ay_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.4',	'ay_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.4',	'ay_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.4',	'ay_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.4',	'ay_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.4',	'ay_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.4',	'ay_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.4',	'ay_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.6',	'ay_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.6',	'ay_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.6',	'ay_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.6',	'ay_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.6',	'ay_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.6',	'ay_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.6',	'ay_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.6',	'ay_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.8',	'ay_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.8',	'ay_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.8',	'ay_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.8',	'ay_fft_coefficient__attr_"real"__coeff_0',	'ay_fft_coefficient__attr_"real"__coeff_1',	'ay_fft_coefficient__attr_"real"__coeff_2',	'ay_fft_coefficient__attr_"real"__coeff_3',	'ay_fft_coefficient__attr_"real"__coeff_4',	'ay_fft_coefficient__attr_"real"__coeff_5',	'ay_fft_coefficient__attr_"real"__coeff_6',	'ay_fft_coefficient__attr_"real"__coeff_7',	'ay_fft_coefficient__attr_"real"__coeff_8',	'ay_fft_coefficient__attr_"real"__coeff_9',	'ay_fft_coefficient__attr_"real"__coeff_10',	'ay_fft_coefficient__attr_"real"__coeff_11',	'ay_fft_coefficient__attr_"real"__coeff_12',	'ay_fft_coefficient__attr_"real"__coeff_13',	'ay_fft_coefficient__attr_"real"__coeff_14',	'ay_fft_coefficient__attr_"real"__coeff_15',	'ay_fft_coefficient__attr_"real"__coeff_16',	'ay_fft_coefficient__attr_"real"__coeff_17',	'ay_fft_coefficient__attr_"real"__coeff_18',	'ay_fft_coefficient__attr_"real"__coeff_19',	'ay_fft_coefficient__attr_"real"__coeff_20',	'ay_fft_coefficient__attr_"real"__coeff_21',	'ay_fft_coefficient__attr_"real"__coeff_22',	'ay_fft_coefficient__attr_"real"__coeff_23',	'ay_fft_coefficient__attr_"real"__coeff_24',	'ay_fft_coefficient__attr_"real"__coeff_25',	'ay_fft_coefficient__attr_"real"__coeff_26',	'ay_fft_coefficient__attr_"real"__coeff_27',	'ay_fft_coefficient__attr_"real"__coeff_28',	'ay_fft_coefficient__attr_"real"__coeff_29',	'ay_fft_coefficient__attr_"real"__coeff_30',	'ay_fft_coefficient__attr_"real"__coeff_31',	'ay_fft_coefficient__attr_"real"__coeff_32',	'ay_fft_coefficient__attr_"real"__coeff_33',	'ay_fft_coefficient__attr_"real"__coeff_34',	'ay_fft_coefficient__attr_"real"__coeff_35',	'ay_fft_coefficient__attr_"real"__coeff_36',	'ay_fft_coefficient__attr_"real"__coeff_37',	'ay_fft_coefficient__attr_"real"__coeff_38',	'ay_fft_coefficient__attr_"real"__coeff_39',	'ay_fft_coefficient__attr_"real"__coeff_40',	'ay_fft_coefficient__attr_"real"__coeff_41',	'ay_fft_coefficient__attr_"real"__coeff_42',	'ay_fft_coefficient__attr_"real"__coeff_43',	'ay_fft_coefficient__attr_"real"__coeff_44',	'ay_fft_coefficient__attr_"real"__coeff_45',	'ay_fft_coefficient__attr_"real"__coeff_46',	'ay_fft_coefficient__attr_"real"__coeff_47',	'ay_fft_coefficient__attr_"real"__coeff_48',	'ay_fft_coefficient__attr_"real"__coeff_49',	'ay_fft_coefficient__attr_"real"__coeff_50',	'ay_fft_coefficient__attr_"real"__coeff_51',	'ay_fft_coefficient__attr_"real"__coeff_52',	'ay_fft_coefficient__attr_"real"__coeff_53',	'ay_fft_coefficient__attr_"real"__coeff_54',	'ay_fft_coefficient__attr_"real"__coeff_55',	'ay_fft_coefficient__attr_"real"__coeff_56',	'ay_fft_coefficient__attr_"real"__coeff_57',	'ay_fft_coefficient__attr_"real"__coeff_58',	'ay_fft_coefficient__attr_"real"__coeff_59',	'ay_fft_coefficient__attr_"real"__coeff_60',	'ay_fft_coefficient__attr_"real"__coeff_61',	'ay_fft_coefficient__attr_"real"__coeff_62',	'ay_fft_coefficient__attr_"real"__coeff_63',	'ay_fft_coefficient__attr_"real"__coeff_64',	'ay_fft_coefficient__attr_"real"__coeff_65',	'ay_fft_coefficient__attr_"real"__coeff_66',	'ay_fft_coefficient__attr_"real"__coeff_67',	'ay_fft_coefficient__attr_"real"__coeff_68',	'ay_fft_coefficient__attr_"real"__coeff_69',	'ay_fft_coefficient__attr_"real"__coeff_70',	'ay_fft_coefficient__attr_"real"__coeff_71',	'ay_fft_coefficient__attr_"real"__coeff_72',	'ay_fft_coefficient__attr_"real"__coeff_73',	'ay_fft_coefficient__attr_"real"__coeff_74',	'ay_fft_coefficient__attr_"real"__coeff_75',	'ay_fft_coefficient__attr_"real"__coeff_76',	'ay_fft_coefficient__attr_"real"__coeff_77',	'ay_fft_coefficient__attr_"real"__coeff_78',	'ay_fft_coefficient__attr_"real"__coeff_79',	'ay_fft_coefficient__attr_"real"__coeff_80',	'ay_fft_coefficient__attr_"real"__coeff_81',	'ay_fft_coefficient__attr_"real"__coeff_82',	'ay_fft_coefficient__attr_"real"__coeff_83',	'ay_fft_coefficient__attr_"real"__coeff_84',	'ay_fft_coefficient__attr_"real"__coeff_85',	'ay_fft_coefficient__attr_"real"__coeff_86',	'ay_fft_coefficient__attr_"real"__coeff_87',	'ay_fft_coefficient__attr_"real"__coeff_88',	'ay_fft_coefficient__attr_"real"__coeff_89',	'ay_fft_coefficient__attr_"real"__coeff_90',	'ay_fft_coefficient__attr_"real"__coeff_91',	'ay_fft_coefficient__attr_"real"__coeff_92',	'ay_fft_coefficient__attr_"real"__coeff_93',	'ay_fft_coefficient__attr_"real"__coeff_94',	'ay_fft_coefficient__attr_"real"__coeff_95',	'ay_fft_coefficient__attr_"real"__coeff_96',	'ay_fft_coefficient__attr_"real"__coeff_97',	'ay_fft_coefficient__attr_"real"__coeff_98',	'ay_fft_coefficient__attr_"real"__coeff_99',	'ay_fft_coefficient__attr_"imag"__coeff_0',	'ay_fft_coefficient__attr_"imag"__coeff_1',	'ay_fft_coefficient__attr_"imag"__coeff_2',	'ay_fft_coefficient__attr_"imag"__coeff_3',	'ay_fft_coefficient__attr_"imag"__coeff_4',	'ay_fft_coefficient__attr_"imag"__coeff_5',	'ay_fft_coefficient__attr_"imag"__coeff_6',	'ay_fft_coefficient__attr_"imag"__coeff_7',	'ay_fft_coefficient__attr_"imag"__coeff_8',	'ay_fft_coefficient__attr_"imag"__coeff_9',	'ay_fft_coefficient__attr_"imag"__coeff_10',	'ay_fft_coefficient__attr_"imag"__coeff_11',	'ay_fft_coefficient__attr_"imag"__coeff_12',	'ay_fft_coefficient__attr_"imag"__coeff_13',	'ay_fft_coefficient__attr_"imag"__coeff_14',	'ay_fft_coefficient__attr_"imag"__coeff_15',	'ay_fft_coefficient__attr_"imag"__coeff_16',	'ay_fft_coefficient__attr_"imag"__coeff_17',	'ay_fft_coefficient__attr_"imag"__coeff_18',	'ay_fft_coefficient__attr_"imag"__coeff_19',	'ay_fft_coefficient__attr_"imag"__coeff_20',	'ay_fft_coefficient__attr_"imag"__coeff_21',	'ay_fft_coefficient__attr_"imag"__coeff_22',	'ay_fft_coefficient__attr_"imag"__coeff_23',	'ay_fft_coefficient__attr_"imag"__coeff_24',	'ay_fft_coefficient__attr_"imag"__coeff_25',	'ay_fft_coefficient__attr_"imag"__coeff_26',	'ay_fft_coefficient__attr_"imag"__coeff_27',	'ay_fft_coefficient__attr_"imag"__coeff_28',	'ay_fft_coefficient__attr_"imag"__coeff_29',	'ay_fft_coefficient__attr_"imag"__coeff_30',	'ay_fft_coefficient__attr_"imag"__coeff_31',	'ay_fft_coefficient__attr_"imag"__coeff_32',	'ay_fft_coefficient__attr_"imag"__coeff_33',	'ay_fft_coefficient__attr_"imag"__coeff_34',	'ay_fft_coefficient__attr_"imag"__coeff_35',	'ay_fft_coefficient__attr_"imag"__coeff_36',	'ay_fft_coefficient__attr_"imag"__coeff_37',	'ay_fft_coefficient__attr_"imag"__coeff_38',	'ay_fft_coefficient__attr_"imag"__coeff_39',	'ay_fft_coefficient__attr_"imag"__coeff_40',	'ay_fft_coefficient__attr_"imag"__coeff_41',	'ay_fft_coefficient__attr_"imag"__coeff_42',	'ay_fft_coefficient__attr_"imag"__coeff_43',	'ay_fft_coefficient__attr_"imag"__coeff_44',	'ay_fft_coefficient__attr_"imag"__coeff_45',	'ay_fft_coefficient__attr_"imag"__coeff_46',	'ay_fft_coefficient__attr_"imag"__coeff_47',	'ay_fft_coefficient__attr_"imag"__coeff_48',	'ay_fft_coefficient__attr_"imag"__coeff_49',	'ay_fft_coefficient__attr_"imag"__coeff_50',	'ay_fft_coefficient__attr_"imag"__coeff_51',	'ay_fft_coefficient__attr_"imag"__coeff_52',	'ay_fft_coefficient__attr_"imag"__coeff_53',	'ay_fft_coefficient__attr_"imag"__coeff_54',	'ay_fft_coefficient__attr_"imag"__coeff_55',	'ay_fft_coefficient__attr_"imag"__coeff_56',	'ay_fft_coefficient__attr_"imag"__coeff_57',	'ay_fft_coefficient__attr_"imag"__coeff_58',	'ay_fft_coefficient__attr_"imag"__coeff_59',	'ay_fft_coefficient__attr_"imag"__coeff_60',	'ay_fft_coefficient__attr_"imag"__coeff_61',	'ay_fft_coefficient__attr_"imag"__coeff_62',	'ay_fft_coefficient__attr_"imag"__coeff_63',	'ay_fft_coefficient__attr_"imag"__coeff_64',	'ay_fft_coefficient__attr_"imag"__coeff_65',	'ay_fft_coefficient__attr_"imag"__coeff_66',	'ay_fft_coefficient__attr_"imag"__coeff_67',	'ay_fft_coefficient__attr_"imag"__coeff_68',	'ay_fft_coefficient__attr_"imag"__coeff_69',	'ay_fft_coefficient__attr_"imag"__coeff_70',	'ay_fft_coefficient__attr_"imag"__coeff_71',	'ay_fft_coefficient__attr_"imag"__coeff_72',	'ay_fft_coefficient__attr_"imag"__coeff_73',	'ay_fft_coefficient__attr_"imag"__coeff_74',	'ay_fft_coefficient__attr_"imag"__coeff_75',	'ay_fft_coefficient__attr_"imag"__coeff_76',	'ay_fft_coefficient__attr_"imag"__coeff_77',	'ay_fft_coefficient__attr_"imag"__coeff_78',	'ay_fft_coefficient__attr_"imag"__coeff_79',	'ay_fft_coefficient__attr_"imag"__coeff_80',	'ay_fft_coefficient__attr_"imag"__coeff_81',	'ay_fft_coefficient__attr_"imag"__coeff_82',	'ay_fft_coefficient__attr_"imag"__coeff_83',	'ay_fft_coefficient__attr_"imag"__coeff_84',	'ay_fft_coefficient__attr_"imag"__coeff_85',	'ay_fft_coefficient__attr_"imag"__coeff_86',	'ay_fft_coefficient__attr_"imag"__coeff_87',	'ay_fft_coefficient__attr_"imag"__coeff_88',	'ay_fft_coefficient__attr_"imag"__coeff_89',	'ay_fft_coefficient__attr_"imag"__coeff_90',	'ay_fft_coefficient__attr_"imag"__coeff_91',	'ay_fft_coefficient__attr_"imag"__coeff_92',	'ay_fft_coefficient__attr_"imag"__coeff_93',	'ay_fft_coefficient__attr_"imag"__coeff_94',	'ay_fft_coefficient__attr_"imag"__coeff_95',	'ay_fft_coefficient__attr_"imag"__coeff_96',	'ay_fft_coefficient__attr_"imag"__coeff_97',	'ay_fft_coefficient__attr_"imag"__coeff_98',	'ay_fft_coefficient__attr_"imag"__coeff_99',	'ay_fft_coefficient__attr_"abs"__coeff_0',	'ay_fft_coefficient__attr_"abs"__coeff_1',	'ay_fft_coefficient__attr_"abs"__coeff_2',	'ay_fft_coefficient__attr_"abs"__coeff_3',	'ay_fft_coefficient__attr_"abs"__coeff_4',	'ay_fft_coefficient__attr_"abs"__coeff_5',	'ay_fft_coefficient__attr_"abs"__coeff_6',	'ay_fft_coefficient__attr_"abs"__coeff_7',	'ay_fft_coefficient__attr_"abs"__coeff_8',	'ay_fft_coefficient__attr_"abs"__coeff_9',	'ay_fft_coefficient__attr_"abs"__coeff_10',	'ay_fft_coefficient__attr_"abs"__coeff_11',	'ay_fft_coefficient__attr_"abs"__coeff_12',	'ay_fft_coefficient__attr_"abs"__coeff_13',	'ay_fft_coefficient__attr_"abs"__coeff_14',	'ay_fft_coefficient__attr_"abs"__coeff_15',	'ay_fft_coefficient__attr_"abs"__coeff_16',	'ay_fft_coefficient__attr_"abs"__coeff_17',	'ay_fft_coefficient__attr_"abs"__coeff_18',	'ay_fft_coefficient__attr_"abs"__coeff_19',	'ay_fft_coefficient__attr_"abs"__coeff_20',	'ay_fft_coefficient__attr_"abs"__coeff_21',	'ay_fft_coefficient__attr_"abs"__coeff_22',	'ay_fft_coefficient__attr_"abs"__coeff_23',	'ay_fft_coefficient__attr_"abs"__coeff_24',	'ay_fft_coefficient__attr_"abs"__coeff_25',	'ay_fft_coefficient__attr_"abs"__coeff_26',	'ay_fft_coefficient__attr_"abs"__coeff_27',	'ay_fft_coefficient__attr_"abs"__coeff_28',	'ay_fft_coefficient__attr_"abs"__coeff_29',	'ay_fft_coefficient__attr_"abs"__coeff_30',	'ay_fft_coefficient__attr_"abs"__coeff_31',	'ay_fft_coefficient__attr_"abs"__coeff_32',	'ay_fft_coefficient__attr_"abs"__coeff_33',	'ay_fft_coefficient__attr_"abs"__coeff_34',	'ay_fft_coefficient__attr_"abs"__coeff_35',	'ay_fft_coefficient__attr_"abs"__coeff_36',	'ay_fft_coefficient__attr_"abs"__coeff_37',	'ay_fft_coefficient__attr_"abs"__coeff_38',	'ay_fft_coefficient__attr_"abs"__coeff_39',	'ay_fft_coefficient__attr_"abs"__coeff_40',	'ay_fft_coefficient__attr_"abs"__coeff_41',	'ay_fft_coefficient__attr_"abs"__coeff_42',	'ay_fft_coefficient__attr_"abs"__coeff_43',	'ay_fft_coefficient__attr_"abs"__coeff_44',	'ay_fft_coefficient__attr_"abs"__coeff_45',	'ay_fft_coefficient__attr_"abs"__coeff_46',	'ay_fft_coefficient__attr_"abs"__coeff_47',	'ay_fft_coefficient__attr_"abs"__coeff_48',	'ay_fft_coefficient__attr_"abs"__coeff_49',	'ay_fft_coefficient__attr_"abs"__coeff_50',	'ay_fft_coefficient__attr_"abs"__coeff_51',	'ay_fft_coefficient__attr_"abs"__coeff_52',	'ay_fft_coefficient__attr_"abs"__coeff_53',	'ay_fft_coefficient__attr_"abs"__coeff_54',	'ay_fft_coefficient__attr_"abs"__coeff_55',	'ay_fft_coefficient__attr_"abs"__coeff_56',	'ay_fft_coefficient__attr_"abs"__coeff_57',	'ay_fft_coefficient__attr_"abs"__coeff_58',	'ay_fft_coefficient__attr_"abs"__coeff_59',	'ay_fft_coefficient__attr_"abs"__coeff_60',	'ay_fft_coefficient__attr_"abs"__coeff_61',	'ay_fft_coefficient__attr_"abs"__coeff_62',	'ay_fft_coefficient__attr_"abs"__coeff_63',	'ay_fft_coefficient__attr_"abs"__coeff_64',	'ay_fft_coefficient__attr_"abs"__coeff_65',	'ay_fft_coefficient__attr_"abs"__coeff_66',	'ay_fft_coefficient__attr_"abs"__coeff_67',	'ay_fft_coefficient__attr_"abs"__coeff_68',	'ay_fft_coefficient__attr_"abs"__coeff_69',	'ay_fft_coefficient__attr_"abs"__coeff_70',	'ay_fft_coefficient__attr_"abs"__coeff_71',	'ay_fft_coefficient__attr_"abs"__coeff_72',	'ay_fft_coefficient__attr_"abs"__coeff_73',	'ay_fft_coefficient__attr_"abs"__coeff_74',	'ay_fft_coefficient__attr_"abs"__coeff_75',	'ay_fft_coefficient__attr_"abs"__coeff_76',	'ay_fft_coefficient__attr_"abs"__coeff_77',	'ay_fft_coefficient__attr_"abs"__coeff_78',	'ay_fft_coefficient__attr_"abs"__coeff_79',	'ay_fft_coefficient__attr_"abs"__coeff_80',	'ay_fft_coefficient__attr_"abs"__coeff_81',	'ay_fft_coefficient__attr_"abs"__coeff_82',	'ay_fft_coefficient__attr_"abs"__coeff_83',	'ay_fft_coefficient__attr_"abs"__coeff_84',	'ay_fft_coefficient__attr_"abs"__coeff_85',	'ay_fft_coefficient__attr_"abs"__coeff_86',	'ay_fft_coefficient__attr_"abs"__coeff_87',	'ay_fft_coefficient__attr_"abs"__coeff_88',	'ay_fft_coefficient__attr_"abs"__coeff_89',	'ay_fft_coefficient__attr_"abs"__coeff_90',	'ay_fft_coefficient__attr_"abs"__coeff_91',	'ay_fft_coefficient__attr_"abs"__coeff_92',	'ay_fft_coefficient__attr_"abs"__coeff_93',	'ay_fft_coefficient__attr_"abs"__coeff_94',	'ay_fft_coefficient__attr_"abs"__coeff_95',	'ay_fft_coefficient__attr_"abs"__coeff_96',	'ay_fft_coefficient__attr_"abs"__coeff_97',	'ay_fft_coefficient__attr_"abs"__coeff_98',	'ay_fft_coefficient__attr_"abs"__coeff_99',	'ay_fft_coefficient__attr_"angle"__coeff_0',	'ay_fft_coefficient__attr_"angle"__coeff_1',	'ay_fft_coefficient__attr_"angle"__coeff_2',	'ay_fft_coefficient__attr_"angle"__coeff_3',	'ay_fft_coefficient__attr_"angle"__coeff_4',	'ay_fft_coefficient__attr_"angle"__coeff_5',	'ay_fft_coefficient__attr_"angle"__coeff_6',	'ay_fft_coefficient__attr_"angle"__coeff_7',	'ay_fft_coefficient__attr_"angle"__coeff_8',	'ay_fft_coefficient__attr_"angle"__coeff_9',	'ay_fft_coefficient__attr_"angle"__coeff_10',	'ay_fft_coefficient__attr_"angle"__coeff_11',	'ay_fft_coefficient__attr_"angle"__coeff_12',	'ay_fft_coefficient__attr_"angle"__coeff_13',	'ay_fft_coefficient__attr_"angle"__coeff_14',	'ay_fft_coefficient__attr_"angle"__coeff_15',	'ay_fft_coefficient__attr_"angle"__coeff_16',	'ay_fft_coefficient__attr_"angle"__coeff_17',	'ay_fft_coefficient__attr_"angle"__coeff_18',	'ay_fft_coefficient__attr_"angle"__coeff_19',	'ay_fft_coefficient__attr_"angle"__coeff_20',	'ay_fft_coefficient__attr_"angle"__coeff_21',	'ay_fft_coefficient__attr_"angle"__coeff_22',	'ay_fft_coefficient__attr_"angle"__coeff_23',	'ay_fft_coefficient__attr_"angle"__coeff_24',	'ay_fft_coefficient__attr_"angle"__coeff_25',	'ay_fft_coefficient__attr_"angle"__coeff_26',	'ay_fft_coefficient__attr_"angle"__coeff_27',	'ay_fft_coefficient__attr_"angle"__coeff_28',	'ay_fft_coefficient__attr_"angle"__coeff_29',	'ay_fft_coefficient__attr_"angle"__coeff_30',	'ay_fft_coefficient__attr_"angle"__coeff_31',	'ay_fft_coefficient__attr_"angle"__coeff_32',	'ay_fft_coefficient__attr_"angle"__coeff_33',	'ay_fft_coefficient__attr_"angle"__coeff_34',	'ay_fft_coefficient__attr_"angle"__coeff_35',	'ay_fft_coefficient__attr_"angle"__coeff_36',	'ay_fft_coefficient__attr_"angle"__coeff_37',	'ay_fft_coefficient__attr_"angle"__coeff_38',	'ay_fft_coefficient__attr_"angle"__coeff_39',	'ay_fft_coefficient__attr_"angle"__coeff_40',	'ay_fft_coefficient__attr_"angle"__coeff_41',	'ay_fft_coefficient__attr_"angle"__coeff_42',	'ay_fft_coefficient__attr_"angle"__coeff_43',	'ay_fft_coefficient__attr_"angle"__coeff_44',	'ay_fft_coefficient__attr_"angle"__coeff_45',	'ay_fft_coefficient__attr_"angle"__coeff_46',	'ay_fft_coefficient__attr_"angle"__coeff_47',	'ay_fft_coefficient__attr_"angle"__coeff_48',	'ay_fft_coefficient__attr_"angle"__coeff_49',	'ay_fft_coefficient__attr_"angle"__coeff_50',	'ay_fft_coefficient__attr_"angle"__coeff_51',	'ay_fft_coefficient__attr_"angle"__coeff_52',	'ay_fft_coefficient__attr_"angle"__coeff_53',	'ay_fft_coefficient__attr_"angle"__coeff_54',	'ay_fft_coefficient__attr_"angle"__coeff_55',	'ay_fft_coefficient__attr_"angle"__coeff_56',	'ay_fft_coefficient__attr_"angle"__coeff_57',	'ay_fft_coefficient__attr_"angle"__coeff_58',	'ay_fft_coefficient__attr_"angle"__coeff_59',	'ay_fft_coefficient__attr_"angle"__coeff_60',	'ay_fft_coefficient__attr_"angle"__coeff_61',	'ay_fft_coefficient__attr_"angle"__coeff_62',	'ay_fft_coefficient__attr_"angle"__coeff_63',	'ay_fft_coefficient__attr_"angle"__coeff_64',	'ay_fft_coefficient__attr_"angle"__coeff_65',	'ay_fft_coefficient__attr_"angle"__coeff_66',	'ay_fft_coefficient__attr_"angle"__coeff_67',	'ay_fft_coefficient__attr_"angle"__coeff_68',	'ay_fft_coefficient__attr_"angle"__coeff_69',	'ay_fft_coefficient__attr_"angle"__coeff_70',	'ay_fft_coefficient__attr_"angle"__coeff_71',	'ay_fft_coefficient__attr_"angle"__coeff_72',	'ay_fft_coefficient__attr_"angle"__coeff_73',	'ay_fft_coefficient__attr_"angle"__coeff_74',	'ay_fft_coefficient__attr_"angle"__coeff_75',	'ay_fft_coefficient__attr_"angle"__coeff_76',	'ay_fft_coefficient__attr_"angle"__coeff_77',	'ay_fft_coefficient__attr_"angle"__coeff_78',	'ay_fft_coefficient__attr_"angle"__coeff_79',	'ay_fft_coefficient__attr_"angle"__coeff_80',	'ay_fft_coefficient__attr_"angle"__coeff_81',	'ay_fft_coefficient__attr_"angle"__coeff_82',	'ay_fft_coefficient__attr_"angle"__coeff_83',	'ay_fft_coefficient__attr_"angle"__coeff_84',	'ay_fft_coefficient__attr_"angle"__coeff_85',	'ay_fft_coefficient__attr_"angle"__coeff_86',	'ay_fft_coefficient__attr_"angle"__coeff_87',	'ay_fft_coefficient__attr_"angle"__coeff_88',	'ay_fft_coefficient__attr_"angle"__coeff_89',	'ay_fft_coefficient__attr_"angle"__coeff_90',	'ay_fft_coefficient__attr_"angle"__coeff_91',	'ay_fft_coefficient__attr_"angle"__coeff_92',	'ay_fft_coefficient__attr_"angle"__coeff_93',	'ay_fft_coefficient__attr_"angle"__coeff_94',	'ay_fft_coefficient__attr_"angle"__coeff_95',	'ay_fft_coefficient__attr_"angle"__coeff_96',	'ay_fft_coefficient__attr_"angle"__coeff_97',	'ay_fft_coefficient__attr_"angle"__coeff_98',	'ay_fft_coefficient__attr_"angle"__coeff_99',	'ay_fft_aggregated__aggtype_"centroid"',	'ay_fft_aggregated__aggtype_"variance"',	'ay_fft_aggregated__aggtype_"skew"',	'ay_fft_aggregated__aggtype_"kurtosis"',	'ay_value_count__value_0',	'ay_value_count__value_1',	'ay_value_count__value_-1',	'ay_range_count__may_min_-1',	'ay_range_count__may_0__min_1000000000000.0',	'ay_range_count__may_1000000000000.0__min_0',	'ay_approximate_entropy__m_2__r_0.1',	'ay_approximate_entropy__m_2__r_0.3',	'ay_approximate_entropy__m_2__r_0.5',	'ay_approximate_entropy__m_2__r_0.7',	'ay_approximate_entropy__m_2__r_0.9',	'ay_friedrich_coefficients__coeff_0__m_3__r_30',	'ay_friedrich_coefficients__coeff_m_3__r_30',	'ay_friedrich_coefficients__coeff_2__m_3__r_30',	'ay_friedrich_coefficients__coeff_3__m_3__r_30',	'ay_may_langevin_fixed_point__m_3__r_30',	'ay_linear_trend__attr_"pvalue"',	'ay_linear_trend__attr_"rvalue"',	'ay_linear_trend__attr_"intercept"',	'ay_linear_trend__attr_"slope"',	'ay_linear_trend__attr_"stderr"',	'ay_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"may"',	'ay_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"min"',	'ay_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"mean"',	'ay_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"var"',	'ay_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"may"',	'ay_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"min"',	'ay_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"mean"',	'ay_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"var"',	'ay_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"may"',	'ay_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"min"',	'ay_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"mean"',	'ay_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"var"',	'ay_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"may"',	'ay_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"min"',	'ay_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"mean"',	'ay_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"var"',	'ay_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"may"',	'ay_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"min"',	'ay_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"mean"',	'ay_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"var"',	'ay_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"may"',	'ay_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"min"',	'ay_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"mean"',	'ay_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"var"',	'ay_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"may"',	'ay_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"min"',	'ay_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"mean"',	'ay_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"var"',	'ay_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"may"',	'ay_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"min"',	'ay_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"mean"',	'ay_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"var"',	'ay_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"may"',	'ay_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"min"',	'ay_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"mean"',	'ay_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"var"',	'ay_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"may"',	'ay_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"min"',	'ay_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"mean"',	'ay_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"var"',	'ay_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"may"',	'ay_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"min"',	'ay_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"mean"',	'ay_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"var"',	'ay_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"may"',	'ay_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"min"',	'ay_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"mean"',	'ay_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"var"',	'ay_augmented_dickey_fuller__attr_"teststat"__autolag_"AIC"',	'ay_augmented_dickey_fuller__attr_"pvalue"__autolag_"AIC"',	'ay_augmented_dickey_fuller__attr_"usedlag"__autolag_"AIC"',	'ay_number_crossing_m__m_0',	'ay_number_crossing_m__m_-1',	'ay_number_crossing_m__m_1',	'ay_eneray_ratio_by_chunks__num_segments_10__segment_focus_0',	'ay_eneray_ratio_by_chunks__num_segments_10__segment_focus_1',	'ay_eneray_ratio_by_chunks__num_segments_10__segment_focus_2',	'ay_eneray_ratio_by_chunks__num_segments_10__segment_focus_3',	'ay_eneray_ratio_by_chunks__num_segments_10__segment_focus_4',	'ay_eneray_ratio_by_chunks__num_segments_10__segment_focus_5',	'ay_eneray_ratio_by_chunks__num_segments_10__segment_focus_6',	'ay_eneray_ratio_by_chunks__num_segments_10__segment_focus_7',	'ay_eneray_ratio_by_chunks__num_segments_10__segment_focus_8',	'ay_eneray_ratio_by_chunks__num_segments_10__segment_focus_9',	'ay_ratio_beyond_r_sigma__r_0.5',	'ay_ratio_beyond_r_sigma__r_1',	'ay_ratio_beyond_r_sigma__r_1.5',	'ay_ratio_beyond_r_sigma__r_2',	'ay_ratio_beyond_r_sigma__r_2.5',	'ay_ratio_beyond_r_sigma__r_3',	'ay_ratio_beyond_r_sigma__r_5',	'ay_ratio_beyond_r_sigma__r_6',	'ay_ratio_beyond_r_sigma__r_7',	'ay_ratio_beyond_r_sigma__r_10',	'ay_count_above__t_0',	'ay_count_below__t_0',	'ay_lempel_ziv_complexity__bins_2',	'ay_lempel_ziv_complexity__bins_3',	'ay_lempel_ziv_complexity__bins_5',	'ay_lempel_ziv_complexity__bins_10',	'ay_lempel_ziv_complexity__bins_100',	'ay_fourier_entropy__bins_2',	'ay_fourier_entropy__bins_3',	'ay_fourier_entropy__bins_5',	'ay_fourier_entropy__bins_10',	'ay_fourier_entropy__bins_100',	'ay_permutation_entropy__dimension_3__tau_1',	'ay_permutation_entropy__dimension_4__tau_1',	'ay_permutation_entropy__dimension_5__tau_1',	'ay_permutation_entropy__dimension_6__tau_1',	'ay_permutation_entropy__dimension_7__tau_1',	'ay_query_similarity_count__query_None__threshold_0.0',	'ay_matrix_profile__feature_"min"__threshold_0.98',	'ay_matrix_profile__feature_"may"__threshold_0.98',	'ay_matrix_profile__feature_"mean"__threshold_0.98',	'ay_matrix_profile__feature_"median"__threshold_0.98',	'ay_matrix_profile__feature_"25"__threshold_0.98',	'ay_matrix_profile__feature_"75"__threshold_0.98']
fea_user_1ay=pd.DataFrame(fea_user_1ay.reshape(Gx_list_trans.shape[1], 787))
fea_user_1ay.columns=[header_ay]
fea_user_1ay

,ay_variance_larger_than_standard_deviation,ay_has_duplicate_max,ay_has_duplicate_min,ay_has_duplicate,ay_sum_values,ay_abs_eneray,ay_mean_abs_change,ay_mean_change,ay_mean_second_derivative_central,ay_median,...,ay_permutation_entropy__dimension_5__tau_1,ay_permutation_entropy__dimension_6__tau_1,ay_permutation_entropy__dimension_7__tau_1,ay_query_similarity_count__query_None__threshold_0.0,"ay_matrix_profile__feature_""min""__threshold_0.98","ay_matrix_profile__feature_""may""__threshold_0.98","ay_matrix_profile__feature_""mean""__threshold_0.98","ay_matrix_profile__feature_""median""__threshold_0.98","ay_matrix_profile__feature_""25""__threshold_0.98","ay_matrix_profile__feature_""75""__threshold_0.98"
0,0.0,0.0,0.0,0.0,-0.5334,0.042064,0.034558,-0.004516,0.001686,-0.03540,...,2.566599,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,-0.3059,0.023882,0.022142,0.002984,0.001272,-0.01035,...,2.306669,2.615631,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,0.0,-0.2060,0.019480,0.021926,-0.001253,0.001222,-0.00850,...,2.479956,2.615631,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,0.0,0.0,-0.1183,0.016741,0.021474,-0.000768,-0.000919,-0.00775,...,2.772589,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,0.0,0.0,-0.1566,0.015797,0.020600,-0.000253,0.000389,-0.00750,...,2.772589,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,0.0,0.0,0.0,0.0,-0.5904,0.255723,0.099058,-0.001953,0.000214,-0.02370,...,2.685945,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
805,0.0,0.0,0.0,0.0,-0.2326,0.049636,0.039111,0.008805,-0.001986,-0.00485,...,2.512659,2.523211,2.540036,NaN,NaN,NaN,NaN,NaN,NaN,NaN
806,0.0,0.0,0.0,0.0,0.0290,0.019134,0.027521,0.000300,-0.001433,0.00260,...,2.685945,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
807,0.0,0.0,0.0,0.0,0.1597,0.020122,0.031005,-0.000363,0.000858,0.00540,...,2.772589,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
r_ay=fea_user_1ay.dropna(axis='columns',how='all')
r_ay=r_ay.fillna(0)
binary_features_ay=['ay_variance_larger_than_standard_deviation','ay_has_duplicate_max',	'ay_has_duplicate_min',	'ay_has_duplicate','ay_symmetry_looking__r_0.0',	'ay_symmetry_looking__r_0.05',	'ay_symmetry_looking__r_0.1',	'ay_symmetry_looking__r_0.15000000000000002',	'ay_symmetry_looking__r_0.2',	'ay_symmetry_looking__r_0.25',	'ay_symmetry_looking__r_0.30000000000000004',	'ay_symmetry_looking__r_0.35000000000000003',	'ay_symmetry_looking__r_0.4',	'ay_symmetry_looking__r_0.45',	'ay_symmetry_looking__r_0.5',	'ay_symmetry_looking__r_0.55',	'ay_symmetry_looking__r_0.6000000000000001',	'ay_symmetry_looking__r_0.65',	'ay_symmetry_looking__r_0.7000000000000001',	'ay_symmetry_looking__r_0.75',	'ay_symmetry_looking__r_0.8',	'ay_symmetry_looking__r_0.8500000000000001',	'ay_symmetry_looking__r_0.9',	'ay_symmetry_looking__r_0.9500000000000001',	'ay_large_standard_deviation__r_0.05',	'ay_large_standard_deviation__r_0.1',	'ay_large_standard_deviation__r_0.15000000000000002',	'ay_large_standard_deviation__r_0.2',	'ay_large_standard_deviation__r_0.25',	'ay_large_standard_deviation__r_0.30000000000000004',	'ay_large_standard_deviation__r_0.35000000000000003',	'ay_large_standard_deviation__r_0.4',	'ay_large_standard_deviation__r_0.45',	'ay_large_standard_deviation__r_0.5',	'ay_large_standard_deviation__r_0.55',	'ay_large_standard_deviation__r_0.6000000000000001',	'ay_large_standard_deviation__r_0.65',	'ay_large_standard_deviation__r_0.7000000000000001',	'ay_large_standard_deviation__r_0.75',	'ay_large_standard_deviation__r_0.8',	'ay_large_standard_deviation__r_0.8500000000000001',	'ay_large_standard_deviation__r_0.9',	'ay_large_standard_deviation__r_0.9500000000000001','ay_value_count__value_0',	'ay_value_count__value_1',	'ay_value_count__value_-1']
r_ay=r_ay.drop(binary_features_ay, axis=1)
r_ay

d:\m-tech-data-science\tools\python 3.7\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,ay_sum_values,ay_abs_eneray,ay_mean_abs_change,ay_mean_change,ay_mean_second_derivative_central,ay_median,ay_mean,ay_length,ay_standard_deviation,ay_variation_coefficient,...,ay_permutation_entropy__dimension_4__tau_1,ay_permutation_entropy__dimension_5__tau_1,ay_permutation_entropy__dimension_6__tau_1,ay_permutation_entropy__dimension_7__tau_1,"ay_matrix_profile__feature_""min""__threshold_0.98","ay_matrix_profile__feature_""may""__threshold_0.98","ay_matrix_profile__feature_""mean""__threshold_0.98","ay_matrix_profile__feature_""median""__threshold_0.98","ay_matrix_profile__feature_""25""__threshold_0.98","ay_matrix_profile__feature_""75""__threshold_0.98"
0,-0.5334,0.042064,0.034558,-0.004516,0.001686,-0.03540,-0.026670,20.0,0.037308,-1.398891,...,2.278303,2.566599,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.3059,0.023882,0.022142,0.002984,0.001272,-0.01035,-0.015295,20.0,0.030986,-2.025923,...,1.870569,2.306669,2.615631,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.2060,0.019480,0.021926,-0.001253,0.001222,-0.00850,-0.010300,20.0,0.029461,-2.860253,...,2.262386,2.479956,2.615631,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.1183,0.016741,0.021474,-0.000768,-0.000919,-0.00775,-0.005915,20.0,0.028321,-4.787920,...,2.588573,2.772589,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.1566,0.015797,0.020600,-0.000253,0.000389,-0.00750,-0.007830,20.0,0.026991,-3.447136,...,2.476247,2.772589,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,-0.5904,0.255723,0.099058,-0.001953,0.000214,-0.02370,-0.029520,20.0,0.109154,-3.697645,...,2.476247,2.685945,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
805,-0.2326,0.049636,0.039111,0.008805,-0.001986,-0.00485,-0.011630,20.0,0.048441,-4.165172,...,2.115209,2.512659,2.523211,2.540036,0.0,0.0,0.0,0.0,0.0,0.0
806,0.0290,0.019134,0.027521,0.000300,-0.001433,0.00260,0.001450,20.0,0.030897,21.308141,...,2.231607,2.685945,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
807,0.1597,0.020122,0.031005,-0.000363,0.000858,0.00540,0.007985,20.0,0.030698,3.844438,...,2.588573,2.772589,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0


### Z-axis accelerometer 

In [28]:
Az_list=pd.DataFrame(Az_list)
Az_list=Az_list.astype(float)
Az_list_trans=Az_list.T
Az_list_trans

,0,1,2,3,4,5,6,7,8,9,...,799,800,801,802,803,804,805,806,807,808
0,0.7691,0.0324,0.1286,-0.0118,-0.0021,-0.0319,-0.0502,0.4245,-0.0859,0.0654,...,-0.9532,0.6667,-0.0229,0.0080,0.0698,0.0674,-1.8851,-0.1562,0.0152,0.0079
1,0.2126,0.0820,0.0334,0.0449,-0.1045,-0.1408,-0.1716,0.2395,0.1125,0.0756,...,-0.9316,0.3494,0.0840,-0.1202,0.0061,0.0451,0.0302,0.1240,0.1158,0.0101
2,-0.4076,0.0832,-0.0412,-0.1249,0.0691,-0.0379,0.0586,-0.2123,0.1125,0.0591,...,1.2669,-0.2088,0.1705,-0.0389,-0.0264,0.0435,1.0355,0.3560,0.2505,0.0644
3,-0.3333,-0.0058,-0.0013,-0.1216,0.1243,0.0240,0.1940,-0.3663,-0.0026,0.0061,...,1.3499,-0.4646,0.2518,0.1018,-0.0240,0.0869,0.9239,0.1933,0.2334,0.0556
4,-0.0488,-0.2217,-0.0015,-0.0070,0.0507,-0.0356,0.1917,-0.2894,-0.1066,-0.0950,...,0.6036,-0.3692,0.1858,0.1107,0.0186,-0.1585,-0.3049,0.0365,0.1018,-0.0360
5,-0.1329,-0.2396,-0.0761,-0.0817,0.0197,-0.0452,-0.1779,-0.1481,-0.1615,-0.2194,...,-0.2532,-0.1043,0.0916,0.0603,0.0038,-2.3341,-0.5457,-0.0218,0.1022,-0.1175
6,-0.0805,-0.0634,-0.0745,-0.1956,-0.0976,-0.0902,-0.4136,0.0713,-0.0548,-0.1897,...,-0.7592,0.1671,0.0166,0.0681,0.0527,3.5905,0.0013,0.0140,0.0279,0.0838
7,-0.1636,-0.0822,0.0125,-0.2677,-0.1215,-0.1114,0.0351,0.1444,-0.0499,-0.0861,...,-0.6622,0.1691,0.0800,0.0320,0.0707,2.0796,0.5726,0.0120,0.0499,0.3038
8,-0.1768,-0.0533,-0.0009,-0.1463,-0.0607,-0.1116,-0.1319,-0.0734,-0.1361,0.1246,...,0.0031,0.2002,0.1162,0.0503,0.1471,-0.1751,0.3338,-0.0377,0.0185,0.1807
9,-0.0152,0.0271,-0.0551,-0.0517,0.0269,0.0226,-0.0350,-0.1759,-0.0526,0.1955,...,0.5379,0.0606,0.0324,0.1164,0.0504,-1.7060,-0.1058,0.0075,-0.0233,0.1618


In [29]:
from tsfresh import extract_features
#c=c.fillna(0)
l_az=[]
z_az=[]
u_az=[]
X_az=[]
for i in range (Gx_list_trans.shape[1]):
    p_az=Az_list_trans.iloc[:,i]
    p_az=pd.DataFrame(p_az)
    p_az['ID'] = '1'
    X_az= extract_features(p_az, column_id='ID')
    #print(X.shape)
    l_az.append(X_az)
    #m.append(l)
    z_az=l_az
u_az=z_az  

Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.75s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.74s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.84s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.22s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.17s/it]


Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.14s/it]


In [30]:
fea_user_1az=np.array(u_az)
header_az=['az_variance_larger_than_standard_deviation',	'az_has_duplicate_max',	'az_has_duplicate_min',	'az_has_duplicate',	'az_sum_values',	'az_abs_eneraz',	'az_mean_abs_change',	'az_mean_change',	'az_mean_second_derivative_central',	'az_median',	'az_mean',	'az_length',	'az_standard_deviation',	'az_variation_coefficient',	'az_variance',	'az_skewness',	'az_kurtosis',	'az_root_mean_square',	'az_absolute_sum_of_changes',	'az_longest_strike_below_mean',	'az_longest_strike_above_mean',	'az_count_above_mean',	'az_count_below_mean',	'az_last_location_of_mazimum',	'az_first_location_of_mazimum',	'az_last_location_of_minimum',	'az_first_location_of_minimum',	'az_percentage_of_reoccurring_values_to_all_values',	'az_percentage_of_reoccurring_datapoints_to_all_datapoints',	'az_sum_of_reoccurring_values',	'az_sum_of_reoccurring_data_points',	'az_ratio_value_number_to_time_series_length',	'az_sample_entropy',	'az_mazimum',	'az_minimum',	'az_benford_correlation',	'az_time_reversal_asymmetry_statistic__lag_1',	'az_time_reversal_asymmetry_statistic__lag_2',	'az_time_reversal_asymmetry_statistic__lag_3',	'az_c3__lag_1',	'az_c3__lag_2',	'az_c3__lag_3',	'az_cid_ce__normalize_True',	'az_cid_ce__normalize_False',	'az_symmetry_looking__r_0.0',	'az_symmetry_looking__r_0.05',	'az_symmetry_looking__r_0.1',	'az_symmetry_looking__r_0.15000000000000002',	'az_symmetry_looking__r_0.2',	'az_symmetry_looking__r_0.25',	'az_symmetry_looking__r_0.30000000000000004',	'az_symmetry_looking__r_0.35000000000000003',	'az_symmetry_looking__r_0.4',	'az_symmetry_looking__r_0.45',	'az_symmetry_looking__r_0.5',	'az_symmetry_looking__r_0.55',	'az_symmetry_looking__r_0.6000000000000001',	'az_symmetry_looking__r_0.65',	'az_symmetry_looking__r_0.7000000000000001',	'az_symmetry_looking__r_0.75',	'az_symmetry_looking__r_0.8',	'az_symmetry_looking__r_0.8500000000000001',	'az_symmetry_looking__r_0.9',	'az_symmetry_looking__r_0.9500000000000001',	'az_large_standard_deviation__r_0.05',	'az_large_standard_deviation__r_0.1',	'az_large_standard_deviation__r_0.15000000000000002',	'az_large_standard_deviation__r_0.2',	'az_large_standard_deviation__r_0.25',	'az_large_standard_deviation__r_0.30000000000000004',	'az_large_standard_deviation__r_0.35000000000000003',	'az_large_standard_deviation__r_0.4',	'az_large_standard_deviation__r_0.45',	'az_large_standard_deviation__r_0.5',	'az_large_standard_deviation__r_0.55',	'az_large_standard_deviation__r_0.6000000000000001',	'az_large_standard_deviation__r_0.65',	'az_large_standard_deviation__r_0.7000000000000001',	'az_large_standard_deviation__r_0.75',	'az_large_standard_deviation__r_0.8',	'az_large_standard_deviation__r_0.8500000000000001',	'az_large_standard_deviation__r_0.9',	'az_large_standard_deviation__r_0.9500000000000001',	'az_quantile__q_0.1',	'az_quantile__q_0.2',	'az_quantile__q_0.3',	'az_quantile__q_0.4',	'az_quantile__q_0.6',	'az_quantile__q_0.7',	'az_quantile__q_0.8',	'az_quantile__q_0.9',	'az_autocorrelation__lag_0',	'az_autocorrelation__lag_1',	'az_autocorrelation__lag_2',	'az_autocorrelation__lag_3',	'az_autocorrelation__lag_4',	'az_autocorrelation__lag_5',	'az_autocorrelation__lag_6',	'az_autocorrelation__lag_7',	'az_autocorrelation__lag_8',	'az_autocorrelation__lag_9',	'az_agg_autocorrelation__f_agg_"mean"__mazlag_40',	'az_agg_autocorrelation__f_agg_"median"__mazlag_40',	'az_agg_autocorrelation__f_agg_"var"__mazlag_40',	'az_partial_autocorrelation__lag_0',	'az_partial_autocorrelation__lag_1',	'az_partial_autocorrelation__lag_2',	'az_partial_autocorrelation__lag_3',	'az_partial_autocorrelation__lag_4',	'az_partial_autocorrelation__lag_5',	'az_partial_autocorrelation__lag_6',	'az_partial_autocorrelation__lag_7',	'az_partial_autocorrelation__lag_8',	'az_partial_autocorrelation__lag_9',	'az_number_cwt_peaks__n_1',	'az_number_cwt_peaks__n_5',	'az_number_peaks__n_1',	'az_number_peaks__n_3',	'az_number_peaks__n_5',	'az_number_peaks__n_10',	'az_number_peaks__n_50',	'az_binned_entropy__maz_bins_10',	'az_index_mass_quantile__q_0.1',	'az_index_mass_quantile__q_0.2',	'az_index_mass_quantile__q_0.3',	'az_index_mass_quantile__q_0.4',	'az_index_mass_quantile__q_0.6',	'az_index_mass_quantile__q_0.7',	'az_index_mass_quantile__q_0.8',	'az_index_mass_quantile__q_0.9',	'az_cwt_coefficients__coeff_0__w_2__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_0__w_5__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_0__w_10__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_0__w_20__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_w_2__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_w_5__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_w_10__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_w_20__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_2__w_2__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_2__w_5__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_2__w_10__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_2__w_20__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_3__w_2__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_3__w_5__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_3__w_10__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_3__w_20__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_4__w_2__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_4__w_5__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_4__w_10__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_4__w_20__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_5__w_2__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_5__w_5__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_5__w_10__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_5__w_20__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_6__w_2__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_6__w_5__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_6__w_10__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_6__w_20__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_7__w_2__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_7__w_5__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_7__w_10__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_7__w_20__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_8__w_2__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_8__w_5__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_8__w_10__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_8__w_20__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_9__w_2__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_9__w_5__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_9__w_10__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_9__w_20__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_10__w_2__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_10__w_5__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_10__w_10__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_10__w_20__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_1w_2__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_1w_5__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_1w_10__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_1w_20__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_12__w_2__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_12__w_5__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_12__w_10__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_12__w_20__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_13__w_2__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_13__w_5__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_13__w_10__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_13__w_20__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_14__w_2__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_14__w_5__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_14__w_10__widths_(2 5 10 20)',	'az_cwt_coefficients__coeff_14__w_20__widths_(2 5 10 20)',	'az_spkt_welch_density__coeff_2',	'az_spkt_welch_density__coeff_5',	'az_spkt_welch_density__coeff_8',	'az_ar_coefficient__coeff_0__k_10',	'az_ar_coefficient__coeff_k_10',	'az_ar_coefficient__coeff_2__k_10',	'az_ar_coefficient__coeff_3__k_10',	'az_ar_coefficient__coeff_4__k_10',	'az_ar_coefficient__coeff_5__k_10',	'az_ar_coefficient__coeff_6__k_10',	'az_ar_coefficient__coeff_7__k_10',	'az_ar_coefficient__coeff_8__k_10',	'az_ar_coefficient__coeff_9__k_10',	'az_ar_coefficient__coeff_10__k_10',	'az_change_quantiles__f_agg_"mean"__isabs_False__qh_0.2__ql_0.0',	'az_change_quantiles__f_agg_"var"__isabs_False__qh_0.2__ql_0.0',	'az_change_quantiles__f_agg_"mean"__isabs_True__qh_0.2__ql_0.0',	'az_change_quantiles__f_agg_"var"__isabs_True__qh_0.2__ql_0.0',	'az_change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.0',	'az_change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.0',	'az_change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.0',	'az_change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.0',	'az_change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.0',	'az_change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.0',	'az_change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0',	'az_change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.0',	'az_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.0',	'az_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0',	'az_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0',	'az_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.0',	'az_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.0',	'az_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.0',	'az_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0',	'az_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.0',	'az_change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.2',	'az_change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.2',	'az_change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.2',	'az_change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.2',	'az_change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.2',	'az_change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.2',	'az_change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.2',	'az_change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.2',	'az_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.2',	'az_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.2',	'az_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2',	'az_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.2',	'az_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.2',	'az_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.2',	'az_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.2',	'az_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.2',	'az_change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.4',	'az_change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.4',	'az_change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.4',	'az_change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.4',	'az_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.4',	'az_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.4',	'az_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.4',	'az_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.4',	'az_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.4',	'az_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.4',	'az_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.4',	'az_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.4',	'az_change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.6',	'az_change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.6',	'az_change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.6',	'az_change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.6',	'az_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.6',	'az_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.6',	'az_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.6',	'az_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.6',	'az_change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.8',	'az_change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.8',	'az_change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.8',	'az_change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.8',	'az_fft_coefficient__attr_"real"__coeff_0',	'az_fft_coefficient__attr_"real"__coeff_1',	'az_fft_coefficient__attr_"real"__coeff_2',	'az_fft_coefficient__attr_"real"__coeff_3',	'az_fft_coefficient__attr_"real"__coeff_4',	'az_fft_coefficient__attr_"real"__coeff_5',	'az_fft_coefficient__attr_"real"__coeff_6',	'az_fft_coefficient__attr_"real"__coeff_7',	'az_fft_coefficient__attr_"real"__coeff_8',	'az_fft_coefficient__attr_"real"__coeff_9',	'az_fft_coefficient__attr_"real"__coeff_10',	'az_fft_coefficient__attr_"real"__coeff_11',	'az_fft_coefficient__attr_"real"__coeff_12',	'az_fft_coefficient__attr_"real"__coeff_13',	'az_fft_coefficient__attr_"real"__coeff_14',	'az_fft_coefficient__attr_"real"__coeff_15',	'az_fft_coefficient__attr_"real"__coeff_16',	'az_fft_coefficient__attr_"real"__coeff_17',	'az_fft_coefficient__attr_"real"__coeff_18',	'az_fft_coefficient__attr_"real"__coeff_19',	'az_fft_coefficient__attr_"real"__coeff_20',	'az_fft_coefficient__attr_"real"__coeff_21',	'az_fft_coefficient__attr_"real"__coeff_22',	'az_fft_coefficient__attr_"real"__coeff_23',	'az_fft_coefficient__attr_"real"__coeff_24',	'az_fft_coefficient__attr_"real"__coeff_25',	'az_fft_coefficient__attr_"real"__coeff_26',	'az_fft_coefficient__attr_"real"__coeff_27',	'az_fft_coefficient__attr_"real"__coeff_28',	'az_fft_coefficient__attr_"real"__coeff_29',	'az_fft_coefficient__attr_"real"__coeff_30',	'az_fft_coefficient__attr_"real"__coeff_31',	'az_fft_coefficient__attr_"real"__coeff_32',	'az_fft_coefficient__attr_"real"__coeff_33',	'az_fft_coefficient__attr_"real"__coeff_34',	'az_fft_coefficient__attr_"real"__coeff_35',	'az_fft_coefficient__attr_"real"__coeff_36',	'az_fft_coefficient__attr_"real"__coeff_37',	'az_fft_coefficient__attr_"real"__coeff_38',	'az_fft_coefficient__attr_"real"__coeff_39',	'az_fft_coefficient__attr_"real"__coeff_40',	'az_fft_coefficient__attr_"real"__coeff_41',	'az_fft_coefficient__attr_"real"__coeff_42',	'az_fft_coefficient__attr_"real"__coeff_43',	'az_fft_coefficient__attr_"real"__coeff_44',	'az_fft_coefficient__attr_"real"__coeff_45',	'az_fft_coefficient__attr_"real"__coeff_46',	'az_fft_coefficient__attr_"real"__coeff_47',	'az_fft_coefficient__attr_"real"__coeff_48',	'az_fft_coefficient__attr_"real"__coeff_49',	'az_fft_coefficient__attr_"real"__coeff_50',	'az_fft_coefficient__attr_"real"__coeff_51',	'az_fft_coefficient__attr_"real"__coeff_52',	'az_fft_coefficient__attr_"real"__coeff_53',	'az_fft_coefficient__attr_"real"__coeff_54',	'az_fft_coefficient__attr_"real"__coeff_55',	'az_fft_coefficient__attr_"real"__coeff_56',	'az_fft_coefficient__attr_"real"__coeff_57',	'az_fft_coefficient__attr_"real"__coeff_58',	'az_fft_coefficient__attr_"real"__coeff_59',	'az_fft_coefficient__attr_"real"__coeff_60',	'az_fft_coefficient__attr_"real"__coeff_61',	'az_fft_coefficient__attr_"real"__coeff_62',	'az_fft_coefficient__attr_"real"__coeff_63',	'az_fft_coefficient__attr_"real"__coeff_64',	'az_fft_coefficient__attr_"real"__coeff_65',	'az_fft_coefficient__attr_"real"__coeff_66',	'az_fft_coefficient__attr_"real"__coeff_67',	'az_fft_coefficient__attr_"real"__coeff_68',	'az_fft_coefficient__attr_"real"__coeff_69',	'az_fft_coefficient__attr_"real"__coeff_70',	'az_fft_coefficient__attr_"real"__coeff_71',	'az_fft_coefficient__attr_"real"__coeff_72',	'az_fft_coefficient__attr_"real"__coeff_73',	'az_fft_coefficient__attr_"real"__coeff_74',	'az_fft_coefficient__attr_"real"__coeff_75',	'az_fft_coefficient__attr_"real"__coeff_76',	'az_fft_coefficient__attr_"real"__coeff_77',	'az_fft_coefficient__attr_"real"__coeff_78',	'az_fft_coefficient__attr_"real"__coeff_79',	'az_fft_coefficient__attr_"real"__coeff_80',	'az_fft_coefficient__attr_"real"__coeff_81',	'az_fft_coefficient__attr_"real"__coeff_82',	'az_fft_coefficient__attr_"real"__coeff_83',	'az_fft_coefficient__attr_"real"__coeff_84',	'az_fft_coefficient__attr_"real"__coeff_85',	'az_fft_coefficient__attr_"real"__coeff_86',	'az_fft_coefficient__attr_"real"__coeff_87',	'az_fft_coefficient__attr_"real"__coeff_88',	'az_fft_coefficient__attr_"real"__coeff_89',	'az_fft_coefficient__attr_"real"__coeff_90',	'az_fft_coefficient__attr_"real"__coeff_91',	'az_fft_coefficient__attr_"real"__coeff_92',	'az_fft_coefficient__attr_"real"__coeff_93',	'az_fft_coefficient__attr_"real"__coeff_94',	'az_fft_coefficient__attr_"real"__coeff_95',	'az_fft_coefficient__attr_"real"__coeff_96',	'az_fft_coefficient__attr_"real"__coeff_97',	'az_fft_coefficient__attr_"real"__coeff_98',	'az_fft_coefficient__attr_"real"__coeff_99',	'az_fft_coefficient__attr_"imag"__coeff_0',	'az_fft_coefficient__attr_"imag"__coeff_1',	'az_fft_coefficient__attr_"imag"__coeff_2',	'az_fft_coefficient__attr_"imag"__coeff_3',	'az_fft_coefficient__attr_"imag"__coeff_4',	'az_fft_coefficient__attr_"imag"__coeff_5',	'az_fft_coefficient__attr_"imag"__coeff_6',	'az_fft_coefficient__attr_"imag"__coeff_7',	'az_fft_coefficient__attr_"imag"__coeff_8',	'az_fft_coefficient__attr_"imag"__coeff_9',	'az_fft_coefficient__attr_"imag"__coeff_10',	'az_fft_coefficient__attr_"imag"__coeff_11',	'az_fft_coefficient__attr_"imag"__coeff_12',	'az_fft_coefficient__attr_"imag"__coeff_13',	'az_fft_coefficient__attr_"imag"__coeff_14',	'az_fft_coefficient__attr_"imag"__coeff_15',	'az_fft_coefficient__attr_"imag"__coeff_16',	'az_fft_coefficient__attr_"imag"__coeff_17',	'az_fft_coefficient__attr_"imag"__coeff_18',	'az_fft_coefficient__attr_"imag"__coeff_19',	'az_fft_coefficient__attr_"imag"__coeff_20',	'az_fft_coefficient__attr_"imag"__coeff_21',	'az_fft_coefficient__attr_"imag"__coeff_22',	'az_fft_coefficient__attr_"imag"__coeff_23',	'az_fft_coefficient__attr_"imag"__coeff_24',	'az_fft_coefficient__attr_"imag"__coeff_25',	'az_fft_coefficient__attr_"imag"__coeff_26',	'az_fft_coefficient__attr_"imag"__coeff_27',	'az_fft_coefficient__attr_"imag"__coeff_28',	'az_fft_coefficient__attr_"imag"__coeff_29',	'az_fft_coefficient__attr_"imag"__coeff_30',	'az_fft_coefficient__attr_"imag"__coeff_31',	'az_fft_coefficient__attr_"imag"__coeff_32',	'az_fft_coefficient__attr_"imag"__coeff_33',	'az_fft_coefficient__attr_"imag"__coeff_34',	'az_fft_coefficient__attr_"imag"__coeff_35',	'az_fft_coefficient__attr_"imag"__coeff_36',	'az_fft_coefficient__attr_"imag"__coeff_37',	'az_fft_coefficient__attr_"imag"__coeff_38',	'az_fft_coefficient__attr_"imag"__coeff_39',	'az_fft_coefficient__attr_"imag"__coeff_40',	'az_fft_coefficient__attr_"imag"__coeff_41',	'az_fft_coefficient__attr_"imag"__coeff_42',	'az_fft_coefficient__attr_"imag"__coeff_43',	'az_fft_coefficient__attr_"imag"__coeff_44',	'az_fft_coefficient__attr_"imag"__coeff_45',	'az_fft_coefficient__attr_"imag"__coeff_46',	'az_fft_coefficient__attr_"imag"__coeff_47',	'az_fft_coefficient__attr_"imag"__coeff_48',	'az_fft_coefficient__attr_"imag"__coeff_49',	'az_fft_coefficient__attr_"imag"__coeff_50',	'az_fft_coefficient__attr_"imag"__coeff_51',	'az_fft_coefficient__attr_"imag"__coeff_52',	'az_fft_coefficient__attr_"imag"__coeff_53',	'az_fft_coefficient__attr_"imag"__coeff_54',	'az_fft_coefficient__attr_"imag"__coeff_55',	'az_fft_coefficient__attr_"imag"__coeff_56',	'az_fft_coefficient__attr_"imag"__coeff_57',	'az_fft_coefficient__attr_"imag"__coeff_58',	'az_fft_coefficient__attr_"imag"__coeff_59',	'az_fft_coefficient__attr_"imag"__coeff_60',	'az_fft_coefficient__attr_"imag"__coeff_61',	'az_fft_coefficient__attr_"imag"__coeff_62',	'az_fft_coefficient__attr_"imag"__coeff_63',	'az_fft_coefficient__attr_"imag"__coeff_64',	'az_fft_coefficient__attr_"imag"__coeff_65',	'az_fft_coefficient__attr_"imag"__coeff_66',	'az_fft_coefficient__attr_"imag"__coeff_67',	'az_fft_coefficient__attr_"imag"__coeff_68',	'az_fft_coefficient__attr_"imag"__coeff_69',	'az_fft_coefficient__attr_"imag"__coeff_70',	'az_fft_coefficient__attr_"imag"__coeff_71',	'az_fft_coefficient__attr_"imag"__coeff_72',	'az_fft_coefficient__attr_"imag"__coeff_73',	'az_fft_coefficient__attr_"imag"__coeff_74',	'az_fft_coefficient__attr_"imag"__coeff_75',	'az_fft_coefficient__attr_"imag"__coeff_76',	'az_fft_coefficient__attr_"imag"__coeff_77',	'az_fft_coefficient__attr_"imag"__coeff_78',	'az_fft_coefficient__attr_"imag"__coeff_79',	'az_fft_coefficient__attr_"imag"__coeff_80',	'az_fft_coefficient__attr_"imag"__coeff_81',	'az_fft_coefficient__attr_"imag"__coeff_82',	'az_fft_coefficient__attr_"imag"__coeff_83',	'az_fft_coefficient__attr_"imag"__coeff_84',	'az_fft_coefficient__attr_"imag"__coeff_85',	'az_fft_coefficient__attr_"imag"__coeff_86',	'az_fft_coefficient__attr_"imag"__coeff_87',	'az_fft_coefficient__attr_"imag"__coeff_88',	'az_fft_coefficient__attr_"imag"__coeff_89',	'az_fft_coefficient__attr_"imag"__coeff_90',	'az_fft_coefficient__attr_"imag"__coeff_91',	'az_fft_coefficient__attr_"imag"__coeff_92',	'az_fft_coefficient__attr_"imag"__coeff_93',	'az_fft_coefficient__attr_"imag"__coeff_94',	'az_fft_coefficient__attr_"imag"__coeff_95',	'az_fft_coefficient__attr_"imag"__coeff_96',	'az_fft_coefficient__attr_"imag"__coeff_97',	'az_fft_coefficient__attr_"imag"__coeff_98',	'az_fft_coefficient__attr_"imag"__coeff_99',	'az_fft_coefficient__attr_"abs"__coeff_0',	'az_fft_coefficient__attr_"abs"__coeff_1',	'az_fft_coefficient__attr_"abs"__coeff_2',	'az_fft_coefficient__attr_"abs"__coeff_3',	'az_fft_coefficient__attr_"abs"__coeff_4',	'az_fft_coefficient__attr_"abs"__coeff_5',	'az_fft_coefficient__attr_"abs"__coeff_6',	'az_fft_coefficient__attr_"abs"__coeff_7',	'az_fft_coefficient__attr_"abs"__coeff_8',	'az_fft_coefficient__attr_"abs"__coeff_9',	'az_fft_coefficient__attr_"abs"__coeff_10',	'az_fft_coefficient__attr_"abs"__coeff_11',	'az_fft_coefficient__attr_"abs"__coeff_12',	'az_fft_coefficient__attr_"abs"__coeff_13',	'az_fft_coefficient__attr_"abs"__coeff_14',	'az_fft_coefficient__attr_"abs"__coeff_15',	'az_fft_coefficient__attr_"abs"__coeff_16',	'az_fft_coefficient__attr_"abs"__coeff_17',	'az_fft_coefficient__attr_"abs"__coeff_18',	'az_fft_coefficient__attr_"abs"__coeff_19',	'az_fft_coefficient__attr_"abs"__coeff_20',	'az_fft_coefficient__attr_"abs"__coeff_21',	'az_fft_coefficient__attr_"abs"__coeff_22',	'az_fft_coefficient__attr_"abs"__coeff_23',	'az_fft_coefficient__attr_"abs"__coeff_24',	'az_fft_coefficient__attr_"abs"__coeff_25',	'az_fft_coefficient__attr_"abs"__coeff_26',	'az_fft_coefficient__attr_"abs"__coeff_27',	'az_fft_coefficient__attr_"abs"__coeff_28',	'az_fft_coefficient__attr_"abs"__coeff_29',	'az_fft_coefficient__attr_"abs"__coeff_30',	'az_fft_coefficient__attr_"abs"__coeff_31',	'az_fft_coefficient__attr_"abs"__coeff_32',	'az_fft_coefficient__attr_"abs"__coeff_33',	'az_fft_coefficient__attr_"abs"__coeff_34',	'az_fft_coefficient__attr_"abs"__coeff_35',	'az_fft_coefficient__attr_"abs"__coeff_36',	'az_fft_coefficient__attr_"abs"__coeff_37',	'az_fft_coefficient__attr_"abs"__coeff_38',	'az_fft_coefficient__attr_"abs"__coeff_39',	'az_fft_coefficient__attr_"abs"__coeff_40',	'az_fft_coefficient__attr_"abs"__coeff_41',	'az_fft_coefficient__attr_"abs"__coeff_42',	'az_fft_coefficient__attr_"abs"__coeff_43',	'az_fft_coefficient__attr_"abs"__coeff_44',	'az_fft_coefficient__attr_"abs"__coeff_45',	'az_fft_coefficient__attr_"abs"__coeff_46',	'az_fft_coefficient__attr_"abs"__coeff_47',	'az_fft_coefficient__attr_"abs"__coeff_48',	'az_fft_coefficient__attr_"abs"__coeff_49',	'az_fft_coefficient__attr_"abs"__coeff_50',	'az_fft_coefficient__attr_"abs"__coeff_51',	'az_fft_coefficient__attr_"abs"__coeff_52',	'az_fft_coefficient__attr_"abs"__coeff_53',	'az_fft_coefficient__attr_"abs"__coeff_54',	'az_fft_coefficient__attr_"abs"__coeff_55',	'az_fft_coefficient__attr_"abs"__coeff_56',	'az_fft_coefficient__attr_"abs"__coeff_57',	'az_fft_coefficient__attr_"abs"__coeff_58',	'az_fft_coefficient__attr_"abs"__coeff_59',	'az_fft_coefficient__attr_"abs"__coeff_60',	'az_fft_coefficient__attr_"abs"__coeff_61',	'az_fft_coefficient__attr_"abs"__coeff_62',	'az_fft_coefficient__attr_"abs"__coeff_63',	'az_fft_coefficient__attr_"abs"__coeff_64',	'az_fft_coefficient__attr_"abs"__coeff_65',	'az_fft_coefficient__attr_"abs"__coeff_66',	'az_fft_coefficient__attr_"abs"__coeff_67',	'az_fft_coefficient__attr_"abs"__coeff_68',	'az_fft_coefficient__attr_"abs"__coeff_69',	'az_fft_coefficient__attr_"abs"__coeff_70',	'az_fft_coefficient__attr_"abs"__coeff_71',	'az_fft_coefficient__attr_"abs"__coeff_72',	'az_fft_coefficient__attr_"abs"__coeff_73',	'az_fft_coefficient__attr_"abs"__coeff_74',	'az_fft_coefficient__attr_"abs"__coeff_75',	'az_fft_coefficient__attr_"abs"__coeff_76',	'az_fft_coefficient__attr_"abs"__coeff_77',	'az_fft_coefficient__attr_"abs"__coeff_78',	'az_fft_coefficient__attr_"abs"__coeff_79',	'az_fft_coefficient__attr_"abs"__coeff_80',	'az_fft_coefficient__attr_"abs"__coeff_81',	'az_fft_coefficient__attr_"abs"__coeff_82',	'az_fft_coefficient__attr_"abs"__coeff_83',	'az_fft_coefficient__attr_"abs"__coeff_84',	'az_fft_coefficient__attr_"abs"__coeff_85',	'az_fft_coefficient__attr_"abs"__coeff_86',	'az_fft_coefficient__attr_"abs"__coeff_87',	'az_fft_coefficient__attr_"abs"__coeff_88',	'az_fft_coefficient__attr_"abs"__coeff_89',	'az_fft_coefficient__attr_"abs"__coeff_90',	'az_fft_coefficient__attr_"abs"__coeff_91',	'az_fft_coefficient__attr_"abs"__coeff_92',	'az_fft_coefficient__attr_"abs"__coeff_93',	'az_fft_coefficient__attr_"abs"__coeff_94',	'az_fft_coefficient__attr_"abs"__coeff_95',	'az_fft_coefficient__attr_"abs"__coeff_96',	'az_fft_coefficient__attr_"abs"__coeff_97',	'az_fft_coefficient__attr_"abs"__coeff_98',	'az_fft_coefficient__attr_"abs"__coeff_99',	'az_fft_coefficient__attr_"angle"__coeff_0',	'az_fft_coefficient__attr_"angle"__coeff_1',	'az_fft_coefficient__attr_"angle"__coeff_2',	'az_fft_coefficient__attr_"angle"__coeff_3',	'az_fft_coefficient__attr_"angle"__coeff_4',	'az_fft_coefficient__attr_"angle"__coeff_5',	'az_fft_coefficient__attr_"angle"__coeff_6',	'az_fft_coefficient__attr_"angle"__coeff_7',	'az_fft_coefficient__attr_"angle"__coeff_8',	'az_fft_coefficient__attr_"angle"__coeff_9',	'az_fft_coefficient__attr_"angle"__coeff_10',	'az_fft_coefficient__attr_"angle"__coeff_11',	'az_fft_coefficient__attr_"angle"__coeff_12',	'az_fft_coefficient__attr_"angle"__coeff_13',	'az_fft_coefficient__attr_"angle"__coeff_14',	'az_fft_coefficient__attr_"angle"__coeff_15',	'az_fft_coefficient__attr_"angle"__coeff_16',	'az_fft_coefficient__attr_"angle"__coeff_17',	'az_fft_coefficient__attr_"angle"__coeff_18',	'az_fft_coefficient__attr_"angle"__coeff_19',	'az_fft_coefficient__attr_"angle"__coeff_20',	'az_fft_coefficient__attr_"angle"__coeff_21',	'az_fft_coefficient__attr_"angle"__coeff_22',	'az_fft_coefficient__attr_"angle"__coeff_23',	'az_fft_coefficient__attr_"angle"__coeff_24',	'az_fft_coefficient__attr_"angle"__coeff_25',	'az_fft_coefficient__attr_"angle"__coeff_26',	'az_fft_coefficient__attr_"angle"__coeff_27',	'az_fft_coefficient__attr_"angle"__coeff_28',	'az_fft_coefficient__attr_"angle"__coeff_29',	'az_fft_coefficient__attr_"angle"__coeff_30',	'az_fft_coefficient__attr_"angle"__coeff_31',	'az_fft_coefficient__attr_"angle"__coeff_32',	'az_fft_coefficient__attr_"angle"__coeff_33',	'az_fft_coefficient__attr_"angle"__coeff_34',	'az_fft_coefficient__attr_"angle"__coeff_35',	'az_fft_coefficient__attr_"angle"__coeff_36',	'az_fft_coefficient__attr_"angle"__coeff_37',	'az_fft_coefficient__attr_"angle"__coeff_38',	'az_fft_coefficient__attr_"angle"__coeff_39',	'az_fft_coefficient__attr_"angle"__coeff_40',	'az_fft_coefficient__attr_"angle"__coeff_41',	'az_fft_coefficient__attr_"angle"__coeff_42',	'az_fft_coefficient__attr_"angle"__coeff_43',	'az_fft_coefficient__attr_"angle"__coeff_44',	'az_fft_coefficient__attr_"angle"__coeff_45',	'az_fft_coefficient__attr_"angle"__coeff_46',	'az_fft_coefficient__attr_"angle"__coeff_47',	'az_fft_coefficient__attr_"angle"__coeff_48',	'az_fft_coefficient__attr_"angle"__coeff_49',	'az_fft_coefficient__attr_"angle"__coeff_50',	'az_fft_coefficient__attr_"angle"__coeff_51',	'az_fft_coefficient__attr_"angle"__coeff_52',	'az_fft_coefficient__attr_"angle"__coeff_53',	'az_fft_coefficient__attr_"angle"__coeff_54',	'az_fft_coefficient__attr_"angle"__coeff_55',	'az_fft_coefficient__attr_"angle"__coeff_56',	'az_fft_coefficient__attr_"angle"__coeff_57',	'az_fft_coefficient__attr_"angle"__coeff_58',	'az_fft_coefficient__attr_"angle"__coeff_59',	'az_fft_coefficient__attr_"angle"__coeff_60',	'az_fft_coefficient__attr_"angle"__coeff_61',	'az_fft_coefficient__attr_"angle"__coeff_62',	'az_fft_coefficient__attr_"angle"__coeff_63',	'az_fft_coefficient__attr_"angle"__coeff_64',	'az_fft_coefficient__attr_"angle"__coeff_65',	'az_fft_coefficient__attr_"angle"__coeff_66',	'az_fft_coefficient__attr_"angle"__coeff_67',	'az_fft_coefficient__attr_"angle"__coeff_68',	'az_fft_coefficient__attr_"angle"__coeff_69',	'az_fft_coefficient__attr_"angle"__coeff_70',	'az_fft_coefficient__attr_"angle"__coeff_71',	'az_fft_coefficient__attr_"angle"__coeff_72',	'az_fft_coefficient__attr_"angle"__coeff_73',	'az_fft_coefficient__attr_"angle"__coeff_74',	'az_fft_coefficient__attr_"angle"__coeff_75',	'az_fft_coefficient__attr_"angle"__coeff_76',	'az_fft_coefficient__attr_"angle"__coeff_77',	'az_fft_coefficient__attr_"angle"__coeff_78',	'az_fft_coefficient__attr_"angle"__coeff_79',	'az_fft_coefficient__attr_"angle"__coeff_80',	'az_fft_coefficient__attr_"angle"__coeff_81',	'az_fft_coefficient__attr_"angle"__coeff_82',	'az_fft_coefficient__attr_"angle"__coeff_83',	'az_fft_coefficient__attr_"angle"__coeff_84',	'az_fft_coefficient__attr_"angle"__coeff_85',	'az_fft_coefficient__attr_"angle"__coeff_86',	'az_fft_coefficient__attr_"angle"__coeff_87',	'az_fft_coefficient__attr_"angle"__coeff_88',	'az_fft_coefficient__attr_"angle"__coeff_89',	'az_fft_coefficient__attr_"angle"__coeff_90',	'az_fft_coefficient__attr_"angle"__coeff_91',	'az_fft_coefficient__attr_"angle"__coeff_92',	'az_fft_coefficient__attr_"angle"__coeff_93',	'az_fft_coefficient__attr_"angle"__coeff_94',	'az_fft_coefficient__attr_"angle"__coeff_95',	'az_fft_coefficient__attr_"angle"__coeff_96',	'az_fft_coefficient__attr_"angle"__coeff_97',	'az_fft_coefficient__attr_"angle"__coeff_98',	'az_fft_coefficient__attr_"angle"__coeff_99',	'az_fft_aggregated__aggtype_"centroid"',	'az_fft_aggregated__aggtype_"variance"',	'az_fft_aggregated__aggtype_"skew"',	'az_fft_aggregated__aggtype_"kurtosis"',	'az_value_count__value_0',	'az_value_count__value_1',	'az_value_count__value_-1',	'az_range_count__maz_min_-1',	'az_range_count__maz_0__min_1000000000000.0',	'az_range_count__maz_1000000000000.0__min_0',	'az_approximate_entropy__m_2__r_0.1',	'az_approximate_entropy__m_2__r_0.3',	'az_approximate_entropy__m_2__r_0.5',	'az_approximate_entropy__m_2__r_0.7',	'az_approximate_entropy__m_2__r_0.9',	'az_friedrich_coefficients__coeff_0__m_3__r_30',	'az_friedrich_coefficients__coeff_m_3__r_30',	'az_friedrich_coefficients__coeff_2__m_3__r_30',	'az_friedrich_coefficients__coeff_3__m_3__r_30',	'az_maz_langevin_fixed_point__m_3__r_30',	'az_linear_trend__attr_"pvalue"',	'az_linear_trend__attr_"rvalue"',	'az_linear_trend__attr_"intercept"',	'az_linear_trend__attr_"slope"',	'az_linear_trend__attr_"stderr"',	'az_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"maz"',	'az_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"min"',	'az_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"mean"',	'az_agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"var"',	'az_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"maz"',	'az_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"min"',	'az_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"mean"',	'az_agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"var"',	'az_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"maz"',	'az_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"min"',	'az_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"mean"',	'az_agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"var"',	'az_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"maz"',	'az_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"min"',	'az_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"mean"',	'az_agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"var"',	'az_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"maz"',	'az_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"min"',	'az_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"mean"',	'az_agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"var"',	'az_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"maz"',	'az_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"min"',	'az_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"mean"',	'az_agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"var"',	'az_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"maz"',	'az_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"min"',	'az_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"mean"',	'az_agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"var"',	'az_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"maz"',	'az_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"min"',	'az_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"mean"',	'az_agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"var"',	'az_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"maz"',	'az_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"min"',	'az_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"mean"',	'az_agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"var"',	'az_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"maz"',	'az_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"min"',	'az_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"mean"',	'az_agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"var"',	'az_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"maz"',	'az_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"min"',	'az_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"mean"',	'az_agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"var"',	'az_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"maz"',	'az_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"min"',	'az_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"mean"',	'az_agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"var"',	'az_augmented_dickey_fuller__attr_"teststat"__autolag_"AIC"',	'az_augmented_dickey_fuller__attr_"pvalue"__autolag_"AIC"',	'az_augmented_dickey_fuller__attr_"usedlag"__autolag_"AIC"',	'az_number_crossing_m__m_0',	'az_number_crossing_m__m_-1',	'az_number_crossing_m__m_1',	'az_eneraz_ratio_by_chunks__num_segments_10__segment_focus_0',	'az_eneraz_ratio_by_chunks__num_segments_10__segment_focus_1',	'az_eneraz_ratio_by_chunks__num_segments_10__segment_focus_2',	'az_eneraz_ratio_by_chunks__num_segments_10__segment_focus_3',	'az_eneraz_ratio_by_chunks__num_segments_10__segment_focus_4',	'az_eneraz_ratio_by_chunks__num_segments_10__segment_focus_5',	'az_eneraz_ratio_by_chunks__num_segments_10__segment_focus_6',	'az_eneraz_ratio_by_chunks__num_segments_10__segment_focus_7',	'az_eneraz_ratio_by_chunks__num_segments_10__segment_focus_8',	'az_eneraz_ratio_by_chunks__num_segments_10__segment_focus_9',	'az_ratio_beyond_r_sigma__r_0.5',	'az_ratio_beyond_r_sigma__r_1',	'az_ratio_beyond_r_sigma__r_1.5',	'az_ratio_beyond_r_sigma__r_2',	'az_ratio_beyond_r_sigma__r_2.5',	'az_ratio_beyond_r_sigma__r_3',	'az_ratio_beyond_r_sigma__r_5',	'az_ratio_beyond_r_sigma__r_6',	'az_ratio_beyond_r_sigma__r_7',	'az_ratio_beyond_r_sigma__r_10',	'az_count_above__t_0',	'az_count_below__t_0',	'az_lempel_ziv_complexity__bins_2',	'az_lempel_ziv_complexity__bins_3',	'az_lempel_ziv_complexity__bins_5',	'az_lempel_ziv_complexity__bins_10',	'az_lempel_ziv_complexity__bins_100',	'az_fourier_entropy__bins_2',	'az_fourier_entropy__bins_3',	'az_fourier_entropy__bins_5',	'az_fourier_entropy__bins_10',	'az_fourier_entropy__bins_100',	'az_permutation_entropy__dimension_3__tau_1',	'az_permutation_entropy__dimension_4__tau_1',	'az_permutation_entropy__dimension_5__tau_1',	'az_permutation_entropy__dimension_6__tau_1',	'az_permutation_entropy__dimension_7__tau_1',	'az_query_similarity_count__query_None__threshold_0.0',	'az_matrix_profile__feature_"min"__threshold_0.98',	'az_matrix_profile__feature_"maz"__threshold_0.98',	'az_matrix_profile__feature_"mean"__threshold_0.98',	'az_matrix_profile__feature_"median"__threshold_0.98',	'az_matrix_profile__feature_"25"__threshold_0.98',	'az_matrix_profile__feature_"75"__threshold_0.98']
fea_user_1az=pd.DataFrame(fea_user_1az.reshape(Gx_list_trans.shape[1], 787))
fea_user_1az.columns=[header_az]
fea_user_1az

,az_variance_larger_than_standard_deviation,az_has_duplicate_max,az_has_duplicate_min,az_has_duplicate,az_sum_values,az_abs_eneraz,az_mean_abs_change,az_mean_change,az_mean_second_derivative_central,az_median,...,az_permutation_entropy__dimension_5__tau_1,az_permutation_entropy__dimension_6__tau_1,az_permutation_entropy__dimension_7__tau_1,az_query_similarity_count__query_None__threshold_0.0,"az_matrix_profile__feature_""min""__threshold_0.98","az_matrix_profile__feature_""maz""__threshold_0.98","az_matrix_profile__feature_""mean""__threshold_0.98","az_matrix_profile__feature_""median""__threshold_0.98","az_matrix_profile__feature_""25""__threshold_0.98","az_matrix_profile__feature_""75""__threshold_0.98"
0,0.0,0.0,0.0,0.0,-0.8183,1.134359,0.139758,-0.039053,0.017692,-0.05835,...,2.772589,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,-0.5174,0.169530,0.064216,-0.004605,-0.002883,-0.00365,...,2.685945,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,0.0,-1.0395,0.207154,0.068689,-0.009489,0.005272,-0.04645,...,2.685945,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,0.0,0.0,-1.0591,0.236070,0.084005,0.002037,0.000858,-0.05620,...,2.599302,2.615631,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,0.0,0.0,-0.6537,0.122481,0.069879,0.001300,0.006572,-0.03675,...,2.566599,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,1.0,0.0,0.0,0.0,1.5951,31.968295,1.062474,-0.009116,-0.011592,0.03685,...,2.599302,2.615631,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
805,0.0,0.0,0.0,0.0,0.5834,6.529107,0.387979,0.099611,-0.051947,0.01300,...,2.187322,2.395908,2.540036,NaN,NaN,NaN,NaN,NaN,NaN,NaN
806,0.0,0.0,0.0,0.0,1.4195,0.363328,0.083384,0.006995,-0.008944,0.03220,...,2.685945,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
807,0.0,0.0,0.0,0.0,1.6065,0.336482,0.072989,0.007716,-0.003319,0.06000,...,2.772589,2.708050,2.639057,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
r_az=fea_user_1az.dropna(axis='columns',how='all')
r_az=r_az.fillna(0)
binary_features_az=['az_variance_larger_than_standard_deviation','az_has_duplicate_max',	'az_has_duplicate_min',	'az_has_duplicate','az_symmetry_looking__r_0.0',	'az_symmetry_looking__r_0.05',	'az_symmetry_looking__r_0.1',	'az_symmetry_looking__r_0.15000000000000002',	'az_symmetry_looking__r_0.2',	'az_symmetry_looking__r_0.25',	'az_symmetry_looking__r_0.30000000000000004',	'az_symmetry_looking__r_0.35000000000000003',	'az_symmetry_looking__r_0.4',	'az_symmetry_looking__r_0.45',	'az_symmetry_looking__r_0.5',	'az_symmetry_looking__r_0.55',	'az_symmetry_looking__r_0.6000000000000001',	'az_symmetry_looking__r_0.65',	'az_symmetry_looking__r_0.7000000000000001',	'az_symmetry_looking__r_0.75',	'az_symmetry_looking__r_0.8',	'az_symmetry_looking__r_0.8500000000000001',	'az_symmetry_looking__r_0.9',	'az_symmetry_looking__r_0.9500000000000001',	'az_large_standard_deviation__r_0.05',	'az_large_standard_deviation__r_0.1',	'az_large_standard_deviation__r_0.15000000000000002',	'az_large_standard_deviation__r_0.2',	'az_large_standard_deviation__r_0.25',	'az_large_standard_deviation__r_0.30000000000000004',	'az_large_standard_deviation__r_0.35000000000000003',	'az_large_standard_deviation__r_0.4',	'az_large_standard_deviation__r_0.45',	'az_large_standard_deviation__r_0.5',	'az_large_standard_deviation__r_0.55',	'az_large_standard_deviation__r_0.6000000000000001',	'az_large_standard_deviation__r_0.65',	'az_large_standard_deviation__r_0.7000000000000001',	'az_large_standard_deviation__r_0.75',	'az_large_standard_deviation__r_0.8',	'az_large_standard_deviation__r_0.8500000000000001',	'az_large_standard_deviation__r_0.9',	'az_large_standard_deviation__r_0.9500000000000001','az_value_count__value_0',	'az_value_count__value_1',	'az_value_count__value_-1']
r_az=r_az.drop(binary_features_az, axis=1)
r_az

d:\m-tech-data-science\tools\python 3.7\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,az_sum_values,az_abs_eneraz,az_mean_abs_change,az_mean_change,az_mean_second_derivative_central,az_median,az_mean,az_length,az_standard_deviation,az_variation_coefficient,...,az_permutation_entropy__dimension_4__tau_1,az_permutation_entropy__dimension_5__tau_1,az_permutation_entropy__dimension_6__tau_1,az_permutation_entropy__dimension_7__tau_1,"az_matrix_profile__feature_""min""__threshold_0.98","az_matrix_profile__feature_""maz""__threshold_0.98","az_matrix_profile__feature_""mean""__threshold_0.98","az_matrix_profile__feature_""median""__threshold_0.98","az_matrix_profile__feature_""25""__threshold_0.98","az_matrix_profile__feature_""75""__threshold_0.98"
0,-0.8183,1.134359,0.139758,-0.039053,0.017692,-0.05835,-0.040915,20.0,0.234614,-5.734190,...,2.588573,2.772589,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.5174,0.169530,0.064216,-0.004605,-0.002883,-0.00365,-0.025870,20.0,0.088359,-3.415483,...,2.588573,2.685945,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
2,-1.0395,0.207154,0.068689,-0.009489,0.005272,-0.04645,-0.051975,20.0,0.087500,-1.683509,...,2.394700,2.685945,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
3,-1.0591,0.236070,0.084005,0.002037,0.000858,-0.05620,-0.052955,20.0,0.094864,-1.791415,...,2.282374,2.599302,2.615631,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.6537,0.122481,0.069879,0.001300,0.006572,-0.03675,-0.032685,20.0,0.071104,-2.175424,...,2.115209,2.566599,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,1.5951,31.968295,1.062474,-0.009116,-0.011592,0.03685,0.079755,20.0,1.261766,15.820528,...,2.200828,2.599302,2.615631,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
805,0.5834,6.529107,0.387979,0.099611,-0.051947,0.01300,0.029170,20.0,0.570618,19.561797,...,1.874641,2.187322,2.395908,2.540036,0.0,0.0,0.0,0.0,0.0,0.0
806,1.4195,0.363328,0.083384,0.006995,-0.008944,0.03220,0.070975,20.0,0.114582,1.614396,...,2.200828,2.685945,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
807,1.6065,0.336482,0.072989,0.007716,-0.003319,0.06000,0.080325,20.0,0.101843,1.267887,...,2.394700,2.772589,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
user1_train_features=pd.concat([r_gx, r_gy,r_gz,r_ax,r_ay,r_az], axis=1)
user1_train_features

,gx_sum_values,gx_abs_energy,gx_mean_abs_change,gx_mean_change,gx_mean_second_derivative_central,gx_median,gx_mean,gx_length,gx_standard_deviation,gx_variation_coefficient,...,az_permutation_entropy__dimension_4__tau_1,az_permutation_entropy__dimension_5__tau_1,az_permutation_entropy__dimension_6__tau_1,az_permutation_entropy__dimension_7__tau_1,"az_matrix_profile__feature_""min""__threshold_0.98","az_matrix_profile__feature_""maz""__threshold_0.98","az_matrix_profile__feature_""mean""__threshold_0.98","az_matrix_profile__feature_""median""__threshold_0.98","az_matrix_profile__feature_""25""__threshold_0.98","az_matrix_profile__feature_""75""__threshold_0.98"
0,0.9613,0.046208,0.000153,-0.000047,0.000003,0.04810,0.048065,20.0,0.000362,0.007538,...,2.588573,2.772589,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
1,0.9596,0.046043,0.000153,0.000026,0.000006,0.04800,0.047980,20.0,0.000268,0.005577,...,2.588573,2.685945,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
2,0.9666,0.046721,0.000168,0.000095,0.000003,0.04815,0.048330,20.0,0.000516,0.010673,...,2.394700,2.685945,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
3,0.9902,0.049046,0.000226,0.000111,0.000003,0.04970,0.049510,20.0,0.001024,0.020676,...,2.282374,2.599302,2.615631,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0104,0.051048,0.000174,0.000079,-0.000008,0.05040,0.050520,20.0,0.000392,0.007758,...,2.115209,2.566599,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,-0.2141,0.002301,0.000279,-0.000005,0.000006,-0.01050,-0.010705,20.0,0.000668,-0.062418,...,2.200828,2.599302,2.615631,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
805,-0.1986,0.001979,0.000158,0.000095,0.000003,-0.01000,-0.009930,20.0,0.000589,-0.059331,...,1.874641,2.187322,2.395908,2.540036,0.0,0.0,0.0,0.0,0.0,0.0
806,-0.1850,0.001715,0.000158,0.000063,0.000003,-0.00915,-0.009250,20.0,0.000451,-0.048769,...,2.200828,2.685945,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0
807,-0.1634,0.001357,0.000216,0.000089,-0.000003,-0.00860,-0.008170,20.0,0.001038,-0.127089,...,2.394700,2.772589,2.708050,2.639057,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
user1_train_features.to_csv(r'C:\Users\hp\Desktop\Feature_sensor\features\user9\train\user9_train_features.csv')